# 使用LLaMA Factory微调Qwen3模型

[LLaMA Factory](https://github.com/hiyouga/LLaMA-Factory)是一款开源低代码大模型微调框架，集成了业界最广泛使用的微调技术，支持通过Web UI界面零代码微调大模型，目前已经成为开源社区内最受欢迎的微调框架，GitHub星标超过2万。
本教程将基于开源的Qwen3 8B模型，介绍如何使用PAI平台及LLaMA Factory训练框架完成中文医疗领域的微调和评估。

## 运行环境要求

### 1. 阿里云 PAI-DSW 环境配置

本教程在阿里云 PAI-DSW 平台上开发和测试，推荐使用以下配置：

| 配置项 | 规格说明 | 备注 |
|--------|---------|------|
| **GPU 类型** | NVIDIA A10 | 实例规格：`ecs.gn7i-c8g1.2xlarge` |
| **GPU 显存** | 24GB | 用于加载模型和训练数据 |
| **官方镜像** | `pytorch:2.6.0-gpu-py310-cu124-ubuntu22.04` | PAI-DSW 预配置镜像 |
| **操作系统** | Ubuntu 22.04 LTS | 长期支持版本 |
| **CUDA 版本** | 12.4 | 与 PyTorch 2.6.0 兼容 |
| **Python 版本** | 3.10.x | 推荐使用 Python 3.10 或 3.11 |
| **PyTorch 版本** | 2.6.0 | 深度学习框架 |
| **磁盘空间** | ≥ 100GB | 用于存储模型、数据集和检查点 |

### 2. 私有环境部署要求

如果您需要在本地服务器或私有云环境中部署，请确保满足以下软硬件要求：

#### 2.1 硬件要求

| 硬件配置 | 最低要求 | 推荐配置 | 说明 |
|---------|---------|---------|------|
| **GPU** | NVIDIA RTX 3090 (24GB) | NVIDIA A10/A100 (≥24GB) | 必须支持 CUDA，显存至少 24GB |
| **GPU 显存** | 24GB | 40GB+ | Qwen3 8B 模型使用 LoRA 微调需要约 20-24GB |
| **CPU** | 8核心 | 16核心+ | 用于数据预处理和模型推理 |
| **内存 (RAM)** | 32GB | 64GB+ | 避免数据加载时的内存溢出 |
| **磁盘空间** | 100GB | 500GB+ SSD | 模型文件约 15GB，数据集和日志需额外空间 |

#### 2.2 软件要求

| 软件组件 | 版本要求 | 安装命令/说明 |
|---------|---------|--------------|
| **操作系统** | Ubuntu 20.04 / 22.04 LTS<br>CentOS 7/8<br>Windows 10/11 (WSL2) | 推荐使用 Ubuntu 22.04 LTS |
| **Python** | 3.10.x / 3.11.x | `python --version` 检查版本 |
| **CUDA Toolkit** | 12.1 / 12.2 / 12.4 | [NVIDIA 官方下载](https://developer.nvidia.com/cuda-downloads) |
| **cuDNN** | 与 CUDA 版本匹配 | 通常随 CUDA Toolkit 安装 |
| **PyTorch** | ≥ 2.3.0 | `pip install torch>=2.3.0` |
| **NVIDIA Driver** | ≥ 525.x (for CUDA 12.x) | `nvidia-smi` 检查驱动版本 |
| **Git** | 任意版本 | 用于克隆 LLaMA Factory 仓库 |

#### 2.3 环境验证命令

在开始微调之前，请运行以下命令验证环境配置：

```bash
# 检查 NVIDIA 驱动和 GPU
nvidia-smi

# 检查 CUDA 版本
nvcc --version

# 检查 Python 版本
python --version

# 检查 PyTorch 和 CUDA 是否正确安装
python -c "import torch; print(f'PyTorch: {torch.__version__}'); print(f'CUDA Available: {torch.cuda.is_available()}'); print(f'CUDA Version: {torch.version.cuda}')"
```

#### 2.4 网络要求

| 网络需求 | 说明 |
|---------|------|
| **模型下载** | 需访问 Hugging Face Hub 或 ModelScope<br>Qwen3-8B 模型约 15GB |
| **依赖安装** | 需访问 PyPI 镜像源 (可使用阿里云/清华镜像加速) |
| **防火墙配置** | 如使用 SSH 远程访问，需开放相应端口 |

---

> **💡 提示**：如果您的 GPU 显存不足 24GB，可以考虑：
> - 使用更小的模型（如 Qwen3-1.8B 或 Qwen3-4B）
> - 启用梯度检查点（Gradient Checkpointing）降低显存占用
> - 减小批次大小（Batch Size）
> - 使用 QLoRA（量化 LoRA）进一步降低显存需求

## 系统环境检查

In [ ]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                              |
|:-------------|:----------------------------------------------------------------------------------|
| 操作系统     | Linux 5.10.134-18.0.4.lifsea8.x86_64                                              |
| CPU 信息     | Intel(R) Xeon(R) Platinum 8369B CPU @ 2.90GHz (4 physical cores, 8 logical cores) |
| 内存信息     | 29.37 GB (Available: 27.89 GB)                                                    |
| GPU 信息     | NVIDIA A10 (24564 MiB)                                                            |
| CUDA 信息    | 12.4                                                                              |
| Python 版本  | 3.10.19                                                                           |
| Conda 版本   | Conda not found                                               

## 检查软硬件依赖

| 必需项       | 至少   | 推荐   |
| ------------ | ------ | ------ |
| python       | 3.9    | 3.10   |
| torch        | 2.0.0  | 2.6.0  |
| torchvision  | 0.15.0 | 0.21.0 |
| transformers | 4.45.0 | 4.50.0 |
| datasets     | 2.16.0 | 3.2.0  |
| accelerate   | 0.34.0 | 1.2.1  |
| peft         | 0.14.0 | 0.15.1 |
| trl          | 0.8.6  | 0.9.6  |

| 可选项       | 至少   | 推荐   |
| ------------ | ------ | ------ |
| CUDA         | 11.6   | 12.2   |
| deepspeed    | 0.10.0 | 0.16.4 |
| bitsandbytes | 0.39.0 | 0.43.1 |
| vllm         | 0.4.3  | 0.8.2  |
| flash-attn   | 2.5.6  | 2.7.2  |

### 硬件依赖

\* *估算值*

| 方法                            | 精度 | 7B    | 14B   | 30B   | 70B    | `x`B    |
| ------------------------------- | ---- | ----- | ----- | ----- | ------ | ------- |
| Full (`bf16` or `fp16`)         | 32   | 120GB | 240GB | 600GB | 1200GB | `18x`GB |
| Full (`pure_bf16`)              | 16   | 60GB  | 120GB | 300GB | 600GB  | `8x`GB  |
| Freeze/LoRA/GaLore/APOLLO/BAdam | 16   | 16GB  | 32GB  | 64GB  | 160GB  | `2x`GB  |
| QLoRA                           | 8    | 10GB  | 20GB  | 40GB  | 80GB   | `x`GB   |
| QLoRA                           | 4    | 6GB   | 12GB  | 24GB  | 48GB   | `x/2`GB |
| QLoRA                           | 2    | 4GB   | 8GB   | 16GB  | 24GB   | `x/4`GB |

In [ ]:
import sys
import platform
import importlib.metadata
from packaging import version
import torch

# ================= 您的最新配置要求 (已再次更新) =================
REQUIRED_PACKAGES = {
    "python":       ("3.9",    "3.10"),
    "torch":        ("2.0.0",  "2.6.0"),
    "torchvision":  ("0.15.0", "0.21.0"),
    "transformers": ("4.45.0", "4.50.0"),
    "datasets":     ("2.16.0", "3.2.0"),
    "accelerate":   ("0.34.0", "1.2.1"),
    "peft":         ("0.14.0", "0.15.1"),
    "trl":          ("0.8.6",  "0.9.6"),
}

OPTIONAL_PACKAGES = {
    "cuda":         ("11.6",   "12.2"),
    "deepspeed":    ("0.10.0", "0.16.4"),
    "bitsandbytes": ("0.39.0", "0.43.1"),
    "vllm":         ("0.4.3",  "0.8.2"),
    "flash-attn":   ("2.5.6",  "2.7.2"),
}

# ================= 检测逻辑 =================
def get_package_version(package_name):
    try:
        if package_name == "python": return platform.python_version()
        elif package_name == "cuda": return torch.version.cuda if torch.cuda.is_available() else None
        elif package_name == "flash-attn": return importlib.metadata.version("flash_attn")
        else: return importlib.metadata.version(package_name)
    except importlib.metadata.PackageNotFoundError: return None

def check_requirement(name, constraints, is_optional=False):
    min_v, rec_v = constraints
    current_v = get_package_version(name)

    # 定义列宽以保持对齐
    col_name = 15
    col_curr = 15
    col_req = 10

    if current_v is None:
        status, msg, color = ("⚪", "未安装 (可选)", "\033[90m") if is_optional else ("❌", "未安装 !!", "\033[91m")
    else:
        try:
            curr_p, min_p, rec_p = version.parse(current_v), version.parse(min_v), version.parse(rec_v)
            if curr_p >= rec_p:   status, msg, color = ("✅", f"完美 ({current_v})", "\033[92m") # Green
            elif curr_p >= min_p: status, msg, color = ("⚠️", f"达标 ({current_v})", "\033[93m") # Yellow
            else:                 status, msg, color = ("❌", f"过低 ({current_v})", "\033[91m") # Red
        except Exception as e:
            status, msg, color = ("❓", f"解析错误", "\033[93m")

    print(f"{color}{status} {name:<{col_name}} | 当前: {msg:<{col_curr}} | 必需: >={min_v:<{col_req}} | 推荐: >={rec_v}\033[0m")

def check_gpu():
    print("\n[ 硬件简测 ]")
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        print(f"✅ 检测到 {device_count} 个 GPU 设备:")
        for i in range(device_count):
            gpu_name = torch.cuda.get_device_name(i)
            # 转换为 GB
            total_mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"   - GPU {i}: {gpu_name} ({total_mem:.1f} GB VRAM)")
    else:
        print("⚪ 未检测到可用 GPU (CUDA不可用)")

print("="*75)
print(f"🛠️  环境依赖检测报告 (标准已再次更新)")
print("="*75)
print("[ 必需项 ]")
for k, v in REQUIRED_PACKAGES.items(): check_requirement(k, v)
print("\n[ 可选项 ]")
for k, v in OPTIONAL_PACKAGES.items(): check_requirement(k, v, True)
check_gpu()
print("="*75)

🛠️  环境依赖检测报告 (标准已再次更新)
[ 必需项 ]
✅ python          | 当前: 完美 (3.10.19)    | 必需: >=3.9        | 推荐: >=3.10
✅ torch           | 当前: 完美 (2.6.0+cu124) | 必需: >=2.0.0      | 推荐: >=2.6.0
✅ torchvision     | 当前: 完美 (0.21.0+cu124) | 必需: >=0.15.0     | 推荐: >=0.21.0
❌ transformers    | 当前: 未安装 !!          | 必需: >=4.45.0     | 推荐: >=4.50.0
❌ datasets        | 当前: 未安装 !!          | 必需: >=2.16.0     | 推荐: >=3.2.0
❌ accelerate      | 当前: 未安装 !!          | 必需: >=0.34.0     | 推荐: >=1.2.1
❌ peft            | 当前: 未安装 !!          | 必需: >=0.14.0     | 推荐: >=0.15.1
❌ trl             | 当前: 未安装 !!          | 必需: >=0.8.6      | 推荐: >=0.9.6

[ 可选项 ]
✅ cuda            | 当前: 完美 (12.4)       | 必需: >=11.6       | 推荐: >=12.2
⚪ deepspeed       | 当前: 未安装 (可选)        | 必需: >=0.10.0     | 推荐: >=0.16.4
✅ bitsandbytes    | 当前: 完美 (0.48.2)     | 必需: >=0.39.0     | 推荐: >=0.43.1
⚪ vllm            | 当前: 未安装 (可选)        | 必需: >=0.4.3      | 推荐: >=0.8.2
⚪ flash-attn      | 当前: 未安装 (可选)        | 必需: >=2.5.6      | 推荐: >=2.7.2

[ 硬件

## 1. 安装LLaMA Factory


首先，拉取LLaMA-Factory项目到DSW实例。

In [ ]:
import os
import shutil

# 定义仓库文件夹名称
repo_name = "LLaMA-Factory"

# 1. 判断文件夹是否存在
if os.path.exists(repo_name):
    # 获取用户输入
    user_input = input(f"⚠️ 检测到旧目录 '{repo_name}'，是否删除？(输入 y 确认删除，其他键取消): ")

    # 判断用户输入 (支持 y 或 Y)
    if user_input.lower() == 'y':
        print(f"正在删除 '{repo_name}'...")
        try:
            shutil.rmtree(repo_name) # 执行实际删除
            print("🗑️ 旧目录已清除！")
        except Exception as e:
            print(f"❌ 删除失败: {e}")
    else:
        print("🚫 操作已取消，保留旧目录。")
else:
    print(f"✅ 目录 '{repo_name}' 不存在，准备下载仓库...")
    # 克隆仓库
    !git clone -b v0.9.3 https://github.com/hiyouga/LLaMA-Factory.git

✅ 目录 'LLaMA-Factory' 不存在，准备下载仓库...
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 24605, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 24605 (delta 67), reused 24 (delta 24), pack-reused 24475 (from 2)
Receiving objects: 100% (24605/24605), 12.22 MiB | 1.19 MiB/s, done.
Resolving deltas: 100% (17696/17696), done.
Note: switching to 'ca75f1edf3cb50343ed1c98605141c3e22075b5f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



接着，我们安装LLaMA-Factory依赖环境。

In [ ]:
# 1. 【关键】使用 %cd 进入目录 (不要用 !cd)
%cd /mnt/workspace/LLaMA-Factory

# 2. 去掉无关依赖
!pip install --upgrade pip
!pip install modelscope>=1.11.0
!pip install bitsandbytes>=0.39.0
# !pip install vllm==0.8.3

# 3. 安装项目依赖
# 这会自动安装 v0.9.3 所需的其他库
!pip install -e ".[torch,metrics]"

/mnt/workspace/LLaMA-Factory


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Obtaining file:///mnt/workspace/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 1.3 MB/s  0:00:07m0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.3 MB/s  0:00:02 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 1.2 MB/s  0:00:44m0:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 1.2 MB/s  0:00:30m0:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.5 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.3 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 1.3 MB/s  0:00:06 eta 0:00:01
     ━━━

### 检验依赖是否满足

In [ ]:

import sys
import platform
import importlib.metadata
from packaging import version
import torch

# ================= 您的最新配置要求 (已再次更新) =================
REQUIRED_PACKAGES = {
    "python":       ("3.9",    "3.10"),
    "torch":        ("2.0.0",  "2.6.0"),
    "torchvision":  ("0.15.0", "0.21.0"),
    "transformers": ("4.45.0", "4.50.0"),
    "datasets":     ("2.16.0", "3.2.0"),
    "accelerate":   ("0.34.0", "1.2.1"),
    "peft":         ("0.14.0", "0.15.1"),
    "trl":          ("0.8.6",  "0.9.6"),
}

OPTIONAL_PACKAGES = {
    "cuda":         ("11.6",   "12.2"),
    "deepspeed":    ("0.10.0", "0.16.4"),
    "bitsandbytes": ("0.39.0", "0.43.1"),
    "vllm":         ("0.4.3",  "0.8.2"),
    "flash-attn":   ("2.5.6",  "2.7.2"),
}

# ================= 检测逻辑 =================
def get_package_version(package_name):
    try:
        if package_name == "python": return platform.python_version()
        elif package_name == "cuda": return torch.version.cuda if torch.cuda.is_available() else None
        elif package_name == "flash-attn": return importlib.metadata.version("flash_attn")
        else: return importlib.metadata.version(package_name)
    except importlib.metadata.PackageNotFoundError: return None

def check_requirement(name, constraints, is_optional=False):
    min_v, rec_v = constraints
    current_v = get_package_version(name)

    # 定义列宽以保持对齐
    col_name = 15
    col_curr = 15
    col_req = 10

    if current_v is None:
        status, msg, color = ("⚪", "未安装 (可选)", "\033[90m") if is_optional else ("❌", "未安装 !!", "\033[91m")
    else:
        try:
            curr_p, min_p, rec_p = version.parse(current_v), version.parse(min_v), version.parse(rec_v)
            if curr_p >= rec_p:   status, msg, color = ("✅", f"完美 ({current_v})", "\033[92m") # Green
            elif curr_p >= min_p: status, msg, color = ("⚠️", f"达标 ({current_v})", "\033[93m") # Yellow
            else:                 status, msg, color = ("❌", f"过低 ({current_v})", "\033[91m") # Red
        except Exception as e:
            status, msg, color = ("❓", f"解析错误", "\033[93m")

    print(f"{color}{status} {name:<{col_name}} | 当前: {msg:<{col_curr}} | 必需: >={min_v:<{col_req}} | 推荐: >={rec_v}\033[0m")

def check_gpu():
    print("\n[ 硬件简测 ]")
    if torch.cuda.is_available():
        device_count = torch.cuda.device_count()
        print(f"✅ 检测到 {device_count} 个 GPU 设备:")
        for i in range(device_count):
            gpu_name = torch.cuda.get_device_name(i)
            # 转换为 GB
            total_mem = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"   - GPU {i}: {gpu_name} ({total_mem:.1f} GB VRAM)")
    else:
        print("⚪ 未检测到可用 GPU (CUDA不可用)")

print("="*75)
print(f"🛠️  环境依赖检测报告 (标准已再次更新)")
print("="*75)
print("[ 必需项 ]")
for k, v in REQUIRED_PACKAGES.items(): check_requirement(k, v)
print("\n[ 可选项 ]")
for k, v in OPTIONAL_PACKAGES.items(): check_requirement(k, v, True)
check_gpu()
print("="*75)

🛠️  环境依赖检测报告 (标准已再次更新)
[ 必需项 ]
✅ python          | 当前: 完美 (3.10.19)    | 必需: >=3.9        | 推荐: >=3.10
✅ torch           | 当前: 完美 (2.6.0+cu124) | 必需: >=2.0.0      | 推荐: >=2.6.0
✅ torchvision     | 当前: 完美 (0.21.0+cu124) | 必需: >=0.15.0     | 推荐: >=0.21.0
✅ transformers    | 当前: 完美 (4.52.4)     | 必需: >=4.45.0     | 推荐: >=4.50.0
✅ datasets        | 当前: 完美 (3.6.0)      | 必需: >=2.16.0     | 推荐: >=3.2.0
✅ accelerate      | 当前: 完美 (1.7.0)      | 必需: >=0.34.0     | 推荐: >=1.2.1
✅ peft            | 当前: 完美 (0.15.2)     | 必需: >=0.14.0     | 推荐: >=0.15.1
✅ trl             | 当前: 完美 (0.9.6)      | 必需: >=0.8.6      | 推荐: >=0.9.6

[ 可选项 ]
✅ cuda            | 当前: 完美 (12.4)       | 必需: >=11.6       | 推荐: >=12.2
⚪ deepspeed       | 当前: 未安装 (可选)        | 必需: >=0.10.0     | 推荐: >=0.16.4
✅ bitsandbytes    | 当前: 完美 (0.48.2)     | 必需: >=0.39.0     | 推荐: >=0.43.1
⚪ vllm            | 当前: 未安装 (可选)        | 必需: >=0.4.3      | 推荐: >=0.8.2
⚪ flash-attn      | 当前: 未安装 (可选)        | 必需: >=2.5.6      | 推荐: >=2.7.2

[ 硬件

运行如下命令，如果显示llamafactory-cli的版本，则表示安装成功。

In [ ]:
!llamafactory-cli version

----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.3                |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------


## 2. 下载数据集

LLaMA-Factory项目内置了丰富的数据集，放在了`data`目录下。您可以跳过本步骤，直接使用内置数据集。您也可以准备自定义数据集，将数据处理为框架特定的格式，放在`data`下，并且修改`dataset_info.json`文件。

本教程准备了一份多轮对话数据集，运行下述命令下载数据。

In [ ]:
!wget https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/llama_factory/data.zip
!mv data rawdata && unzip data.zip -d data

--2025-12-05 16:25:33--  https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/release/tutorials/llama_factory/data.zip
Resolving atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)... 47.101.88.43
Connecting to atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)|47.101.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 402043 (393K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 392.62K  --.-KB/s    in 0.1s    

2025-12-05 16:25:34 (3.76 MB/s) - ‘data.zip’ saved [402043/402043]

Archive:  data.zip
  inflating: data/eval.json          
  inflating: data/__MACOSX/._eval.json  
  inflating: data/LICENSE            
  inflating: data/__MACOSX/._LICENSE  
  inflating: data/dataset_info.json  
  inflating: data/__MACOSX/._dataset_info.json  
  inflating: data/train.json         
  inflating: data/__MACOSX/._train.json  


上传医疗数据集，将jsonl文件拖拽到`/mnt/workspace`文件下。
```
root@dsw-523480-fd7f95d6d-cgjwf:/mnt/workspace# pwd
/mnt/workspace
root@dsw-523480-fd7f95d6d-cgjwf:/mnt/workspace# ll
total 63916
drwxr-xr-x  7 root root     4096 Dec  3 16:32  ./
drwxr-xr-x  1 root root     4096 Dec  3 15:06  ../
drwxr-xr-x  2 root root     4096 Dec  3 15:19  _html/
drwxr-xr-x 16 root root     4096 Dec  3 15:59  LLaMA-Factory/
-rw-r--r--  1 root root    87581 Dec  3 16:32  llama-factory-in-ali-pai.ipynb
drwxr-xr-x  3 root root     4096 Dec  3 15:06  .local/
-rw-r--r--  1 root root 65325060 Dec  3 16:21  medical_o1_sft_Chinese_alpaca_cot.jsonl
drwxr-xr-x  2 root root     4096 Dec  3 15:10  .virtual_documents/
```


修改`/mnt/workspace`文件夹下“dataset_json.info”文件，在json文件开始位置添加如下所示脚本。
```
"medical_sft": {
    "file_name": "/workspace/LLaMA-Factory/data/medical_o1_sft_Chinese_alpaca_cot.jsonl",
    "columns": {
      "prompt": "prompt",
      "response": "response"
    }
  }
```

“dataset_json.info”文件最终如下
```
{
  "medical_sft": {
    "file_name": "/mnt/workspace/medical_o1_sft_Chinese_alpaca_cot.jsonl",
    "columns": {
      "prompt": "prompt",
      "response": "response"
    }
  },
  "train": {
    "file_name": "train.json",
    "formatting": "sharegpt"
  },
  "eval": {
    "file_name": "eval.json",
    "formatting": "sharegpt"
  }
}
```

In [ ]:
import json
import os

# 1. 你的数据集路径
jsonl_file = "/mnt/workspace/medical_o1_sft_Chinese_alpaca_cot.jsonl"
# 2. LLaMA-Factory 的配置文件路径
config_file = "/mnt/workspace/LLaMA-Factory/data/dataset_info.json"

# 3. 要添加的配置内容 (Python字典格式，不需要手动管逗号)
new_data = {
    "medical_sft": {
        "file_name": jsonl_file,
        "columns": {
            "prompt": "prompt",
            "response": "response"
        }
    }
}

# 4. 执行自动写入
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        content = json.load(f)

    # 更新数据（程序会自动处理逗号分割）
    content.update(new_data)

    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(content, f, indent=2, ensure_ascii=False)

    print(f"✅ 成功！已自动添加逗号并注册 'medical_sft' 到：\n{config_file}")

except FileNotFoundError:
    print("❌ 错误：找不到 LLaMA-Factory 的配置文件，请检查目录结构。")
except json.JSONDecodeError:
    print("❌ 错误：原配置文件格式损坏（可能之前手动改坏了），请还原文件。")

✅ 成功！已自动添加逗号并注册 'medical_sft' 到：
/mnt/workspace/LLaMA-Factory/data/dataset_info.json


## 3. 模型微调
### 3.1 启动Web UI

做好前序准备工作后，直接运行下述命令就可以启动Web UI。这里用到的环境变量解释如下：
- `USE_MODELSCOPE_HUB`设为1，表示模型来源是ModelScope。使用HuggingFace模型可能会有网络问题。

点击返回的URL地址`http://0.0.0.0:7860`，进入Web UI页面。

In [ ]:
!export USE_MODELSCOPE_HUB=1 && \
llamafactory-cli webui

Visit http://ip:port for Web UI, e.g., http://127.0.0.1:7860
* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
[WARNING|2025-12-05 16:35:32] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-12-05 16:35:32] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.bfloat16
[INFO|tokenization_utils_base.py:2021] 2025-12-05 16:35:32,574 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2021] 2025-12-05 16:35:32,574 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2021] 2025-12-05 16:35:32,574 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2021] 2025-12-05 16:35:32,575 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2021] 2025-12-05 16:35:32,575 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2021] 2025-12-05 16:35:32,575 >>

### 3.2 配置参数
结合 **A10 (24G)** 显卡与 **Qwen3-8B** 的环境，以下是针对配置界面的详细参数解读。这些设置共同构成了 **QLoRA** 微调方案，是在单卡环境下处理医疗推理（Reasoning）长文本数据的最佳实践。

![image-20251204185719305](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041857405.png)

#### 3.2.1 模型路径与名称配置 (Model Configuration)
* **参数项**：`模型名称` & `模型路径`
* **当前值**：
    * 名称：`Qwen3-8B-Instruct`
    * 路径：`/mnt/workspace/Qwen3-8B`
* **配置作用**：
    * **模型路径**：这是“地基”。程序会从 `/mnt/workspace/Qwen3-8B` 读取模型权重的物理文件。
    * **模型名称**：这是“标识”。LLaMA-Factory 会根据这个名称自动匹配默认的超参数和模板。
* **硬件影响**：A10 的 24G 显存加载 8B 模型毫无压力，这一步是基础。

模型路径使用本地路径，如：`/mnt/workspace/Qwen3-8B`。（考虑公网有时候下载模型慢，提供给大家准备好了离线模型压缩包，可以解压到`/mnt/workspace/Qwen3-8B`）

其中`/mnt/workspace/Qwen3-8B`内容如下，遇到出错，请核对文件夹是不是少内容

```bash
root@dsw-523480-6d6bb75cf6-kz46d:/mnt/workspace/Qwen3-8B# ll
total 16013364
drwxr-xr-x  2 root root       4096 Dec  4 17:47 ./
drwxr-xr-x 10 root root       4096 Dec  4 18:38 ../
-rw-r--r--  1 root root        757 Dec  4 17:02 config.json
-rw-r--r--  1 root root         73 Dec  4 17:02 configuration.json
-rw-r--r--  1 root root        251 Dec  4 17:02 generation_config.json
-rw-r--r--  1 root root       2175 Dec  4 17:02 .gitattributes
-rw-r--r--  1 root root      11544 Dec  4 17:02 LICENSE
-rw-r--r--  1 root root    1823241 Dec  4 17:02 merges.txt
-rw-r--r--  1 root root 3996250744 Dec  4 17:41 model-00001-of-00005.safetensors
-rw-r--r--  1 root root 3993160032 Dec  4 17:45 model-00002-of-00005.safetensors
-rw-r--r--  1 root root 3959604768 Dec  4 17:47 model-00003-of-00005.safetensors
-rw-r--r--  1 root root 3187841392 Dec  4 17:43 model-00004-of-00005.safetensors
-rw-r--r--  1 root root 1244659840 Dec  4 17:14 model-00005-of-00005.safetensors
-rw-r--r--  1 root root      33284 Dec  4 17:02 model.safetensors.index.json
-rw-r--r--  1 root root      17012 Dec  4 17:02 README.md
-rw-r--r--  1 root root       9971 Dec  4 17:02 tokenizer_config.json
-rw-r--r--  1 root root   11422654 Dec  4 17:02 tokenizer.json
-rw-r--r--  1 root root    2776833 Dec  4 17:02 vocab.json
```

这是典型的 Hugging Face `transformers` 格式的模型目录结构（Qwen 系列）。

这个目录包含了 **模型架构配置**、**分词器（Tokenizer）** 和 **模型权重（Weights）** 三大核心部分。

以下是逐行注释解释：

```bash
# ----------------- 核心配置区 -----------------
-rw-r--r--  1 root root        757 Dec  4 17:02 config.json             # 【最关键】模型架构配置文件。定义了模型有多少层、隐藏层大小、注意力头数等。代码通过它来构建模型骨架。
-rw-r--r--  1 root root         73 Dec  4 17:02 configuration.json      # 辅助配置文件（通常较少见，可能是特定框架生成的元数据，通常 config.json 是主文件）。
-rw-r--r--  1 root root        251 Dec  4 17:02 generation_config.json  # 推理参数配置。定义了模型生成文本时的默认行为（如 max_length, temperature, top_p, eos_token_id 等）。

# ----------------- 版本控制与协议 -----------------
-rw-r--r--  1 root root       2175 Dec  4 17:02 .gitattributes          # Git LFS 配置文件。告诉 git 哪些文件是大文件（如 .safetensors），需要用 LFS 方式下载。
-rw-r--r--  1 root root      11544 Dec  4 17:02 LICENSE                 # 模型的开源许可证文件（如 Apache 2.0 或 Qwen Research License），规定了你能否商用。

# ----------------- 分词器 (Tokenizer) 词表区 -----------------
# 作用：将人类文字转换成数字 ID，反之亦然
-rw-r--r--  1 root root    1823241 Dec  4 17:02 merges.txt              # BPE 合并规则文件。记录了哪些字符组合可以合并成一个 Token（常见于 GPT 类模型）。
-rw-r--r--  1 root root    2776833 Dec  4 17:02 vocab.json              # 词表文件。一个巨大的字典，记录了所有 Token 字符串到数字 ID 的映射关系。
-rw-r--r--  1 root root   11422654 Dec  4 17:02 tokenizer.json          # 【重要】Fast Tokenizer 的完整定义文件。包含了分词的所有逻辑，加载速度比 Python 代码快。
-rw-r--r--  1 root root       9971 Dec  4 17:02 tokenizer_config.json   # 分词器设置文件。定义了特殊 Token（如 <|endoftext|>），以及使用哪个 Tokenizer 类。

# ----------------- 模型权重区 (Weights) -----------------
# 作用：模型的“大脑”，存储了训练好的参数（矩阵数值）
# 由于 8B 模型很大（约 15GB+），所以被切分成了 5 个文件（Sharding）
-rw-r--r--  1 root root 3996250744 Dec  4 17:41 model-00001-of-00005.safetensors # 权重分片 1/5
-rw-r--r--  1 root root 3993160032 Dec  4 17:45 model-00002-of-00005.safetensors # 权重分片 2/5
-rw-r--r--  1 root root 3959604768 Dec  4 17:47 model-00003-of-00005.safetensors # 权重分片 3/5
-rw-r--r--  1 root root 3187841392 Dec  4 17:43 model-00004-of-00005.safetensors # 权重分片 4/5
-rw-r--r--  1 root root 1244659840 Dec  4 17:14 model-00005-of-00005.safetensors # 权重分片 5/5

# ----------------- 权重索引与说明 -----------------
-rw-r--r--  1 root root      33284 Dec  4 17:02 model.safetensors.index.json     # 权重索引映射表。因为权重被切分了，这个文件告诉加载器：“第 X 层的参数在第 Y 个 safetensors 文件里”。
-rw-r--r--  1 root root      17012 Dec  4 17:02 README.md               # 模型说明书（Markdown 格式）。通常包含模型介绍、引用方法、使用示例代码等。
```

当你运行 `AutoModelForCausalLM.from_pretrained("这个目录")` 时：

1. **读取 `config.json`**：搭建空的神经网络骨架。
2. **读取 `model.safetensors.index.json`**：查找参数都在哪。
3. **加载 `model-0000\*.safetensors`**：把具体的数字填入骨架中。

当你运行 `AutoTokenizer.from_pretrained("这个目录")` 时：

1. **读取 `tokenizer_config.json`**：确认配置。
2. **优先加载 `tokenizer.json`**：如果失败，才去尝试组合 `vocab.json` 和 `merges.txt`。

#### 3.2.2 微调核心方法 (Finetuning Method)
* **参数项**：`微调方法`
* **当前值**：`lora`
* **配置作用**：
    * 选择 **LoRA** (Low-Rank Adaptation) 技术。它不会修改模型原本的几十亿个参数，而是在模型旁挂载一些小的“适配器”矩阵来训练。
* **为什么不选 full（全量）**：
    * 全量微调 8B 模型需要保存庞大的优化器状态，至少需要 100GB+ 的显存。
    * **A10 (24G) 的唯一出路**：对于单卡 A10，必须使用 LoRA 或 QLoRA 才能跑得起来。

#### 3.2.3 量化设置 (Quantization) —— 关键优化
这一步是将 **LoRA** 升级为 **QLoRA** 的关键，直接决定了您能训练多长的医疗病例。

* **参数项**：`量化等级`
* **当前值**：`4`
* **配置作用**：
    * 将模型权重的精度从 16-bit (半精度) 压缩到 4-bit。
* **对 A10 (24G) 的巨大影响**：
    * **不量化 (16-bit)**：8B 模型仅权重就占用约 **16GB** 显存。此时 A10 只剩 8GB 给训练数据，只要医疗文本稍微长一点（例如包含复杂的病情描述），显存就会瞬间溢出 (OOM)。
    * **4-bit 量化**：模型权重仅占用约 **5.5GB** 显存。
    * **收益**：您空出了近 **18GB** 的显存！这意味着您可以将 `Context Length`（上下文长度）拉得很大（例如 4096 或 8192），这对于 **medical-o1-reasoning** 这种需要长逻辑链推理的数据集至关重要。

* **参数项**：`量化方法`
* **当前值**：`bnb` (bitsandbytes)
* **配置作用**：
    * 这是实现 4-bit 量化的底层加速库。您的环境是 Ubuntu 22.04 + CUDA 12.4，`bnb` 能够完美调用 A10 的硬件加速能力。

#### 3.2.4 数据模板与格式 (Template)
* **参数项**：`对话模板`
* **当前值**：`qwen3` (注意：截图可能需确认为 qwen 或 qwen2.5/3 对应模板，Qwen3 通常兼容 Qwen2.5 模板)
* **配置作用**：
    * 如同“翻译器”，它将您的医疗数据集格式化为 Qwen 模型能听懂的特殊字符结构（例如 `<|im_start|>`）。
* **严重后果**：如果选错（例如选成 llama3），模型会因为看不懂数据结构而输出乱码，或者在训练时 loss 无法下降。务必确保与模型系列一致。

#### 3.2.5 硬件加速策略 (Booster)
* **参数项**：`加速方式`
* **当前值**：`auto`
* **配置作用**：
    * 自动选择最优的注意力机制加速算法，通常会启用 **FlashAttention-2**。
* **环境匹配**：
    * A10 显卡属于 Ampere 架构，完美支持 FlashAttention-2。
    * **效果**：相比不开启加速，训练速度通常能提升 2-3 倍，且能进一步节省显存。保持 `auto` 是最省心的选择。
    

#### 3.2.6 训练阶段 (Training Stage)
* **配置项**：`训练阶段`
* **当前设置**：`Supervised Fine-Tuning` (SFT)
* **详细解释**：
    * **定义**：全称是“有监督微调”。这是目前让大模型适配特定领域（如医疗）最直接、最高效的方法。
    * **原理**：就好比给学生（模型）看大量的“问题+标准答案”的试卷，让它学会照葫芦画瓢。与之相对的还有“预训练（Pre-Training）”（读课本，学通识）和“DPO/RLHF”（学价值观，分辨好坏）。
    * **适用场景**：您现在的目标是让 Qwen3 拥有医疗推理能力，SFT 是标准选择。
    * **环境影响**：SFT 相比预训练极其节省资源。在您的 A10 (24G) 上，配合 LoRA，SFT 运行起来非常轻量。

#### 3.2.7 数据路径与数据集选择 (Data Configuration)
* **配置项**：`数据路径 (Data Path)` & `数据集 (Dataset)`
* **当前设置**：
    * 路径：`data`
    * 数据集：`medical_sft` (截图显示您已选中此项)
* **详细解释 (新手必读)**：
    * **数据路径**：指向 LLaMA-Factory 根目录下的 `data` 文件夹。这里面存放着至关重要的 `dataset_info.json` 配置文件。
    * **数据集选择**：
        * 这里的 `medical_sft` 并不是文件名，而是在 `dataset_info.json` 中注册的一个**标签（Key）**。
        * **关键操作**：您之前提到了 `medical-o1-reasoning-SFT` 数据集。您需要确保已经在 `data/dataset_info.json` 文件中添加了对应的配置，将 `medical_sft` 这个标签指向您实际下载的 JSON 或 Parquet 文件。
        * *错误排查*：如果您在下拉菜单里找不到您的数据集，通常是因为没有在 `dataset_info.json` 里注册，或者文件没放对位置。
* **作用**：告诉程序使用哪一份具体的医疗病历数据来喂给模型。

#### 3.2.8 数据预览功能 (Dataset Preview)
* **配置项**：`预览数据集` (按钮)
* **建议操作**：**强烈建议点击**。
* **详细解释**：
    * **作用**：它会读取前几行数据并展示出来。
    * **新手检查点**：
        1.  **列名匹配**：检查展示的数据中，是否包含“instruction”（指令/问题）、“input”（输入/病历详情，可选）、“output”（输出/诊断结果）这几列。
        2.  **格式正常**：确保没有乱码，且内容确实是您预期的医疗推理数据。
    * **为什么重要**：如果数据格式不对（比如列名写成了 `question` 而不是 `instruction`），模型训练时会找不到输入，导致训练虽然在跑，但什么都没学到（Loss 不下降）。在开始漫长的训练前，花 10 秒钟预览是性价比最高的操作。

数据集使用我们的医疗数据集`medical_sft`。
![image-20251203165018105](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031650258.png)
可以点击「预览数据集」。点击关闭返回训练界面。
![image-20251203165125842](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031651285.png)







设置`学习率`为`1e-4`，`梯度累积`为`2`，有利于模型拟合。如果显卡是`V100`，计算类型保持为`fp16`；如果使用了`A10`，可以更改计算类型为`bf16`。
![image-20251203165304488](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031653595.png)

点击`LoRA参数设置`展开参数列表，设置`LoRA+学习率比例`为`16`，`LoRA+`被证明是比LoRA学习效果更好的算法。在`LoRA作用模块`中填写`all`，即将LoRA层挂载到模型的所有线性层上，提高拟合效果。
![image-20251203165600495](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031656725.png)


#### 3.2.9 学习率 (Learning Rate)

- **配置项**：`学习率`
- **当前值**：`1e-4` (即 0.0001)
- **详细解释**：
  - **作用**：决定了模型“迈步子”的大小。
  - **新手指南**：
    - 对于 **LoRA** 微调，`1e-4` 是一个非常标准且稳健的“黄金初始值”。
    - 如果是全量微调，这个值通常要小得多（如 `1e-5`）。但对于 LoRA，保持 `1e-4` 通常能获得不错的收敛速度。
  - **调整建议**：初次运行建议保持不变。如果发现 Loss（损失值）完全不下降，可以尝试稍微调大；如果 Loss 剧烈震荡，则调小。

#### 3.2.10 训练轮数 (Epochs)

- **配置项**：`训练轮数`
- **当前值**：`3.0`
- **详细解释**：
  - **作用**：指的是模型要把你的整个医疗数据集完整地“看”几遍。
  - **影响**：
    - **过少 (比如 1)**：模型可能还没学会，导致“欠拟合”。
    - **过多 (比如 10)**：模型可能会死记硬背训练题，导致“过拟合”，遇到新病历就不会治了。
  - **推荐**：`3.0` 是绝大多数 SFT 任务的标准起点，非常合适。

#### 3.2.11 最大样本数 (Max Samples) —— **重要警示**

- **配置项**：`最大样本数`
- **当前值**：`1000`
- **详细解释**：
  - **作用**：强行限制只使用数据集里的前 1000 条数据来训练。
  - **新手误区警报**：
    - **调试模式**：如果你只是想花几分钟测试一下 A10 显卡能不能跑通代码，设置 `1000` 是对的。
    - **正式训练**：**请务必清空此项**（留空）。如果你的医疗数据集有 10 万条数据，填了 `1000` 就意味着模型丢弃了 99% 的知识，这会导致训练出来的模型极其弱智。
  - **操作建议**：跑通测试后，正式跑数据时记得删掉这个数字。

#### 3.2.12 截断长度 (Cutoff Length)

- **配置项**：`截断长度`
- **当前值**：`2048`
- **详细解释**：
  - **作用**：限制一条数据（输入问题 + 模型回答）的最大总字数（Token 数）。超过的部分会被“一刀切”切掉。
  - **生产建议**： **Medical O1 Reasoning (医疗推理)**。这类数据通常包含很长的思维链（Chain of Thought），详细描述病情和推理过程。`2048` 可能偏短，容易把关键的推理结论切掉。

#### 3.2.13 显存控制核心：批处理与梯度累积

这里是决定你训练速度和显存占用的“油门”和“刹车”。

- **配置项**：`批处理大小 (Batch Size)`
- **当前值**：`1`
- **详细解释**：
  - **含义**：每个 GPU 一次读入并计算 1 条数据。
- **配置项**：`梯度累积 (Gradient Accumulation)`
- **当前值**：`4`
- **详细解释**：
  - **含义**：攒够 4 次（1x4=4）再修改一次模型参数。
  - **真实批大小**：实际生效的 Batch Size = `批处理大小` * `梯度累积` = 1 * 4 = 4。

#### 3.2.14 验证集比例 (Val Ratio)

- **配置项**：`验证集比例`
- **当前值**：`0.15` (15%)
- **详细解释**：
  - **作用**：从训练数据里切出 15% 的题不给模型学，专门留到考试用，看看模型是不是真的懂了（还是死记硬背）。
  - **建议**：
    - 如果数据总量很大（比如 >1万条），15% 可能太多了（浪费数据），可以改为 `0.05`。
    - 如果数据量少（<1000条），`0.15` 是合理的。

#### 3.2.15 计算类型 (Compute Type)

- **配置项**：`计算类型`
- **当前值**：`bf16`
- **详细解释**：
  - **含义**：使用 Brain Float 16 精度进行计算。
  - **环境匹配**：**完美**。你的 A10 显卡（Ampere 架构）原生支持 bf16。相比 fp16，bf16 更加稳定，训练过程中不容易出现 Loss 变成 NaN（数值溢出）的报错。请务必保持 `bf16`。

![image-20251204190718700](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041907846.png)

#### 3.2.16 日志与保存配置 (Logging & Checkpointing)

- **配置项**：`日志间隔`
- **当前值**：`5`
- **详细解释**：
  - **作用**：每训练 5 步（Steps），在屏幕上打印一次当前的 Loss（损失值）和学习率。
  - **建议**：保持默认。这个频率能让你实时看到模型是不是“学废了”（Loss 暴涨）或者“学得好”（Loss 稳步下降），方便及时终止错误训练。
- **配置项**：`保存间隔`
- **当前值**：`100`
- **详细解释**：
  - **作用**：每训练 100 步，自动把当前训练好的模型权重保存为一个文件夹（Checkpoint）。
  - **硬盘空间预警**：虽然 LoRA 文件很小（几十 MB），但如果你的训练总步数很多（比如 10000 步），每 100 步存一次会产生 100 个文件夹，管理起来很麻烦。
  - **优化建议**：如果是正式训练，建议改为 `500` 或 `1000`，避免生成太多中间文件。

#### 3.2.17 预热步数 (Warmup Steps)

- **配置项**：`预热步数`
- **当前值**：`0`
- **详细解释**：
  - **含义**：在训练刚开始时，先用很小的学习率慢慢热身，然后再增加到 `1e-4`。
  - **作用**：防止模型一开始因为“步子迈太大”而跑偏。
  - **建议**：对于医疗推理这种复杂任务，建议设置为总步数的 10% 左右（例如填 `10` 或 `20`）。这能让模型更平滑地进入学习状态。填 `0` 也能跑，但不够稳健。

#### 3.2.18 额外参数深度解析 (Extra Arguments) —— **核心优化**

输入框中填写的这段代码是整个配置的“点睛之笔”：

`{"optim": "adamw_torch", "gradient_checkpointing": true}`

这两个参数对 A10 显卡的影响极大，下面逐一解释：

**1. 优化器设置 (`"optim": "adamw_torch"`)**

- **含义**：指定使用 PyTorch 官方实现的 AdamW 优化器。
- **作用**：
  - **稳定性**：这是最标准、兼容性最好的优化器实现。相比于一些魔改版（如 8bit-adam），它更不容易报错。
  - **显存代价**：它需要的显存稍多一点点，但在你的 24G 显卡 + 4-bit 量化环境下，这点开销完全可以接受。

**2. 梯度检查点 (`"gradient_checkpointing": true`) —— 省显存神器**

- **含义**：这是一种“以时间换空间”的技术。
  - **不开启时**：模型在向前计算（Forward）时，会把每一层的中间结果（Activations）都存满显存，留着给反向传播（Backward）用。这非常吃显存！
  - **开启后**：模型不存中间结果了。等需要反向传播时，再临时重新算一遍。
- **对 A10 (24G) 的巨大价值**：
  - **显存暴降**：开启此选项通常能节省 **20%** 的显存占用。
  - **代价**：训练速度会变慢约 20-30%。
  - **结论**：**必须开启**。对于单卡训练，为了能跑长文本（医疗病历），牺牲一点速度换取更大的上下文空间是非常划算的。

#### 3.2.19 NEFTune 噪声参数

- **配置项**：`NEFTune 噪声参数`
- **当前值**：`0`
- **详细解释**：
  - **作用**：一种给 Embedding 层加噪声的技术，能防止过拟合。
  - **初学者建议**：保持 `0`（关闭）。目前这个技术还在实验阶段，有时候会导致模型变得“胡言乱语”。对于初次上手，我们先追求“跑通”和“稳定”。

![image-20251204191239259](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041912330.png)

#### 3.2.20 LoRA 秩 (LoRA Rank)

- **配置项**：`LoRA 秩`
- **当前值**：`8`
- **详细解释**：
  - **含义**：可以理解为 LoRA 这个“外挂大脑”的容量或带宽。数值越大，模型能学到的复杂知识越多，但生成的权重文件（Adapter）也越大，显存占用稍高。
  - **A10 24G 建议**：
    - `8` 是 LLaMA-Factory 的默认值，非常节省资源，适合入门跑通流程。
    - **进阶建议**：针对 **Medical O1 Reasoning** 这种需要强逻辑推理的任务，`8` 可能略显单薄。如果您的显存还很富裕（结合 4-bit 量化通常是富裕的），建议尝试将此值调高至 **`16`** 或 **`32`**，甚至 **`64`**。这能显著提升模型对复杂医疗逻辑的拟合能力。

#### 3.2.21 LoRA 缩放系数 (LoRA Alpha)

- **配置项**：`LoRA 缩放系数`
- **当前值**：`16`
- **详细解释**：
  - **含义**：决定了 LoRA 学到的新权重对原模型影响的“倍率”。
  - **黄金法则**：通常设置为 `LoRA 秩` 的 2 倍。
  - **操作建议**：如果您把上面的 Rank 改成了 16，记得把这里的 Alpha 改成 32；如果 Rank 是 32，这里就填 64。保持 `Alpha = 2 * Rank` 这个比例是最稳健的做法。

#### 3.2.22 LoRA+ 学习率比例 (LoRA+ LR Ratio) —— **高级技巧**

- **配置项**：`LoRA+ 学习率比例`
- **当前值**：`16`
- **详细解释**：
  - **含义**：这是一个名为 **LoRA+** 的优化算法。它让 LoRA 的两个矩阵（矩阵 A 和矩阵 B）使用不同的学习率进行训练。
  - **作用**：研究表明，让矩阵 B 学得比矩阵 A 快（通常快 16 倍左右），能大幅提升训练效率和效果。
  - **环境影响**：开启这个选项（只要数值不为 0 即开启）不会增加显存消耗，属于“免费的午餐”。对于 Qwen 系列模型，保留默认值 `16` 效果通常很好。

#### 3.2.23 LoRA 作用模块 (Target Modules) —— **关键配置**

- **配置项**：`LoRA 作用模块`
- **当前值**：`all`
- **详细解释**：
  - **含义**：指定 LoRA 具体要挂载在模型的哪些层上（例如 Q、K、V 注意力层，或者 MLP 层）。
  - **当前设置 "all"**：这是一个非常激进且优秀的设置。它意味着 LoRA 会挂载到模型所有的线性层上（`q_proj`, `k_proj`, `v_proj`, `o_proj`, `gate_proj`, `up_proj`, `down_proj`）。
  - **对医疗推理的影响**：
    - 仅挂载 Q/V（传统做法）：模型只能学到一点皮毛。
    - **挂载 "all"**：模型能全方位地调整自己的逻辑处理能力。对于推理类（Reasoning）任务，必须选 `all` 才能获得最佳效果。
  - **硬件压力**：相比只挂载 Q/V，选择 `all` 会增加一些显存消耗和计算量。但在 **A10 (24G)** 上，这完全在承受范围内。**请务必保持 "all"**。

![image-20251204191714495](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512041917591.png)

### 3.3 启动微调
将输出目录修改为`train_qwen3_8b`，训练后的LoRA权重将会保存在此目录中。点击「预览命令」可展示所有已配置的参数，您如果想通过代码运行微调，可以复制这段命令，在命令行运行。

点击「开始」启动模型微调。
![image-20251203170030807](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512031700144.png)

启动微调后需要等待一段时间，待模型下载完毕后可在界面观察到训练进度和损失曲线。模型微调大约需要50分钟，显示“训练完毕”代表微调成功。

![image-20251204200651649](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042006782.png)

训练完整日志

```bash
Visit http://ip:port for Web UI, e.g., http://127.0.0.1:7860
* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
[WARNING|2025-12-04 19:05:54] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-12-04 19:05:54] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.bfloat16
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,370 >> loading file chat_template.jinja
[INFO|tokenization_utils_base.py:2299] 2025-12-04 19:05:54,685 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|configuration_utils.py:696] 2025-12-04 19:05:54,686 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:05:54,689 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,689 >> loading file vocab.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,689 >> loading file merges.txt
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,689 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,689 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,689 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,689 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2021] 2025-12-04 19:05:54,690 >> loading file chat_template.jinja
[INFO|tokenization_utils_base.py:2299] 2025-12-04 19:05:55,018 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|2025-12-04 19:05:55] llamafactory.data.loader:143 >> Loading dataset /mnt/workspace/medical_o1_sft_Chinese_alpaca_cot.jsonl...
training example:
input_ids:
[151644, 872, 198, 100345, 53481, 3837, 46944, 16, 104252, 99657, 18493, 106084, 117405, 100347, 42140, 44290, 30709, 36885, 55502, 3837, 101930, 16530, 100939, 39762, 3837, 100136, 99601, 116066, 26288, 29524, 100711, 3837, 101776, 99577, 88653, 115623, 3837, 39426, 16530, 118609, 3837, 117405, 16872, 18830, 34794, 100743, 3837, 99811, 44290, 102010, 49185, 99696, 1773, 100137, 116067, 18493, 104823, 15946, 105262, 100678, 99252, 11319, 220, 14880, 104137, 113272, 62926, 107485, 102349, 8997, 99487, 109661, 18493, 104797, 117405, 17447, 45861, 112312, 30709, 36885, 55502, 3837, 99725, 105562, 52801, 3837, 103957, 105231, 115623, 67279, 3837, 88653, 117633, 115623, 1773, 105839, 104797, 100624, 99259, 3837, 87267, 33108, 100102, 99259, 101063, 1773, 99306, 14777, 92015, 104006, 99406, 3837, 112575, 102347, 102441, 99193, 3837, 104797, 9370, 100102, 99259, 70927, 99308, 80158, 99873, 100585, 34187, 101099, 3407, 11622, 104823, 106413, 100192, 3837, 100347, 30709, 36885, 55502, 5373, 105566, 101930, 16530, 100939, 39762, 3837, 100001, 101368, 104029, 114316, 64355, 116066, 1773, 109661, 111335, 49828, 100001, 116771, 3837, 99558, 99519, 100102, 99259, 18493, 31914, 20742, 100601, 36885, 3407, 77288, 87256, 101997, 3837, 117405, 16872, 100626, 34794, 100743, 3837, 43288, 87267, 106015, 20412, 105172, 64355, 116066, 1773, 104544, 106141, 101174, 105806, 3837, 110090, 115623, 101561, 70927, 117083, 1773, 101893, 99559, 104823, 15946, 104854, 105792, 53556, 123, 116066, 100631, 100102, 116066, 3837, 74763, 104560, 109851, 99559, 3407, 49567, 100158, 3837, 117405, 101913, 34794, 100743, 33108, 102010, 49185, 99696, 118329, 101160, 99461, 100403, 26939, 117405, 16872, 3837, 100346, 99520, 66394, 102410, 20412, 88653, 25074, 57191, 121813, 120201, 11319, 100001, 101419, 38953, 53481, 107200, 57191, 117591, 9370, 101120, 101304, 3837, 104050, 18830, 32108, 115623, 16530, 100939, 39762, 3837, 99518, 101894, 102257, 57191, 34794, 100743, 102072, 3407, 104857, 105839, 3837, 35946, 99494, 100681, 100001, 101368, 33126, 111892, 121813, 120201, 105437, 11319, 101150, 106350, 104661, 106021, 33108, 104797, 106806, 105419, 33071, 100741, 3837, 100102, 99259, 104560, 35568, 62112, 3837, 77288, 87267, 103943, 79599, 99586, 100631, 110776, 100102, 105998, 101742, 99337, 3407, 102112, 99797, 111492, 53481, 108734, 3837, 100137, 101930, 16530, 100939, 39762, 99518, 106888, 104215, 3837, 62244, 100374, 104823, 33126, 99835, 29258, 9370, 99252, 13072, 3837, 104074, 102410, 20412, 102031, 33126, 113879, 9370, 101304, 26850, 87256, 101118, 100158, 3837, 43288, 99730, 99520, 112507, 121813, 120201, 3837, 49828, 104857, 101042, 117405, 49185, 99696, 62926, 100347, 34794, 100743, 101893, 101120, 101368, 1773, 104823, 69249, 64355, 3837, 101893, 101107, 87267, 33126, 101137, 14009, 101782, 116066, 527, 57191, 14009, 64355, 120471, 527, 1773, 100001, 99252, 13072, 102119, 53481, 107200, 101120, 101304, 104813, 101776, 102581, 33108, 99877, 100250, 99561, 3407, 101997, 105419, 33108, 104661, 108372, 3837, 104797, 99518, 100102, 99518, 99259, 3837, 47815, 102494, 104892, 99873, 17254, 107200, 3837, 111309, 99899, 99932, 9370, 102778, 72448, 115781, 104036, 1773, 64355, 120471, 99487, 99252, 13072, 115371, 101228, 99322, 39762, 3837, 108884, 53481, 9370, 101304, 101120, 3837, 101776, 102581, 26939, 100347, 34794, 100743, 3407, 100632, 3837, 104857, 115104, 33447, 99879, 3837, 100626, 18947, 99252, 13072, 101994, 104652, 100968, 3837, 105792, 14009, 121617, 120657, 119740, 527, 3837, 43288, 99252, 18493, 104823, 69249, 95411, 63367, 65101, 100137, 101120, 101304, 62926, 115563, 99194, 32948, 34794, 100743, 102072, 1773, 99652, 74763, 114369, 32108, 115623, 33108, 102010, 49185, 99696, 100001, 101368, 3407, 104170, 3837, 75882, 99670, 20412, 106084, 100102, 99259, 3837, 100673, 100102, 99586, 110540, 3837, 104661, 108372, 53153, 101515, 3837, 41146, 106141, 99185, 12857, 101893, 101304, 11319, 99799, 101042, 33447, 104288, 14009, 121617, 120657, 119740, 527, 99487, 99252, 13072, 101228, 102084, 101137, 1773, 151645, 198, 151644, 77091, 198, 151667, 198, 45181, 104823, 106413, 100192, 3837, 56568, 31838, 53481, 111492, 101137, 2073, 121617, 120657, 119740, 97907, 116067, 1773, 100137, 116067, 102119, 108132, 117405, 3837, 110966, 42140, 44290, 36885, 55502, 3837, 101776, 99577, 88653, 115623, 3837, 101894, 34794, 100743, 3837, 99811, 44290, 102010, 49185, 99696, 100136, 101930, 16530, 100939, 39762, 1773, 100102, 99259, 99260, 29258, 9370, 106084, 108478, 100673, 100137, 116067, 103949, 3837, 111625, 112575, 99260, 99932, 9370, 101939, 103996, 1773, 101898, 100374, 104823, 9370, 79766, 99259, 49238, 99586, 5373, 103458, 100102, 32320, 101561, 9370, 114464, 71817, 54542, 90395, 102035, 104715, 100182, 101898, 71817, 100700, 105262, 33108, 101899, 8997, 151668, 271, 45181, 104823, 106413, 100192, 3837, 56568, 31838, 53481, 111492, 101137, 2073, 121617, 120657, 119740, 97907, 116067, 1773, 100137, 116067, 102119, 108132, 117405, 3837, 110966, 42140, 44290, 36885, 55502, 3837, 101776, 99577, 88653, 115623, 3837, 101894, 34794, 100743, 3837, 99811, 44290, 102010, 49185, 99696, 100136, 101930, 16530, 100939, 39762, 1773, 100102, 99259, 99260, 29258, 9370, 106084, 108478, 100673, 100137, 116067, 103949, 3837, 111625, 112575, 99260, 99932, 9370, 101939, 103996, 1773, 101898, 100374, 104823, 9370, 79766, 99259, 49238, 99586, 5373, 103458, 100102, 32320, 101561, 9370, 114464, 71817, 54542, 90395, 102035, 104715, 100182, 101898, 71817, 100700, 105262, 33108, 101899, 1773, 151645, 198]
inputs:
<|im_start|>user
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？ 请逐步推理并给出答案。
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。<|im_end|>
<|im_start|>assistant
<think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
</think>

从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。<|im_end|>

label_ids:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 151667, 198, 45181, 104823, 106413, 100192, 3837, 56568, 31838, 53481, 111492, 101137, 2073, 121617, 120657, 119740, 97907, 116067, 1773, 100137, 116067, 102119, 108132, 117405, 3837, 110966, 42140, 44290, 36885, 55502, 3837, 101776, 99577, 88653, 115623, 3837, 101894, 34794, 100743, 3837, 99811, 44290, 102010, 49185, 99696, 100136, 101930, 16530, 100939, 39762, 1773, 100102, 99259, 99260, 29258, 9370, 106084, 108478, 100673, 100137, 116067, 103949, 3837, 111625, 112575, 99260, 99932, 9370, 101939, 103996, 1773, 101898, 100374, 104823, 9370, 79766, 99259, 49238, 99586, 5373, 103458, 100102, 32320, 101561, 9370, 114464, 71817, 54542, 90395, 102035, 104715, 100182, 101898, 71817, 100700, 105262, 33108, 101899, 8997, 151668, 271, 45181, 104823, 106413, 100192, 3837, 56568, 31838, 53481, 111492, 101137, 2073, 121617, 120657, 119740, 97907, 116067, 1773, 100137, 116067, 102119, 108132, 117405, 3837, 110966, 42140, 44290, 36885, 55502, 3837, 101776, 99577, 88653, 115623, 3837, 101894, 34794, 100743, 3837, 99811, 44290, 102010, 49185, 99696, 100136, 101930, 16530, 100939, 39762, 1773, 100102, 99259, 99260, 29258, 9370, 106084, 108478, 100673, 100137, 116067, 103949, 3837, 111625, 112575, 99260, 99932, 9370, 101939, 103996, 1773, 101898, 100374, 104823, 9370, 79766, 99259, 49238, 99586, 5373, 103458, 100102, 32320, 101561, 9370, 114464, 71817, 54542, 90395, 102035, 104715, 100182, 101898, 71817, 100700, 105262, 33108, 101899, 1773, 151645, 198]
labels:
<think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
</think>

从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。<|im_end|>

[INFO|configuration_utils.py:696] 2025-12-04 19:05:56,156 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:05:56,157 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|2025-12-04 19:05:56] llamafactory.model.model_utils.quantization:143 >> Quantizing model to 4 bit with bitsandbytes.
[INFO|2025-12-04 19:05:56] llamafactory.model.model_utils.kv_cache:143 >> KV cache is disabled during training.
[INFO|modeling_utils.py:1148] 2025-12-04 19:05:57,751 >> loading weights file /mnt/workspace/Qwen3-8B/model.safetensors.index.json
[INFO|modeling_utils.py:2241] 2025-12-04 19:05:57,752 >> Instantiating Qwen3ForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1135] 2025-12-04 19:05:57,753 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "use_cache": false
}

Loading checkpoint shards: 100%|██████████████████| 5/5 [00:10<00:00,  2.15s/it]
[INFO|modeling_utils.py:5131] 2025-12-04 19:06:08,665 >> All model checkpoint weights were used when initializing Qwen3ForCausalLM.

[INFO|modeling_utils.py:5139] 2025-12-04 19:06:08,665 >> All the weights of Qwen3ForCausalLM were initialized from the model checkpoint at /mnt/workspace/Qwen3-8B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen3ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1088] 2025-12-04 19:06:08,669 >> loading configuration file /mnt/workspace/Qwen3-8B/generation_config.json
[INFO|configuration_utils.py:1135] 2025-12-04 19:06:08,669 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

[INFO|2025-12-04 19:06:08] llamafactory.model.model_utils.checkpointing:143 >> Gradient checkpointing enabled.
[INFO|2025-12-04 19:06:08] llamafactory.model.model_utils.attention:143 >> Using torch SDPA for faster training and inference.
[INFO|2025-12-04 19:06:08] llamafactory.model.adapter:143 >> Upcasting trainable params to float32.
[INFO|2025-12-04 19:06:08] llamafactory.model.adapter:143 >> Fine-tuning method: LoRA
[INFO|2025-12-04 19:06:08] llamafactory.model.model_utils.misc:143 >> Found linear modules: q_proj,down_proj,o_proj,k_proj,v_proj,gate_proj,up_proj
[INFO|2025-12-04 19:06:09] llamafactory.model.loader:143 >> trainable params: 21,823,488 || all params: 8,212,558,848 || trainable%: 0.2657
[INFO|trainer.py:756] 2025-12-04 19:06:09,247 >> Using auto half precision backend
[INFO|2025-12-04 19:06:09] llamafactory.train.trainer_utils:143 >> Using LoRA+ optimizer with loraplus lr ratio 16.00.
[INFO|trainer.py:2409] 2025-12-04 19:06:09,529 >> ***** Running training *****
[INFO|trainer.py:2410] 2025-12-04 19:06:09,529 >>   Num examples = 850
[INFO|trainer.py:2411] 2025-12-04 19:06:09,529 >>   Num Epochs = 3
[INFO|trainer.py:2412] 2025-12-04 19:06:09,529 >>   Instantaneous batch size per device = 1
[INFO|trainer.py:2415] 2025-12-04 19:06:09,529 >>   Total train batch size (w. parallel, distributed & accumulation) = 4
[INFO|trainer.py:2416] 2025-12-04 19:06:09,529 >>   Gradient Accumulation steps = 4
[INFO|trainer.py:2417] 2025-12-04 19:06:09,529 >>   Total optimization steps = 639
[INFO|trainer.py:2418] 2025-12-04 19:06:09,535 >>   Number of trainable parameters = 21,823,488
  1%|▎                                          | 5/639 [00:20<42:31,  4.02s/it][INFO|2025-12-04 19:06:30] llamafactory.train.callbacks:143 >> {'loss': 0.6960, 'learning_rate': 9.9990e-05, 'epoch': 0.02, 'throughput': 697.13}
{'loss': 0.696, 'grad_norm': 0.23502613604068756, 'learning_rate': 9.999033183566353e-05, 'epoch': 0.02, 'num_input_tokens_seen': 14376, 'train_runtime': 20.6278, 'train_tokens_per_second': 696.923}
  2%|▋                                         | 10/639 [00:40<42:08,  4.02s/it][INFO|2025-12-04 19:06:50] llamafactory.train.callbacks:143 >> {'loss': 0.5438, 'learning_rate': 9.9951e-05, 'epoch': 0.05, 'throughput': 699.14}
{'loss': 0.5438, 'grad_norm': 0.3318701684474945, 'learning_rate': 9.995106132599869e-05, 'epoch': 0.05, 'num_input_tokens_seen': 28296, 'train_runtime': 40.4784, 'train_tokens_per_second': 699.039}
  2%|▉                                         | 15/639 [01:04<48:16,  4.64s/it][INFO|2025-12-04 19:07:13] llamafactory.train.callbacks:143 >> {'loss': 0.5772, 'learning_rate': 9.9882e-05, 'epoch': 0.07, 'throughput': 701.83}
{'loss': 0.5772, 'grad_norm': 0.2540542483329773, 'learning_rate': 9.988160792165562e-05, 'epoch': 0.07, 'num_input_tokens_seen': 44992, 'train_runtime': 64.1129, 'train_tokens_per_second': 701.762}
  3%|█▎                                        | 20/639 [01:25<43:45,  4.24s/it][INFO|2025-12-04 19:07:34] llamafactory.train.callbacks:143 >> {'loss': 0.6002, 'learning_rate': 9.9782e-05, 'epoch': 0.09, 'throughput': 700.39}
{'loss': 0.6002, 'grad_norm': 0.38846877217292786, 'learning_rate': 9.978201358980645e-05, 'epoch': 0.09, 'num_input_tokens_seen': 59576, 'train_runtime': 85.0673, 'train_tokens_per_second': 700.34}
  4%|█▋                                        | 25/639 [01:46<42:56,  4.20s/it][INFO|2025-12-04 19:07:55] llamafactory.train.callbacks:143 >> {'loss': 0.5981, 'learning_rate': 9.9652e-05, 'epoch': 0.12, 'throughput': 698.84}
{'loss': 0.5981, 'grad_norm': 0.3376992642879486, 'learning_rate': 9.965233851025814e-05, 'epoch': 0.12, 'num_input_tokens_seen': 74240, 'train_runtime': 106.2394, 'train_tokens_per_second': 698.799}
  5%|█▉                                        | 30/639 [02:08<44:40,  4.40s/it][INFO|2025-12-04 19:08:18] llamafactory.train.callbacks:143 >> {'loss': 0.6136, 'learning_rate': 9.9493e-05, 'epoch': 0.14, 'throughput': 698.80}
{'loss': 0.6136, 'grad_norm': 0.23448772728443146, 'learning_rate': 9.949266103908895e-05, 'epoch': 0.14, 'num_input_tokens_seen': 89944, 'train_runtime': 128.7181, 'train_tokens_per_second': 698.767}
  5%|██▎                                       | 35/639 [02:30<43:42,  4.34s/it][INFO|2025-12-04 19:08:40] llamafactory.train.callbacks:143 >> {'loss': 0.6039, 'learning_rate': 9.9303e-05, 'epoch': 0.16, 'throughput': 698.91}
{'loss': 0.6039, 'grad_norm': 0.25201863050460815, 'learning_rate': 9.930307766130169e-05, 'epoch': 0.16, 'num_input_tokens_seen': 105288, 'train_runtime': 150.6526, 'train_tokens_per_second': 698.879}
  6%|██▋                                       | 40/639 [02:51<41:29,  4.16s/it][INFO|2025-12-04 19:09:00] llamafactory.train.callbacks:143 >> {'loss': 0.5387, 'learning_rate': 9.9084e-05, 'epoch': 0.19, 'throughput': 697.32}
{'loss': 0.5387, 'grad_norm': 0.33039793372154236, 'learning_rate': 9.90837029325229e-05, 'epoch': 0.19, 'num_input_tokens_seen': 119432, 'train_runtime': 171.2783, 'train_tokens_per_second': 697.298}
  7%|██▉                                       | 45/639 [03:10<38:25,  3.88s/it][INFO|2025-12-04 19:09:19] llamafactory.train.callbacks:143 >> {'loss': 0.5764, 'learning_rate': 9.8835e-05, 'epoch': 0.21, 'throughput': 696.40}
{'loss': 0.5764, 'grad_norm': 0.2545849680900574, 'learning_rate': 9.883466940978252e-05, 'epoch': 0.21, 'num_input_tokens_seen': 132576, 'train_runtime': 190.3782, 'train_tokens_per_second': 696.382}
  8%|███▎                                      | 50/639 [03:29<39:13,  4.00s/it][INFO|2025-12-04 19:09:39] llamafactory.train.callbacks:143 >> {'loss': 0.5287, 'learning_rate': 9.8556e-05, 'epoch': 0.24, 'throughput': 695.89}
{'loss': 0.5287, 'grad_norm': 0.28507527709007263, 'learning_rate': 9.855612757141655e-05, 'epoch': 0.24, 'num_input_tokens_seen': 146112, 'train_runtime': 209.9695, 'train_tokens_per_second': 695.872}
  9%|███▌                                      | 55/639 [03:52<43:40,  4.49s/it][INFO|2025-12-04 19:10:02] llamafactory.train.callbacks:143 >> {'loss': 0.5662, 'learning_rate': 9.8248e-05, 'epoch': 0.26, 'throughput': 695.63}
{'loss': 0.5662, 'grad_norm': 0.3804457485675812, 'learning_rate': 9.824824572614051e-05, 'epoch': 0.26, 'num_input_tokens_seen': 162048, 'train_runtime': 232.9568, 'train_tokens_per_second': 695.614}
  9%|███▉                                      | 60/639 [04:13<40:41,  4.22s/it][INFO|2025-12-04 19:10:23] llamafactory.train.callbacks:143 >> {'loss': 0.6070, 'learning_rate': 9.7911e-05, 'epoch': 0.28, 'throughput': 693.66}
{'loss': 0.607, 'grad_norm': 0.2477499544620514, 'learning_rate': 9.791120991134904e-05, 'epoch': 0.28, 'num_input_tokens_seen': 176064, 'train_runtime': 253.8233, 'train_tokens_per_second': 693.648}
 10%|████▎                                     | 65/639 [04:34<39:16,  4.11s/it][INFO|2025-12-04 19:10:43] llamafactory.train.callbacks:143 >> {'loss': 0.5673, 'learning_rate': 9.7545e-05, 'epoch': 0.31, 'throughput': 692.77}
{'loss': 0.5673, 'grad_norm': 0.2974146902561188, 'learning_rate': 9.754522378070297e-05, 'epoch': 0.31, 'num_input_tokens_seen': 190008, 'train_runtime': 274.2796, 'train_tokens_per_second': 692.753}
 11%|████▌                                     | 70/639 [04:53<36:35,  3.86s/it][INFO|2025-12-04 19:11:02] llamafactory.train.callbacks:143 >> {'loss': 0.5041, 'learning_rate': 9.7151e-05, 'epoch': 0.33, 'throughput': 691.84}
{'loss': 0.5041, 'grad_norm': 0.35126590728759766, 'learning_rate': 9.715050848107168e-05, 'epoch': 0.33, 'num_input_tokens_seen': 203000, 'train_runtime': 293.4279, 'train_tokens_per_second': 691.822}
 12%|████▉                                     | 75/639 [05:15<39:34,  4.21s/it][INFO|2025-12-04 19:11:24] llamafactory.train.callbacks:143 >> {'loss': 0.5521, 'learning_rate': 9.6727e-05, 'epoch': 0.35, 'throughput': 691.33}
{'loss': 0.5521, 'grad_norm': 0.9560633897781372, 'learning_rate': 9.67273025189053e-05, 'epoch': 0.35, 'num_input_tokens_seen': 217824, 'train_runtime': 315.0869, 'train_tokens_per_second': 691.314}
 13%|█████▎                                    | 80/639 [05:35<39:50,  4.28s/it][INFO|2025-12-04 19:11:45] llamafactory.train.callbacks:143 >> {'loss': 0.5734, 'learning_rate': 9.6276e-05, 'epoch': 0.38, 'throughput': 690.69}
{'loss': 0.5734, 'grad_norm': 0.5834539532661438, 'learning_rate': 9.627586161611732e-05, 'epoch': 0.38, 'num_input_tokens_seen': 231792, 'train_runtime': 335.5991, 'train_tokens_per_second': 690.681}
 13%|█████▌                                    | 85/639 [05:56<39:50,  4.32s/it][INFO|2025-12-04 19:12:06] llamafactory.train.callbacks:143 >> {'loss': 0.5538, 'learning_rate': 9.5796e-05, 'epoch': 0.40, 'throughput': 689.84}
{'loss': 0.5538, 'grad_norm': 0.30696722865104675, 'learning_rate': 9.57964585555648e-05, 'epoch': 0.4, 'num_input_tokens_seen': 245992, 'train_runtime': 356.6004, 'train_tokens_per_second': 689.825}
 14%|█████▉                                    | 90/639 [06:17<37:45,  4.13s/it][INFO|2025-12-04 19:12:27] llamafactory.train.callbacks:143 >> {'loss': 0.5719, 'learning_rate': 9.5289e-05, 'epoch': 0.42, 'throughput': 689.06}
{'loss': 0.5719, 'grad_norm': 0.2754083275794983, 'learning_rate': 9.528938301621956e-05, 'epoch': 0.42, 'num_input_tokens_seen': 260376, 'train_runtime': 377.8787, 'train_tokens_per_second': 689.046}
 15%|██████▏                                   | 95/639 [06:39<39:08,  4.32s/it][INFO|2025-12-04 19:12:48] llamafactory.train.callbacks:143 >> {'loss': 0.5814, 'learning_rate': 9.4755e-05, 'epoch': 0.45, 'throughput': 688.40}
{'loss': 0.5814, 'grad_norm': 0.2978734076023102, 'learning_rate': 9.475494139812979e-05, 'epoch': 0.45, 'num_input_tokens_seen': 274824, 'train_runtime': 399.2287, 'train_tokens_per_second': 688.387}
 16%|██████▍                                  | 100/639 [07:00<38:17,  4.26s/it][INFO|2025-12-04 19:13:09] llamafactory.train.callbacks:143 >> {'loss': 0.6039, 'learning_rate': 9.4193e-05, 'epoch': 0.47, 'throughput': 688.69}
{'loss': 0.6039, 'grad_norm': 0.5732434988021851, 'learning_rate': 9.419345663727805e-05, 'epoch': 0.47, 'num_input_tokens_seen': 289392, 'train_runtime': 420.2147, 'train_tokens_per_second': 688.677}
 16%|██████▍                                  | 100/639 [07:00<38:17,  4.26s/it][INFO|trainer.py:4327] 2025-12-04 19:13:09,755 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:13:09,755 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:13:09,755 >>   Batch size = 1

  0%|                                                   | 0/150 [00:00<?, ?it/s]
  1%|▌                                          | 2/150 [00:00<00:19,  7.62it/s]
  2%|▊                                          | 3/150 [00:00<00:27,  5.32it/s]
  3%|█▏                                         | 4/150 [00:00<00:37,  3.94it/s]
  3%|█▍                                         | 5/150 [00:01<00:38,  3.72it/s]
  4%|█▋                                         | 6/150 [00:01<00:39,  3.60it/s]
  5%|██                                         | 7/150 [00:01<00:40,  3.51it/s]
  5%|██▎                                        | 8/150 [00:02<00:41,  3.44it/s]
  6%|██▌                                        | 9/150 [00:02<00:39,  3.55it/s]
  7%|██▊                                       | 10/150 [00:02<00:40,  3.49it/s]
  7%|███                                       | 11/150 [00:02<00:40,  3.45it/s]
  8%|███▎                                      | 12/150 [00:03<00:40,  3.43it/s]
  9%|███▋                                      | 13/150 [00:03<00:42,  3.21it/s]
  9%|███▉                                      | 14/150 [00:03<00:43,  3.09it/s]
 10%|████▏                                     | 15/150 [00:04<00:42,  3.17it/s]
 11%|████▍                                     | 16/150 [00:04<00:45,  2.96it/s]
 11%|████▊                                     | 17/150 [00:04<00:43,  3.07it/s]
 12%|█████                                     | 18/150 [00:05<00:42,  3.14it/s]
 13%|█████▎                                    | 19/150 [00:05<00:40,  3.20it/s]
 13%|█████▌                                    | 20/150 [00:05<00:42,  3.09it/s]
 14%|█████▉                                    | 21/150 [00:06<00:37,  3.41it/s]
 15%|██████▏                                   | 22/150 [00:06<00:34,  3.71it/s]
 15%|██████▍                                   | 23/150 [00:06<00:37,  3.40it/s]
 16%|██████▋                                   | 24/150 [00:06<00:37,  3.38it/s]
 17%|███████                                   | 25/150 [00:07<00:37,  3.36it/s]
 17%|███████▎                                  | 26/150 [00:07<00:35,  3.48it/s]
 18%|███████▌                                  | 27/150 [00:07<00:35,  3.49it/s]
 19%|███████▊                                  | 28/150 [00:08<00:37,  3.28it/s]
 19%|████████                                  | 29/150 [00:08<00:38,  3.13it/s]
 20%|████████▍                                 | 30/150 [00:08<00:37,  3.23it/s]
 21%|████████▋                                 | 31/150 [00:09<00:35,  3.37it/s]
 21%|████████▉                                 | 32/150 [00:09<00:31,  3.71it/s]
 22%|█████████▏                                | 33/150 [00:09<00:35,  3.32it/s]
 23%|█████████▌                                | 34/150 [00:09<00:33,  3.45it/s]
 23%|█████████▊                                | 35/150 [00:10<00:35,  3.24it/s]
 24%|██████████                                | 36/150 [00:10<00:32,  3.54it/s]
 25%|██████████▎                               | 37/150 [00:10<00:35,  3.17it/s]
 25%|██████████▋                               | 38/150 [00:11<00:34,  3.22it/s]
 26%|██████████▉                               | 39/150 [00:11<00:35,  3.10it/s]
 27%|███████████▏                              | 40/150 [00:11<00:34,  3.18it/s]
 27%|███████████▍                              | 41/150 [00:12<00:33,  3.28it/s]
 28%|███████████▊                              | 42/150 [00:12<00:32,  3.35it/s]
 29%|████████████                              | 43/150 [00:12<00:31,  3.37it/s]
 29%|████████████▎                             | 44/150 [00:13<00:35,  2.96it/s]
 30%|████████████▌                             | 45/150 [00:13<00:33,  3.11it/s]
 31%|████████████▉                             | 46/150 [00:13<00:35,  2.91it/s]
 31%|█████████████▏                            | 47/150 [00:14<00:31,  3.29it/s]
 32%|█████████████▍                            | 48/150 [00:14<00:31,  3.28it/s]
 33%|█████████████▋                            | 49/150 [00:14<00:29,  3.39it/s]
 33%|██████████████                            | 50/150 [00:14<00:29,  3.42it/s]
 34%|██████████████▎                           | 51/150 [00:15<00:28,  3.46it/s]
 35%|██████████████▌                           | 52/150 [00:15<00:32,  3.04it/s]
 35%|██████████████▊                           | 53/150 [00:15<00:29,  3.25it/s]
 36%|███████████████                           | 54/150 [00:16<00:28,  3.35it/s]
 37%|███████████████▍                          | 55/150 [00:16<00:27,  3.46it/s]
 37%|███████████████▋                          | 56/150 [00:16<00:30,  3.13it/s]
 38%|███████████████▉                          | 57/150 [00:17<00:30,  3.05it/s]
 39%|████████████████▏                         | 58/150 [00:17<00:29,  3.16it/s]
 39%|████████████████▌                         | 59/150 [00:17<00:27,  3.29it/s]
 40%|████████████████▊                         | 60/150 [00:17<00:27,  3.31it/s]
 41%|█████████████████                         | 61/150 [00:18<00:26,  3.31it/s]
 41%|█████████████████▎                        | 62/150 [00:18<00:26,  3.31it/s]
 42%|█████████████████▋                        | 63/150 [00:18<00:26,  3.30it/s]
 43%|█████████████████▉                        | 64/150 [00:19<00:24,  3.58it/s]
 43%|██████████████████▏                       | 65/150 [00:19<00:24,  3.52it/s]
 44%|██████████████████▍                       | 66/150 [00:19<00:25,  3.26it/s]
 45%|██████████████████▊                       | 67/150 [00:20<00:28,  2.90it/s]
 45%|███████████████████                       | 68/150 [00:20<00:27,  3.02it/s]
 46%|███████████████████▎                      | 69/150 [00:20<00:24,  3.35it/s]
 47%|███████████████████▌                      | 70/150 [00:20<00:23,  3.45it/s]
 47%|███████████████████▉                      | 71/150 [00:21<00:21,  3.72it/s]
 48%|████████████████████▏                     | 72/150 [00:21<00:19,  4.01it/s]
 49%|████████████████████▍                     | 73/150 [00:21<00:20,  3.79it/s]
 49%|████████████████████▋                     | 74/150 [00:21<00:19,  3.98it/s]
 50%|█████████████████████                     | 75/150 [00:22<00:19,  3.76it/s]
 51%|█████████████████████▎                    | 76/150 [00:22<00:22,  3.30it/s]
 51%|█████████████████████▌                    | 77/150 [00:22<00:20,  3.62it/s]
 52%|█████████████████████▊                    | 78/150 [00:23<00:20,  3.59it/s]
 53%|██████████████████████                    | 79/150 [00:23<00:20,  3.52it/s]
 53%|██████████████████████▍                   | 80/150 [00:23<00:19,  3.57it/s]
 54%|██████████████████████▋                   | 81/150 [00:23<00:18,  3.83it/s]
 55%|██████████████████████▉                   | 82/150 [00:24<00:19,  3.48it/s]
 55%|███████████████████████▏                  | 83/150 [00:24<00:19,  3.50it/s]
 56%|███████████████████████▌                  | 84/150 [00:24<00:20,  3.27it/s]
 57%|███████████████████████▊                  | 85/150 [00:25<00:19,  3.36it/s]
 57%|████████████████████████                  | 86/150 [00:25<00:21,  3.02it/s]
 58%|████████████████████████▎                 | 87/150 [00:25<00:21,  2.93it/s]
 59%|████████████████████████▋                 | 88/150 [00:26<00:20,  3.05it/s]
 59%|████████████████████████▉                 | 89/150 [00:26<00:21,  2.78it/s]
 60%|█████████████████████████▏                | 90/150 [00:26<00:19,  3.01it/s]
 61%|█████████████████████████▍                | 91/150 [00:27<00:20,  2.94it/s]
 61%|█████████████████████████▊                | 92/150 [00:27<00:19,  2.90it/s]
 62%|██████████████████████████                | 93/150 [00:27<00:17,  3.24it/s]
 63%|██████████████████████████▎               | 94/150 [00:28<00:16,  3.34it/s]
 63%|██████████████████████████▌               | 95/150 [00:28<00:16,  3.33it/s]
 64%|██████████████████████████▉               | 96/150 [00:28<00:17,  3.14it/s]
 65%|███████████████████████████▏              | 97/150 [00:29<00:16,  3.20it/s]
 65%|███████████████████████████▍              | 98/150 [00:29<00:16,  3.23it/s]
 66%|███████████████████████████▋              | 99/150 [00:29<00:16,  3.12it/s]
 67%|███████████████████████████▎             | 100/150 [00:30<00:17,  2.88it/s]
 67%|███████████████████████████▌             | 101/150 [00:30<00:16,  3.06it/s]
 68%|███████████████████████████▉             | 102/150 [00:30<00:14,  3.39it/s]
 69%|████████████████████████████▏            | 103/150 [00:31<00:15,  3.09it/s]
 69%|████████████████████████████▍            | 104/150 [00:31<00:14,  3.17it/s]
 70%|████████████████████████████▋            | 105/150 [00:31<00:13,  3.34it/s]
 71%|████████████████████████████▉            | 106/150 [00:31<00:11,  3.78it/s]
 71%|█████████████████████████████▏           | 107/150 [00:32<00:11,  3.59it/s]
 72%|█████████████████████████████▌           | 108/150 [00:32<00:11,  3.58it/s]
 73%|█████████████████████████████▊           | 109/150 [00:32<00:12,  3.21it/s]
 73%|██████████████████████████████           | 110/150 [00:33<00:12,  3.25it/s]
 74%|██████████████████████████████▎          | 111/150 [00:33<00:11,  3.28it/s]
 75%|██████████████████████████████▌          | 112/150 [00:33<00:11,  3.33it/s]
 75%|██████████████████████████████▉          | 113/150 [00:34<00:11,  3.17it/s]
 76%|███████████████████████████████▏         | 114/150 [00:34<00:10,  3.33it/s]
 77%|███████████████████████████████▍         | 115/150 [00:34<00:09,  3.63it/s]
 77%|███████████████████████████████▋         | 116/150 [00:34<00:10,  3.35it/s]
 78%|███████████████████████████████▉         | 117/150 [00:35<00:10,  3.06it/s]
 79%|████████████████████████████████▎        | 118/150 [00:35<00:10,  3.19it/s]
 79%|████████████████████████████████▌        | 119/150 [00:35<00:09,  3.24it/s]
 80%|████████████████████████████████▊        | 120/150 [00:36<00:09,  3.09it/s]
 81%|█████████████████████████████████        | 121/150 [00:36<00:09,  3.20it/s]
 81%|█████████████████████████████████▎       | 122/150 [00:36<00:09,  3.05it/s]
 82%|█████████████████████████████████▌       | 123/150 [00:37<00:08,  3.23it/s]
 83%|█████████████████████████████████▉       | 124/150 [00:37<00:07,  3.36it/s]
 83%|██████████████████████████████████▏      | 125/150 [00:37<00:07,  3.47it/s]
 84%|██████████████████████████████████▍      | 126/150 [00:37<00:06,  3.43it/s]
 85%|██████████████████████████████████▋      | 127/150 [00:38<00:07,  3.20it/s]
 85%|██████████████████████████████████▉      | 128/150 [00:38<00:06,  3.35it/s]
 86%|███████████████████████████████████▎     | 129/150 [00:38<00:05,  3.63it/s]
 87%|███████████████████████████████████▌     | 130/150 [00:39<00:05,  3.54it/s]
 87%|███████████████████████████████████▊     | 131/150 [00:39<00:05,  3.30it/s]
 88%|████████████████████████████████████     | 132/150 [00:39<00:05,  3.16it/s]
 89%|████████████████████████████████████▎    | 133/150 [00:40<00:05,  3.30it/s]
 89%|████████████████████████████████████▋    | 134/150 [00:40<00:04,  3.36it/s]
 90%|████████████████████████████████████▉    | 135/150 [00:40<00:04,  3.42it/s]
 91%|█████████████████████████████████████▏   | 136/150 [00:40<00:04,  3.40it/s]
 91%|█████████████████████████████████████▍   | 137/150 [00:41<00:04,  2.98it/s]
 92%|█████████████████████████████████████▋   | 138/150 [00:41<00:03,  3.19it/s]
 93%|█████████████████████████████████████▉   | 139/150 [00:41<00:03,  3.05it/s]
 93%|██████████████████████████████████████▎  | 140/150 [00:42<00:03,  3.17it/s]
 94%|██████████████████████████████████████▌  | 141/150 [00:42<00:03,  2.96it/s]
 95%|██████████████████████████████████████▊  | 142/150 [00:42<00:02,  3.06it/s]
 95%|███████████████████████████████████████  | 143/150 [00:43<00:02,  3.15it/s]
 96%|███████████████████████████████████████▎ | 144/150 [00:43<00:01,  3.22it/s]
 97%|███████████████████████████████████████▋ | 145/150 [00:43<00:01,  3.28it/s]
 97%|███████████████████████████████████████▉ | 146/150 [00:44<00:01,  3.61it/s]
 98%|████████████████████████████████████████▏| 147/150 [00:44<00:00,  3.67it/s]
 99%|████████████████████████████████████████▍| 148/150 [00:44<00:00,  4.09it/s]
 99%|████████████████████████████████████████▋| 149/150 [00:44<00:00,  4.21it/s]
                                                                                
{'eval_loss': 0.5516486763954163, 'eval_runtime': 45.2801, 'eval_samples_per_second': 3.313, 'eval_steps_per_second': 3.313, 'epoch': 0.47, 'num_input_tokens_seen': 289392}
 16%|██████▍                                  | 100/639 [07:45<38:17,  4.26s/it]
100%|█████████████████████████████████████████| 150/150 [00:45<00:00,  3.98it/s]
                                                                                [INFO|trainer.py:3993] 2025-12-04 19:13:55,032 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-100
[INFO|configuration_utils.py:696] 2025-12-04 19:13:55,079 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:13:55,080 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:13:55,198 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-100/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:13:55,199 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-100/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:13:55,199 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-100/special_tokens_map.json
 16%|██████▍                                | 105/639 [08:06<1:05:46,  7.39s/it][INFO|2025-12-04 19:14:16] llamafactory.train.callbacks:143 >> {'loss': 0.5618, 'learning_rate': 9.3605e-05, 'epoch': 0.49, 'throughput': 623.96}
{'loss': 0.5618, 'grad_norm': 0.3641050159931183, 'learning_rate': 9.360526801044752e-05, 'epoch': 0.49, 'num_input_tokens_seen': 303552, 'train_runtime': 486.5013, 'train_tokens_per_second': 623.949}
 17%|███████                                  | 110/639 [08:25<39:32,  4.48s/it][INFO|2025-12-04 19:14:35] llamafactory.train.callbacks:143 >> {'loss': 0.5425, 'learning_rate': 9.2991e-05, 'epoch': 0.52, 'throughput': 625.38}
{'loss': 0.5425, 'grad_norm': 0.48988020420074463, 'learning_rate': 9.299073093021405e-05, 'epoch': 0.52, 'num_input_tokens_seen': 316176, 'train_runtime': 505.5794, 'train_tokens_per_second': 625.374}
 18%|███████▍                                 | 115/639 [08:46<37:11,  4.26s/it][INFO|2025-12-04 19:14:55] llamafactory.train.callbacks:143 >> {'loss': 0.5570, 'learning_rate': 9.2350e-05, 'epoch': 0.54, 'throughput': 627.53}
{'loss': 0.557, 'grad_norm': 0.3276370167732239, 'learning_rate': 9.235021673018849e-05, 'epoch': 0.54, 'num_input_tokens_seen': 330344, 'train_runtime': 526.4242, 'train_tokens_per_second': 627.524}
 19%|███████▋                                 | 120/639 [09:08<39:27,  4.56s/it][INFO|2025-12-04 19:15:17] llamafactory.train.callbacks:143 >> {'loss': 0.5373, 'learning_rate': 9.1684e-05, 'epoch': 0.56, 'throughput': 630.29}
{'loss': 0.5373, 'grad_norm': 0.2878979444503784, 'learning_rate': 9.168411244063863e-05, 'epoch': 0.56, 'num_input_tokens_seen': 345608, 'train_runtime': 548.3389, 'train_tokens_per_second': 630.282}
 20%|████████                                 | 125/639 [09:28<35:34,  4.15s/it][INFO|2025-12-04 19:15:38] llamafactory.train.callbacks:143 >> {'loss': 0.5990, 'learning_rate': 9.0993e-05, 'epoch': 0.59, 'throughput': 631.77}
{'loss': 0.599, 'grad_norm': 0.47103503346443176, 'learning_rate': 9.09928205546263e-05, 'epoch': 0.59, 'num_input_tokens_seen': 359168, 'train_runtime': 568.5138, 'train_tokens_per_second': 631.767}
 20%|████████▎                                | 130/639 [09:52<41:07,  4.85s/it][INFO|2025-12-04 19:16:02] llamafactory.train.callbacks:143 >> {'loss': 0.6046, 'learning_rate': 9.0277e-05, 'epoch': 0.61, 'throughput': 634.20}
{'loss': 0.6046, 'grad_norm': 0.37291550636291504, 'learning_rate': 9.027675878480131e-05, 'epoch': 0.61, 'num_input_tokens_seen': 375824, 'train_runtime': 592.6039, 'train_tokens_per_second': 634.191}
 21%|████████▋                                | 135/639 [10:13<35:24,  4.22s/it][INFO|2025-12-04 19:16:22] llamafactory.train.callbacks:143 >> {'loss': 0.5704, 'learning_rate': 8.9536e-05, 'epoch': 0.64, 'throughput': 635.79}
{'loss': 0.5704, 'grad_norm': 0.4764707386493683, 'learning_rate': 8.953635981099887e-05, 'epoch': 0.64, 'num_input_tokens_seen': 389832, 'train_runtime': 613.1557, 'train_tokens_per_second': 635.78}
 22%|████████▉                                | 140/639 [10:34<35:26,  4.26s/it][INFO|2025-12-04 19:16:43] llamafactory.train.callbacks:143 >> {'loss': 0.5996, 'learning_rate': 8.8772e-05, 'epoch': 0.66, 'throughput': 637.31}
{'loss': 0.5996, 'grad_norm': 0.41063907742500305, 'learning_rate': 8.877207101879302e-05, 'epoch': 0.66, 'num_input_tokens_seen': 404320, 'train_runtime': 634.4235, 'train_tokens_per_second': 637.303}
 23%|█████████▎                               | 145/639 [10:55<35:56,  4.37s/it][INFO|2025-12-04 19:17:05] llamafactory.train.callbacks:143 >> {'loss': 0.5650, 'learning_rate': 8.7984e-05, 'epoch': 0.68, 'throughput': 639.12}
{'loss': 0.565, 'grad_norm': 0.28647473454475403, 'learning_rate': 8.798435422916425e-05, 'epoch': 0.68, 'num_input_tokens_seen': 419128, 'train_runtime': 655.7903, 'train_tokens_per_second': 639.119}
 23%|█████████▌                               | 150/639 [11:17<35:15,  4.33s/it][INFO|2025-12-04 19:17:27] llamafactory.train.callbacks:143 >> {'loss': 0.5706, 'learning_rate': 8.7174e-05, 'epoch': 0.71, 'throughput': 640.55}
{'loss': 0.5706, 'grad_norm': 0.35413092374801636, 'learning_rate': 8.717368541944452e-05, 'epoch': 0.71, 'num_input_tokens_seen': 434248, 'train_runtime': 677.9333, 'train_tokens_per_second': 640.547}
 24%|█████████▉                               | 155/639 [11:39<34:42,  4.30s/it][INFO|2025-12-04 19:17:48] llamafactory.train.callbacks:143 >> {'loss': 0.5786, 'learning_rate': 8.6341e-05, 'epoch': 0.73, 'throughput': 642.12}
{'loss': 0.5786, 'grad_norm': 0.3721427023410797, 'learning_rate': 8.634055443570826e-05, 'epoch': 0.73, 'num_input_tokens_seen': 449104, 'train_runtime': 699.4179, 'train_tokens_per_second': 642.111}
 25%|██████████▎                              | 160/639 [12:00<33:18,  4.17s/it][INFO|2025-12-04 19:18:10] llamafactory.train.callbacks:143 >> {'loss': 0.5315, 'learning_rate': 8.5485e-05, 'epoch': 0.75, 'throughput': 643.01}
{'loss': 0.5315, 'grad_norm': 0.43377748131752014, 'learning_rate': 8.548546469678311e-05, 'epoch': 0.75, 'num_input_tokens_seen': 463328, 'train_runtime': 720.5617, 'train_tokens_per_second': 643.009}
 26%|██████████▌                              | 165/639 [12:20<31:17,  3.96s/it][INFO|2025-12-04 19:18:29] llamafactory.train.callbacks:143 >> {'loss': 0.5618, 'learning_rate': 8.4609e-05, 'epoch': 0.78, 'throughput': 644.04}
{'loss': 0.5618, 'grad_norm': 0.40435293316841125, 'learning_rate': 8.460893289005965e-05, 'epoch': 0.78, 'num_input_tokens_seen': 476752, 'train_runtime': 740.2637, 'train_tokens_per_second': 644.03}
 27%|██████████▉                              | 170/639 [12:41<31:33,  4.04s/it][INFO|2025-12-04 19:18:50] llamafactory.train.callbacks:143 >> {'loss': 0.6172, 'learning_rate': 8.3711e-05, 'epoch': 0.80, 'throughput': 644.94}
{'loss': 0.6172, 'grad_norm': 0.755048394203186, 'learning_rate': 8.371148865928319e-05, 'epoch': 0.8, 'num_input_tokens_seen': 490888, 'train_runtime': 761.1456, 'train_tokens_per_second': 644.933}
 27%|███████████▏                             | 175/639 [13:01<30:56,  4.00s/it][INFO|2025-12-04 19:19:11] llamafactory.train.callbacks:143 >> {'loss': 0.5638, 'learning_rate': 8.2794e-05, 'epoch': 0.82, 'throughput': 645.91}
{'loss': 0.5638, 'grad_norm': 0.42955467104911804, 'learning_rate': 8.279367428451702e-05, 'epoch': 0.82, 'num_input_tokens_seen': 504944, 'train_runtime': 781.7652, 'train_tokens_per_second': 645.902}
 28%|███████████▌                             | 180/639 [13:23<33:39,  4.40s/it][INFO|2025-12-04 19:19:32] llamafactory.train.callbacks:143 >> {'loss': 0.5859, 'learning_rate': 8.1856e-05, 'epoch': 0.85, 'throughput': 646.72}
{'loss': 0.5859, 'grad_norm': 0.35513269901275635, 'learning_rate': 8.185604435447002e-05, 'epoch': 0.85, 'num_input_tokens_seen': 519432, 'train_runtime': 803.182, 'train_tokens_per_second': 646.718}
 29%|███████████▊                             | 185/639 [13:43<31:32,  4.17s/it][INFO|2025-12-04 19:19:53] llamafactory.train.callbacks:143 >> {'loss': 0.5548, 'learning_rate': 8.0899e-05, 'epoch': 0.87, 'throughput': 647.38}
{'loss': 0.5548, 'grad_norm': 0.4681668281555176, 'learning_rate': 8.089916543138681e-05, 'epoch': 0.87, 'num_input_tokens_seen': 533328, 'train_runtime': 823.8372, 'train_tokens_per_second': 647.371}
 30%|████████████▏                            | 190/639 [14:04<31:26,  4.20s/it][INFO|2025-12-04 19:20:14] llamafactory.train.callbacks:143 >> {'loss': 0.5770, 'learning_rate': 7.9924e-05, 'epoch': 0.89, 'throughput': 648.45}
{'loss': 0.577, 'grad_norm': 0.4571211636066437, 'learning_rate': 7.992361570870288e-05, 'epoch': 0.89, 'num_input_tokens_seen': 547712, 'train_runtime': 844.6484, 'train_tokens_per_second': 648.45}
 31%|████████████▌                            | 195/639 [14:25<31:07,  4.21s/it][INFO|2025-12-04 19:20:35] llamafactory.train.callbacks:143 >> {'loss': 0.5777, 'learning_rate': 7.8930e-05, 'epoch': 0.92, 'throughput': 649.10}
{'loss': 0.5777, 'grad_norm': 0.34551718831062317, 'learning_rate': 7.892998466167165e-05, 'epoch': 0.92, 'num_input_tokens_seen': 562024, 'train_runtime': 865.8566, 'train_tokens_per_second': 649.096}
 31%|████████████▊                            | 200/639 [14:46<30:36,  4.18s/it][INFO|2025-12-04 19:20:56] llamafactory.train.callbacks:143 >> {'loss': 0.6595, 'learning_rate': 7.7919e-05, 'epoch': 0.94, 'throughput': 649.77}
{'loss': 0.6595, 'grad_norm': 0.4461055099964142, 'learning_rate': 7.791887269117442e-05, 'epoch': 0.94, 'num_input_tokens_seen': 576232, 'train_runtime': 886.8286, 'train_tokens_per_second': 649.767}
 31%|████████████▊                            | 200/639 [14:46<30:36,  4.18s/it][INFO|trainer.py:4327] 2025-12-04 19:20:56,369 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:20:56,369 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:20:56,369 >>   Batch size = 1

  0%|                                                   | 0/150 [00:00<?, ?it/s]
  1%|▌                                          | 2/150 [00:00<00:19,  7.63it/s]
  2%|▊                                          | 3/150 [00:00<00:27,  5.33it/s]
  3%|█▏                                         | 4/150 [00:00<00:37,  3.94it/s]
  3%|█▍                                         | 5/150 [00:01<00:38,  3.72it/s]
  4%|█▋                                         | 6/150 [00:01<00:39,  3.60it/s]
  5%|██                                         | 7/150 [00:01<00:40,  3.52it/s]
  5%|██▎                                        | 8/150 [00:02<00:41,  3.44it/s]
  6%|██▌                                        | 9/150 [00:02<00:39,  3.55it/s]
  7%|██▊                                       | 10/150 [00:02<00:40,  3.49it/s]
  7%|███                                       | 11/150 [00:02<00:40,  3.46it/s]
  8%|███▎                                      | 12/150 [00:03<00:40,  3.43it/s]
  9%|███▋                                      | 13/150 [00:03<00:42,  3.22it/s]
  9%|███▉                                      | 14/150 [00:03<00:43,  3.10it/s]
 10%|████▏                                     | 15/150 [00:04<00:42,  3.18it/s]
 11%|████▍                                     | 16/150 [00:04<00:45,  2.98it/s]
 11%|████▊                                     | 17/150 [00:04<00:43,  3.09it/s]
 12%|█████                                     | 18/150 [00:05<00:41,  3.15it/s]
 13%|█████▎                                    | 19/150 [00:05<00:40,  3.22it/s]
 13%|█████▌                                    | 20/150 [00:05<00:41,  3.10it/s]
 14%|█████▉                                    | 21/150 [00:06<00:37,  3.42it/s]
 15%|██████▏                                   | 22/150 [00:06<00:34,  3.72it/s]
 15%|██████▍                                   | 23/150 [00:06<00:37,  3.39it/s]
 16%|██████▋                                   | 24/150 [00:06<00:37,  3.39it/s]
 17%|███████                                   | 25/150 [00:07<00:37,  3.37it/s]
 17%|███████▎                                  | 26/150 [00:07<00:35,  3.49it/s]
 18%|███████▌                                  | 27/150 [00:07<00:35,  3.49it/s]
 19%|███████▊                                  | 28/150 [00:08<00:37,  3.28it/s]
 19%|████████                                  | 29/150 [00:08<00:38,  3.14it/s]
 20%|████████▍                                 | 30/150 [00:08<00:37,  3.23it/s]
 21%|████████▋                                 | 31/150 [00:09<00:35,  3.38it/s]
 21%|████████▉                                 | 32/150 [00:09<00:31,  3.72it/s]
 22%|█████████▏                                | 33/150 [00:09<00:35,  3.32it/s]
 23%|█████████▌                                | 34/150 [00:09<00:33,  3.45it/s]
 23%|█████████▊                                | 35/150 [00:10<00:35,  3.24it/s]
 24%|██████████                                | 36/150 [00:10<00:32,  3.55it/s]
 25%|██████████▎                               | 37/150 [00:10<00:35,  3.17it/s]
 25%|██████████▋                               | 38/150 [00:11<00:34,  3.22it/s]
 26%|██████████▉                               | 39/150 [00:11<00:35,  3.10it/s]
 27%|███████████▏                              | 40/150 [00:11<00:34,  3.18it/s]
 27%|███████████▍                              | 41/150 [00:12<00:33,  3.29it/s]
 28%|███████████▊                              | 42/150 [00:12<00:32,  3.36it/s]
 29%|████████████                              | 43/150 [00:12<00:31,  3.38it/s]
 29%|████████████▎                             | 44/150 [00:13<00:35,  2.96it/s]
 30%|████████████▌                             | 45/150 [00:13<00:33,  3.11it/s]
 31%|████████████▉                             | 46/150 [00:13<00:35,  2.91it/s]
 31%|█████████████▏                            | 47/150 [00:13<00:31,  3.30it/s]
 32%|█████████████▍                            | 48/150 [00:14<00:31,  3.28it/s]
 33%|█████████████▋                            | 49/150 [00:14<00:29,  3.39it/s]
 33%|██████████████                            | 50/150 [00:14<00:29,  3.42it/s]
 34%|██████████████▎                           | 51/150 [00:15<00:28,  3.46it/s]
 35%|██████████████▌                           | 52/150 [00:15<00:32,  3.04it/s]
 35%|██████████████▊                           | 53/150 [00:15<00:29,  3.25it/s]
 36%|███████████████                           | 54/150 [00:16<00:28,  3.34it/s]
 37%|███████████████▍                          | 55/150 [00:16<00:27,  3.44it/s]
 37%|███████████████▋                          | 56/150 [00:16<00:30,  3.13it/s]
 38%|███████████████▉                          | 57/150 [00:17<00:30,  3.05it/s]
 39%|████████████████▏                         | 58/150 [00:17<00:29,  3.15it/s]
 39%|████████████████▌                         | 59/150 [00:17<00:27,  3.29it/s]
 40%|████████████████▊                         | 60/150 [00:17<00:27,  3.31it/s]
 41%|█████████████████                         | 61/150 [00:18<00:26,  3.30it/s]
 41%|█████████████████▎                        | 62/150 [00:18<00:26,  3.31it/s]
 42%|█████████████████▋                        | 63/150 [00:18<00:26,  3.31it/s]
 43%|█████████████████▉                        | 64/150 [00:19<00:24,  3.58it/s]
 43%|██████████████████▏                       | 65/150 [00:19<00:24,  3.52it/s]
 44%|██████████████████▍                       | 66/150 [00:19<00:25,  3.26it/s]
 45%|██████████████████▊                       | 67/150 [00:20<00:28,  2.91it/s]
 45%|███████████████████                       | 68/150 [00:20<00:27,  3.03it/s]
 46%|███████████████████▎                      | 69/150 [00:20<00:24,  3.35it/s]
 47%|███████████████████▌                      | 70/150 [00:20<00:23,  3.46it/s]
 47%|███████████████████▉                      | 71/150 [00:21<00:21,  3.72it/s]
 48%|████████████████████▏                     | 72/150 [00:21<00:19,  4.00it/s]
 49%|████████████████████▍                     | 73/150 [00:21<00:20,  3.79it/s]
 49%|████████████████████▋                     | 74/150 [00:21<00:19,  3.97it/s]
 50%|█████████████████████                     | 75/150 [00:22<00:20,  3.75it/s]
 51%|█████████████████████▎                    | 76/150 [00:22<00:22,  3.28it/s]
 51%|█████████████████████▌                    | 77/150 [00:22<00:20,  3.61it/s]
 52%|█████████████████████▊                    | 78/150 [00:23<00:20,  3.59it/s]
 53%|██████████████████████                    | 79/150 [00:23<00:20,  3.53it/s]
 53%|██████████████████████▍                   | 80/150 [00:23<00:19,  3.58it/s]
 54%|██████████████████████▋                   | 81/150 [00:23<00:18,  3.83it/s]
 55%|██████████████████████▉                   | 82/150 [00:24<00:19,  3.48it/s]
 55%|███████████████████████▏                  | 83/150 [00:24<00:19,  3.49it/s]
 56%|███████████████████████▌                  | 84/150 [00:24<00:20,  3.27it/s]
 57%|███████████████████████▊                  | 85/150 [00:25<00:19,  3.35it/s]
 57%|████████████████████████                  | 86/150 [00:25<00:21,  3.01it/s]
 58%|████████████████████████▎                 | 87/150 [00:25<00:21,  2.93it/s]
 59%|████████████████████████▋                 | 88/150 [00:26<00:20,  3.05it/s]
 59%|████████████████████████▉                 | 89/150 [00:26<00:21,  2.78it/s]
 60%|█████████████████████████▏                | 90/150 [00:26<00:19,  3.01it/s]
 61%|█████████████████████████▍                | 91/150 [00:27<00:20,  2.94it/s]
 61%|█████████████████████████▊                | 92/150 [00:27<00:19,  2.91it/s]
 62%|██████████████████████████                | 93/150 [00:27<00:17,  3.25it/s]
 63%|██████████████████████████▎               | 94/150 [00:28<00:16,  3.35it/s]
 63%|██████████████████████████▌               | 95/150 [00:28<00:16,  3.34it/s]
 64%|██████████████████████████▉               | 96/150 [00:28<00:17,  3.15it/s]
 65%|███████████████████████████▏              | 97/150 [00:29<00:16,  3.21it/s]
 65%|███████████████████████████▍              | 98/150 [00:29<00:16,  3.24it/s]
 66%|███████████████████████████▋              | 99/150 [00:29<00:16,  3.12it/s]
 67%|███████████████████████████▎             | 100/150 [00:30<00:17,  2.88it/s]
 67%|███████████████████████████▌             | 101/150 [00:30<00:16,  3.06it/s]
 68%|███████████████████████████▉             | 102/150 [00:30<00:14,  3.40it/s]
 69%|████████████████████████████▏            | 103/150 [00:31<00:15,  3.09it/s]
 69%|████████████████████████████▍            | 104/150 [00:31<00:14,  3.16it/s]
 70%|████████████████████████████▋            | 105/150 [00:31<00:13,  3.34it/s]
 71%|████████████████████████████▉            | 106/150 [00:31<00:11,  3.77it/s]
 71%|█████████████████████████████▏           | 107/150 [00:32<00:11,  3.59it/s]
 72%|█████████████████████████████▌           | 108/150 [00:32<00:11,  3.58it/s]
 73%|█████████████████████████████▊           | 109/150 [00:32<00:12,  3.19it/s]
 73%|██████████████████████████████           | 110/150 [00:33<00:12,  3.22it/s]
 74%|██████████████████████████████▎          | 111/150 [00:33<00:11,  3.27it/s]
 75%|██████████████████████████████▌          | 112/150 [00:33<00:11,  3.31it/s]
 75%|██████████████████████████████▉          | 113/150 [00:34<00:11,  3.15it/s]
 76%|███████████████████████████████▏         | 114/150 [00:34<00:10,  3.32it/s]
 77%|███████████████████████████████▍         | 115/150 [00:34<00:09,  3.63it/s]
 77%|███████████████████████████████▋         | 116/150 [00:34<00:10,  3.35it/s]
 78%|███████████████████████████████▉         | 117/150 [00:35<00:10,  3.05it/s]
 79%|████████████████████████████████▎        | 118/150 [00:35<00:10,  3.18it/s]
 79%|████████████████████████████████▌        | 119/150 [00:35<00:09,  3.23it/s]
 80%|████████████████████████████████▊        | 120/150 [00:36<00:09,  3.08it/s]
 81%|█████████████████████████████████        | 121/150 [00:36<00:09,  3.20it/s]
 81%|█████████████████████████████████▎       | 122/150 [00:36<00:09,  3.04it/s]
 82%|█████████████████████████████████▌       | 123/150 [00:37<00:08,  3.22it/s]
 83%|█████████████████████████████████▉       | 124/150 [00:37<00:07,  3.37it/s]
 83%|██████████████████████████████████▏      | 125/150 [00:37<00:07,  3.46it/s]
 84%|██████████████████████████████████▍      | 126/150 [00:37<00:07,  3.43it/s]
 85%|██████████████████████████████████▋      | 127/150 [00:38<00:07,  3.20it/s]
 85%|██████████████████████████████████▉      | 128/150 [00:38<00:06,  3.35it/s]
 86%|███████████████████████████████████▎     | 129/150 [00:38<00:05,  3.63it/s]
 87%|███████████████████████████████████▌     | 130/150 [00:39<00:05,  3.54it/s]
 87%|███████████████████████████████████▊     | 131/150 [00:39<00:05,  3.31it/s]
 88%|████████████████████████████████████     | 132/150 [00:39<00:05,  3.16it/s]
 89%|████████████████████████████████████▎    | 133/150 [00:40<00:05,  3.31it/s]
 89%|████████████████████████████████████▋    | 134/150 [00:40<00:04,  3.37it/s]
 90%|████████████████████████████████████▉    | 135/150 [00:40<00:04,  3.43it/s]
 91%|█████████████████████████████████████▏   | 136/150 [00:40<00:04,  3.42it/s]
 91%|█████████████████████████████████████▍   | 137/150 [00:41<00:04,  2.99it/s]
 92%|█████████████████████████████████████▋   | 138/150 [00:41<00:03,  3.20it/s]
 93%|█████████████████████████████████████▉   | 139/150 [00:41<00:03,  3.05it/s]
 93%|██████████████████████████████████████▎  | 140/150 [00:42<00:03,  3.16it/s]
 94%|██████████████████████████████████████▌  | 141/150 [00:42<00:03,  2.96it/s]
 95%|██████████████████████████████████████▊  | 142/150 [00:42<00:02,  3.05it/s]
 95%|███████████████████████████████████████  | 143/150 [00:43<00:02,  3.15it/s]
 96%|███████████████████████████████████████▎ | 144/150 [00:43<00:01,  3.21it/s]
 97%|███████████████████████████████████████▋ | 145/150 [00:43<00:01,  3.28it/s]
 97%|███████████████████████████████████████▉ | 146/150 [00:44<00:01,  3.60it/s]
 98%|████████████████████████████████████████▏| 147/150 [00:44<00:00,  3.67it/s]
 99%|████████████████████████████████████████▍| 148/150 [00:44<00:00,  4.09it/s]
 99%|████████████████████████████████████████▋| 149/150 [00:44<00:00,  4.21it/s]
                                                                                
{'eval_loss': 0.5446581840515137, 'eval_runtime': 45.2784, 'eval_samples_per_second': 3.313, 'eval_steps_per_second': 3.313, 'epoch': 0.94, 'num_input_tokens_seen': 576232}
 31%|████████████▊                            | 200/639 [15:32<30:36,  4.18s/it]
100%|█████████████████████████████████████████| 150/150 [00:45<00:00,  3.98it/s]
                                                                                [INFO|trainer.py:3993] 2025-12-04 19:21:41,644 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-200
[INFO|configuration_utils.py:696] 2025-12-04 19:21:41,692 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:21:41,693 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:21:41,777 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-200/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:21:41,778 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-200/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:21:41,778 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-200/special_tokens_map.json
 32%|█████████████▏                           | 205/639 [15:53<53:19,  7.37s/it][INFO|2025-12-04 19:22:03] llamafactory.train.callbacks:143 >> {'loss': 0.5866, 'learning_rate': 7.6891e-05, 'epoch': 0.96, 'throughput': 619.48}
{'loss': 0.5866, 'grad_norm': 0.5498479604721069, 'learning_rate': 7.68908907609285e-05, 'epoch': 0.96, 'num_input_tokens_seen': 590944, 'train_runtime': 953.943, 'train_tokens_per_second': 619.475}
 33%|█████████████▍                           | 210/639 [16:15<34:05,  4.77s/it][INFO|2025-12-04 19:22:24] llamafactory.train.callbacks:143 >> {'loss': 0.5886, 'learning_rate': 7.5847e-05, 'epoch': 0.99, 'throughput': 620.87}
{'loss': 0.5886, 'grad_norm': 0.42110908031463623, 'learning_rate': 7.584666002831296e-05, 'epoch': 0.99, 'num_input_tokens_seen': 605408, 'train_runtime': 975.0987, 'train_tokens_per_second': 620.868}
 34%|█████████████▊                           | 215/639 [16:33<27:56,  3.95s/it][INFO|2025-12-04 19:22:42] llamafactory.train.callbacks:143 >> {'loss': 0.4728, 'learning_rate': 7.4787e-05, 'epoch': 1.01, 'throughput': 621.69}
{'loss': 0.4728, 'grad_norm': 0.49210885167121887, 'learning_rate': 7.478681146903448e-05, 'epoch': 1.01, 'num_input_tokens_seen': 617520, 'train_runtime': 993.292, 'train_tokens_per_second': 621.69}
 34%|██████████████                           | 220/639 [16:55<30:12,  4.33s/it][INFO|2025-12-04 19:23:05] llamafactory.train.callbacks:143 >> {'loss': 0.4468, 'learning_rate': 7.3712e-05, 'epoch': 1.03, 'throughput': 623.23}
{'loss': 0.4468, 'grad_norm': 0.36256247758865356, 'learning_rate': 7.371198549586091e-05, 'epoch': 1.03, 'num_input_tokens_seen': 633016, 'train_runtime': 1015.7033, 'train_tokens_per_second': 623.229}
 35%|██████████████▍                          | 225/639 [17:17<30:13,  4.38s/it][INFO|2025-12-04 19:23:27] llamafactory.train.callbacks:143 >> {'loss': 0.3993, 'learning_rate': 7.2623e-05, 'epoch': 1.06, 'throughput': 624.65}
{'loss': 0.3993, 'grad_norm': 0.43740350008010864, 'learning_rate': 7.262283157165219e-05, 'epoch': 1.06, 'num_input_tokens_seen': 648120, 'train_runtime': 1037.5816, 'train_tokens_per_second': 624.645}
 36%|██████████████▊                          | 230/639 [17:38<28:09,  4.13s/it][INFO|2025-12-04 19:23:47] llamafactory.train.callbacks:143 >> {'loss': 0.4697, 'learning_rate': 7.1520e-05, 'epoch': 1.08, 'throughput': 625.75}
{'loss': 0.4697, 'grad_norm': 0.402011513710022, 'learning_rate': 7.152000781692286e-05, 'epoch': 1.08, 'num_input_tokens_seen': 662304, 'train_runtime': 1058.4262, 'train_tokens_per_second': 625.744}
 37%|███████████████                          | 235/639 [18:00<30:19,  4.50s/it][INFO|2025-12-04 19:24:10] llamafactory.train.callbacks:143 >> {'loss': 0.4310, 'learning_rate': 7.0404e-05, 'epoch': 1.10, 'throughput': 627.18}
{'loss': 0.431, 'grad_norm': 0.4554682970046997, 'learning_rate': 7.040418061217325e-05, 'epoch': 1.1, 'num_input_tokens_seen': 677896, 'train_runtime': 1080.8739, 'train_tokens_per_second': 627.174}
 38%|███████████████▍                         | 240/639 [18:22<28:59,  4.36s/it][INFO|2025-12-04 19:24:31] llamafactory.train.callbacks:143 >> {'loss': 0.4295, 'learning_rate': 6.9276e-05, 'epoch': 1.13, 'throughput': 628.28}
{'loss': 0.4295, 'grad_norm': 0.4741499423980713, 'learning_rate': 6.927602419522947e-05, 'epoch': 1.13, 'num_input_tokens_seen': 692648, 'train_runtime': 1102.4599, 'train_tokens_per_second': 628.275}
 38%|███████████████▋                         | 245/639 [18:42<26:38,  4.06s/it][INFO|2025-12-04 19:24:52] llamafactory.train.callbacks:143 >> {'loss': 0.4233, 'learning_rate': 6.8136e-05, 'epoch': 1.15, 'throughput': 629.20}
{'loss': 0.4233, 'grad_norm': 0.4026746451854706, 'learning_rate': 6.813622025383565e-05, 'epoch': 1.15, 'num_input_tokens_seen': 706472, 'train_runtime': 1122.8107, 'train_tokens_per_second': 629.2}
 39%|████████████████                         | 250/639 [19:02<25:34,  3.95s/it][INFO|2025-12-04 19:25:12] llamafactory.train.callbacks:143 >> {'loss': 0.3867, 'learning_rate': 6.6985e-05, 'epoch': 1.17, 'throughput': 630.00}
{'loss': 0.3867, 'grad_norm': 0.48627060651779175, 'learning_rate': 6.698545751374465e-05, 'epoch': 1.17, 'num_input_tokens_seen': 719920, 'train_runtime': 1142.7429, 'train_tokens_per_second': 629.993}
 40%|████████████████▎                        | 255/639 [19:22<24:42,  3.86s/it][INFO|2025-12-04 19:25:31] llamafactory.train.callbacks:143 >> {'loss': 0.3709, 'learning_rate': 6.5824e-05, 'epoch': 1.20, 'throughput': 630.53}
{'loss': 0.3709, 'grad_norm': 0.7616415023803711, 'learning_rate': 6.582443132255592e-05, 'epoch': 1.2, 'num_input_tokens_seen': 732888, 'train_runtime': 1162.3342, 'train_tokens_per_second': 630.531}
 41%|████████████████▋                        | 260/639 [19:42<24:54,  3.94s/it][INFO|2025-12-04 19:25:52] llamafactory.train.callbacks:143 >> {'loss': 0.3777, 'learning_rate': 6.4654e-05, 'epoch': 1.22, 'throughput': 631.43}
{'loss': 0.3777, 'grad_norm': 0.6388319730758667, 'learning_rate': 6.465384322955224e-05, 'epoch': 1.22, 'num_input_tokens_seen': 746904, 'train_runtime': 1182.8759, 'train_tokens_per_second': 631.431}
 41%|█████████████████                        | 265/639 [20:04<26:48,  4.30s/it][INFO|2025-12-04 19:26:14] llamafactory.train.callbacks:143 >> {'loss': 0.3592, 'learning_rate': 6.3474e-05, 'epoch': 1.24, 'throughput': 632.61}
{'loss': 0.3592, 'grad_norm': 0.4034753441810608, 'learning_rate': 6.347440056178904e-05, 'epoch': 1.24, 'num_input_tokens_seen': 762040, 'train_runtime': 1204.6071, 'train_tokens_per_second': 632.605}
 42%|█████████████████▎                       | 270/639 [20:26<26:59,  4.39s/it][INFO|2025-12-04 19:26:35] llamafactory.train.callbacks:143 >> {'loss': 0.4180, 'learning_rate': 6.2287e-05, 'epoch': 1.27, 'throughput': 633.40}
{'loss': 0.418, 'grad_norm': 0.383634477853775, 'learning_rate': 6.228681599669248e-05, 'epoch': 1.27, 'num_input_tokens_seen': 776704, 'train_runtime': 1226.251, 'train_tokens_per_second': 633.397}
 43%|█████████████████▋                       | 275/639 [20:45<23:33,  3.88s/it][INFO|2025-12-04 19:26:55] llamafactory.train.callbacks:143 >> {'loss': 0.3786, 'learning_rate': 6.1092e-05, 'epoch': 1.29, 'throughput': 634.00}
{'loss': 0.3786, 'grad_norm': 0.5070011019706726, 'learning_rate': 6.109180713142465e-05, 'epoch': 1.29, 'num_input_tokens_seen': 789840, 'train_runtime': 1245.8012, 'train_tokens_per_second': 634.002}
 44%|█████████████████▉                       | 280/639 [21:06<24:55,  4.16s/it][INFO|2025-12-04 19:27:16] llamafactory.train.callbacks:143 >> {'loss': 0.3856, 'learning_rate': 5.9890e-05, 'epoch': 1.32, 'throughput': 635.08}
{'loss': 0.3856, 'grad_norm': 0.4170735478401184, 'learning_rate': 5.989009604927587e-05, 'epoch': 1.32, 'num_input_tokens_seen': 804600, 'train_runtime': 1266.9238, 'train_tokens_per_second': 635.082}
 45%|██████████████████▎                      | 285/639 [21:28<24:45,  4.20s/it][INFO|2025-12-04 19:27:37] llamafactory.train.callbacks:143 >> {'loss': 0.3766, 'learning_rate': 5.8682e-05, 'epoch': 1.34, 'throughput': 635.84}
{'loss': 0.3766, 'grad_norm': 0.5109259486198425, 'learning_rate': 5.868240888334653e-05, 'epoch': 1.34, 'num_input_tokens_seen': 818976, 'train_runtime': 1288.0334, 'train_tokens_per_second': 635.834}
 45%|██████████████████▌                      | 290/639 [21:50<26:38,  4.58s/it][INFO|2025-12-04 19:28:00] llamafactory.train.callbacks:143 >> {'loss': 0.4212, 'learning_rate': 5.7469e-05, 'epoch': 1.36, 'throughput': 636.65}
{'loss': 0.4212, 'grad_norm': 0.4675813913345337, 'learning_rate': 5.74694753777815e-05, 'epoch': 1.36, 'num_input_tokens_seen': 834520, 'train_runtime': 1310.8044, 'train_tokens_per_second': 636.647}
 46%|██████████████████▉                      | 295/639 [22:12<25:00,  4.36s/it][INFO|2025-12-04 19:28:22] llamafactory.train.callbacks:143 >> {'loss': 0.4031, 'learning_rate': 5.6252e-05, 'epoch': 1.39, 'throughput': 637.57}
{'loss': 0.4031, 'grad_norm': 0.4627811312675476, 'learning_rate': 5.62520284468228e-05, 'epoch': 1.39, 'num_input_tokens_seen': 849544, 'train_runtime': 1332.479, 'train_tokens_per_second': 637.567}
 47%|███████████████████▏                     | 300/639 [22:34<25:17,  4.48s/it][INFO|2025-12-04 19:28:44] llamafactory.train.callbacks:143 >> {'loss': 0.4076, 'learning_rate': 5.5031e-05, 'epoch': 1.41, 'throughput': 638.29}
{'loss': 0.4076, 'grad_norm': 0.4208119213581085, 'learning_rate': 5.5030803731946665e-05, 'epoch': 1.41, 'num_input_tokens_seen': 864576, 'train_runtime': 1354.5348, 'train_tokens_per_second': 638.283}
 47%|███████████████████▏                     | 300/639 [22:34<25:17,  4.48s/it][INFO|trainer.py:4327] 2025-12-04 19:28:44,075 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:28:44,075 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:28:44,075 >>   Batch size = 1

  0%|                                                   | 0/150 [00:00<?, ?it/s]
  1%|▌                                          | 2/150 [00:00<00:19,  7.59it/s]
  2%|▊                                          | 3/150 [00:00<00:27,  5.30it/s]
  3%|█▏                                         | 4/150 [00:00<00:36,  3.96it/s]
  3%|█▍                                         | 5/150 [00:01<00:38,  3.75it/s]
  4%|█▋                                         | 6/150 [00:01<00:39,  3.62it/s]
  5%|██                                         | 7/150 [00:01<00:40,  3.53it/s]
  5%|██▎                                        | 8/150 [00:02<00:41,  3.44it/s]
  6%|██▌                                        | 9/150 [00:02<00:39,  3.56it/s]
  7%|██▊                                       | 10/150 [00:02<00:40,  3.49it/s]
  7%|███                                       | 11/150 [00:02<00:40,  3.46it/s]
  8%|███▎                                      | 12/150 [00:03<00:40,  3.43it/s]
  9%|███▋                                      | 13/150 [00:03<00:42,  3.22it/s]
  9%|███▉                                      | 14/150 [00:03<00:43,  3.11it/s]
 10%|████▏                                     | 15/150 [00:04<00:42,  3.18it/s]
 11%|████▍                                     | 16/150 [00:04<00:45,  2.97it/s]
 11%|████▊                                     | 17/150 [00:04<00:43,  3.08it/s]
 12%|█████                                     | 18/150 [00:05<00:42,  3.14it/s]
 13%|█████▎                                    | 19/150 [00:05<00:40,  3.21it/s]
 13%|█████▌                                    | 20/150 [00:05<00:42,  3.09it/s]
 14%|█████▉                                    | 21/150 [00:06<00:37,  3.41it/s]
 15%|██████▏                                   | 22/150 [00:06<00:34,  3.71it/s]
 15%|██████▍                                   | 23/150 [00:06<00:37,  3.40it/s]
 16%|██████▋                                   | 24/150 [00:06<00:37,  3.39it/s]
 17%|███████                                   | 25/150 [00:07<00:37,  3.36it/s]
 17%|███████▎                                  | 26/150 [00:07<00:35,  3.48it/s]
 18%|███████▌                                  | 27/150 [00:07<00:35,  3.49it/s]
 19%|███████▊                                  | 28/150 [00:08<00:37,  3.28it/s]
 19%|████████                                  | 29/150 [00:08<00:38,  3.13it/s]
 20%|████████▍                                 | 30/150 [00:08<00:37,  3.24it/s]
 21%|████████▋                                 | 31/150 [00:09<00:35,  3.38it/s]
 21%|████████▉                                 | 32/150 [00:09<00:31,  3.72it/s]
 22%|█████████▏                                | 33/150 [00:09<00:35,  3.32it/s]
 23%|█████████▌                                | 34/150 [00:09<00:33,  3.45it/s]
 23%|█████████▊                                | 35/150 [00:10<00:35,  3.24it/s]
 24%|██████████                                | 36/150 [00:10<00:32,  3.54it/s]
 25%|██████████▎                               | 37/150 [00:10<00:35,  3.17it/s]
 25%|██████████▋                               | 38/150 [00:11<00:34,  3.23it/s]
 26%|██████████▉                               | 39/150 [00:11<00:35,  3.11it/s]
 27%|███████████▏                              | 40/150 [00:11<00:34,  3.18it/s]
 27%|███████████▍                              | 41/150 [00:12<00:33,  3.29it/s]
 28%|███████████▊                              | 42/150 [00:12<00:32,  3.35it/s]
 29%|████████████                              | 43/150 [00:12<00:31,  3.37it/s]
 29%|████████████▎                             | 44/150 [00:13<00:35,  2.96it/s]
 30%|████████████▌                             | 45/150 [00:13<00:33,  3.11it/s]
 31%|████████████▉                             | 46/150 [00:13<00:35,  2.91it/s]
 31%|█████████████▏                            | 47/150 [00:13<00:31,  3.30it/s]
 32%|█████████████▍                            | 48/150 [00:14<00:31,  3.28it/s]
 33%|█████████████▋                            | 49/150 [00:14<00:29,  3.39it/s]
 33%|██████████████                            | 50/150 [00:14<00:29,  3.41it/s]
 34%|██████████████▎                           | 51/150 [00:15<00:28,  3.47it/s]
 35%|██████████████▌                           | 52/150 [00:15<00:32,  3.05it/s]
 35%|██████████████▊                           | 53/150 [00:15<00:29,  3.25it/s]
 36%|███████████████                           | 54/150 [00:16<00:28,  3.35it/s]
 37%|███████████████▍                          | 55/150 [00:16<00:27,  3.46it/s]
 37%|███████████████▋                          | 56/150 [00:16<00:30,  3.13it/s]
 38%|███████████████▉                          | 57/150 [00:17<00:30,  3.05it/s]
 39%|████████████████▏                         | 58/150 [00:17<00:29,  3.16it/s]
 39%|████████████████▌                         | 59/150 [00:17<00:27,  3.29it/s]
 40%|████████████████▊                         | 60/150 [00:17<00:27,  3.31it/s]
 41%|█████████████████                         | 61/150 [00:18<00:26,  3.31it/s]
 41%|█████████████████▎                        | 62/150 [00:18<00:26,  3.31it/s]
 42%|█████████████████▋                        | 63/150 [00:18<00:26,  3.31it/s]
 43%|█████████████████▉                        | 64/150 [00:19<00:23,  3.59it/s]
 43%|██████████████████▏                       | 65/150 [00:19<00:24,  3.53it/s]
 44%|██████████████████▍                       | 66/150 [00:19<00:25,  3.26it/s]
 45%|██████████████████▊                       | 67/150 [00:20<00:28,  2.90it/s]
 45%|███████████████████                       | 68/150 [00:20<00:27,  3.03it/s]
 46%|███████████████████▎                      | 69/150 [00:20<00:24,  3.36it/s]
 47%|███████████████████▌                      | 70/150 [00:20<00:23,  3.47it/s]
 47%|███████████████████▉                      | 71/150 [00:21<00:21,  3.72it/s]
 48%|████████████████████▏                     | 72/150 [00:21<00:19,  4.00it/s]
 49%|████████████████████▍                     | 73/150 [00:21<00:20,  3.80it/s]
 49%|████████████████████▋                     | 74/150 [00:21<00:19,  3.98it/s]
 50%|█████████████████████                     | 75/150 [00:22<00:19,  3.75it/s]
 51%|█████████████████████▎                    | 76/150 [00:22<00:22,  3.30it/s]
 51%|█████████████████████▌                    | 77/150 [00:22<00:20,  3.63it/s]
 52%|█████████████████████▊                    | 78/150 [00:23<00:20,  3.60it/s]
 53%|██████████████████████                    | 79/150 [00:23<00:20,  3.53it/s]
 53%|██████████████████████▍                   | 80/150 [00:23<00:19,  3.59it/s]
 54%|██████████████████████▋                   | 81/150 [00:23<00:17,  3.84it/s]
 55%|██████████████████████▉                   | 82/150 [00:24<00:19,  3.48it/s]
 55%|███████████████████████▏                  | 83/150 [00:24<00:19,  3.49it/s]
 56%|███████████████████████▌                  | 84/150 [00:24<00:20,  3.28it/s]
 57%|███████████████████████▊                  | 85/150 [00:25<00:19,  3.36it/s]
 57%|████████████████████████                  | 86/150 [00:25<00:21,  3.01it/s]
 58%|████████████████████████▎                 | 87/150 [00:25<00:21,  2.93it/s]
 59%|████████████████████████▋                 | 88/150 [00:26<00:20,  3.04it/s]
 59%|████████████████████████▉                 | 89/150 [00:26<00:21,  2.78it/s]
 60%|█████████████████████████▏                | 90/150 [00:26<00:19,  3.01it/s]
 61%|█████████████████████████▍                | 91/150 [00:27<00:20,  2.93it/s]
 61%|█████████████████████████▊                | 92/150 [00:27<00:19,  2.91it/s]
 62%|██████████████████████████                | 93/150 [00:27<00:17,  3.25it/s]
 63%|██████████████████████████▎               | 94/150 [00:28<00:16,  3.35it/s]
 63%|██████████████████████████▌               | 95/150 [00:28<00:16,  3.34it/s]
 64%|██████████████████████████▉               | 96/150 [00:28<00:17,  3.15it/s]
 65%|███████████████████████████▏              | 97/150 [00:29<00:16,  3.21it/s]
 65%|███████████████████████████▍              | 98/150 [00:29<00:16,  3.24it/s]
 66%|███████████████████████████▋              | 99/150 [00:29<00:16,  3.13it/s]
 67%|███████████████████████████▎             | 100/150 [00:30<00:17,  2.89it/s]
 67%|███████████████████████████▌             | 101/150 [00:30<00:16,  3.06it/s]
 68%|███████████████████████████▉             | 102/150 [00:30<00:14,  3.39it/s]
 69%|████████████████████████████▏            | 103/150 [00:31<00:15,  3.09it/s]
 69%|████████████████████████████▍            | 104/150 [00:31<00:14,  3.16it/s]
 70%|████████████████████████████▋            | 105/150 [00:31<00:13,  3.33it/s]
 71%|████████████████████████████▉            | 106/150 [00:31<00:11,  3.78it/s]
 71%|█████████████████████████████▏           | 107/150 [00:32<00:11,  3.60it/s]
 72%|█████████████████████████████▌           | 108/150 [00:32<00:11,  3.58it/s]
 73%|█████████████████████████████▊           | 109/150 [00:32<00:12,  3.21it/s]
 73%|██████████████████████████████           | 110/150 [00:33<00:12,  3.25it/s]
 74%|██████████████████████████████▎          | 111/150 [00:33<00:11,  3.29it/s]
 75%|██████████████████████████████▌          | 112/150 [00:33<00:11,  3.33it/s]
 75%|██████████████████████████████▉          | 113/150 [00:34<00:11,  3.17it/s]
 76%|███████████████████████████████▏         | 114/150 [00:34<00:10,  3.34it/s]
 77%|███████████████████████████████▍         | 115/150 [00:34<00:09,  3.64it/s]
 77%|███████████████████████████████▋         | 116/150 [00:34<00:10,  3.36it/s]
 78%|███████████████████████████████▉         | 117/150 [00:35<00:10,  3.05it/s]
 79%|████████████████████████████████▎        | 118/150 [00:35<00:10,  3.18it/s]
 79%|████████████████████████████████▌        | 119/150 [00:35<00:09,  3.24it/s]
 80%|████████████████████████████████▊        | 120/150 [00:36<00:09,  3.09it/s]
 81%|█████████████████████████████████        | 121/150 [00:36<00:09,  3.20it/s]
 81%|█████████████████████████████████▎       | 122/150 [00:36<00:09,  3.05it/s]
 82%|█████████████████████████████████▌       | 123/150 [00:37<00:08,  3.23it/s]
 83%|█████████████████████████████████▉       | 124/150 [00:37<00:07,  3.36it/s]
 83%|██████████████████████████████████▏      | 125/150 [00:37<00:07,  3.47it/s]
 84%|██████████████████████████████████▍      | 126/150 [00:37<00:06,  3.43it/s]
 85%|██████████████████████████████████▋      | 127/150 [00:38<00:07,  3.20it/s]
 85%|██████████████████████████████████▉      | 128/150 [00:38<00:06,  3.35it/s]
 86%|███████████████████████████████████▎     | 129/150 [00:38<00:05,  3.62it/s]
 87%|███████████████████████████████████▌     | 130/150 [00:39<00:05,  3.54it/s]
 87%|███████████████████████████████████▊     | 131/150 [00:39<00:05,  3.31it/s]
 88%|████████████████████████████████████     | 132/150 [00:39<00:05,  3.16it/s]
 89%|████████████████████████████████████▎    | 133/150 [00:40<00:05,  3.31it/s]
 89%|████████████████████████████████████▋    | 134/150 [00:40<00:04,  3.37it/s]
 90%|████████████████████████████████████▉    | 135/150 [00:40<00:04,  3.43it/s]
 91%|█████████████████████████████████████▏   | 136/150 [00:40<00:04,  3.42it/s]
 91%|█████████████████████████████████████▍   | 137/150 [00:41<00:04,  2.99it/s]
 92%|█████████████████████████████████████▋   | 138/150 [00:41<00:03,  3.20it/s]
 93%|█████████████████████████████████████▉   | 139/150 [00:41<00:03,  3.05it/s]
 93%|██████████████████████████████████████▎  | 140/150 [00:42<00:03,  3.17it/s]
 94%|██████████████████████████████████████▌  | 141/150 [00:42<00:03,  2.96it/s]
 95%|██████████████████████████████████████▊  | 142/150 [00:42<00:02,  3.06it/s]
 95%|███████████████████████████████████████  | 143/150 [00:43<00:02,  3.14it/s]
 96%|███████████████████████████████████████▎ | 144/150 [00:43<00:01,  3.21it/s]
 97%|███████████████████████████████████████▋ | 145/150 [00:43<00:01,  3.28it/s]
 97%|███████████████████████████████████████▉ | 146/150 [00:44<00:01,  3.60it/s]
 98%|████████████████████████████████████████▏| 147/150 [00:44<00:00,  3.67it/s]
 99%|████████████████████████████████████████▍| 148/150 [00:44<00:00,  4.09it/s]
 99%|████████████████████████████████████████▋| 149/150 [00:44<00:00,  4.21it/s]
                                                                                
{'eval_loss': 0.5768356323242188, 'eval_runtime': 45.2451, 'eval_samples_per_second': 3.315, 'eval_steps_per_second': 3.315, 'epoch': 1.41, 'num_input_tokens_seen': 864576}
 47%|███████████████████▏                     | 300/639 [23:19<25:17,  4.48s/it]
100%|█████████████████████████████████████████| 150/150 [00:44<00:00,  3.97it/s]
                                                                                [INFO|trainer.py:3993] 2025-12-04 19:29:29,317 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-300
[INFO|configuration_utils.py:696] 2025-12-04 19:29:29,365 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:29:29,366 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:29:29,450 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-300/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:29:29,451 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-300/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:29:29,451 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-300/special_tokens_map.json
 48%|███████████████████▌                     | 305/639 [23:41<42:16,  7.59s/it][INFO|2025-12-04 19:29:50] llamafactory.train.callbacks:143 >> {'loss': 0.3732, 'learning_rate': 5.3807e-05, 'epoch': 1.43, 'throughput': 618.50}
{'loss': 0.3732, 'grad_norm': 0.5276778340339661, 'learning_rate': 5.380653915735272e-05, 'epoch': 1.43, 'num_input_tokens_seen': 879120, 'train_runtime': 1421.3896, 'train_tokens_per_second': 618.493}
 49%|███████████████████▉                     | 310/639 [24:01<26:25,  4.82s/it][INFO|2025-12-04 19:30:11] llamafactory.train.callbacks:143 >> {'loss': 0.3614, 'learning_rate': 5.2580e-05, 'epoch': 1.46, 'throughput': 619.33}
{'loss': 0.3614, 'grad_norm': 0.4933299124240875, 'learning_rate': 5.2579974484073655e-05, 'epoch': 1.46, 'num_input_tokens_seen': 892880, 'train_runtime': 1441.6816, 'train_tokens_per_second': 619.332}
 49%|████████████████████▏                    | 315/639 [24:22<23:23,  4.33s/it][INFO|2025-12-04 19:30:32] llamafactory.train.callbacks:143 >> {'loss': 0.3661, 'learning_rate': 5.1352e-05, 'epoch': 1.48, 'throughput': 620.15}
{'loss': 0.3661, 'grad_norm': 0.5306238532066345, 'learning_rate': 5.1351850862975315e-05, 'epoch': 1.48, 'num_input_tokens_seen': 906960, 'train_runtime': 1462.4961, 'train_tokens_per_second': 620.145}
 50%|████████████████████▌                    | 320/639 [24:41<20:42,  3.89s/it][INFO|2025-12-04 19:30:51] llamafactory.train.callbacks:143 >> {'loss': 0.3770, 'learning_rate': 5.0123e-05, 'epoch': 1.50, 'throughput': 620.90}
{'loss': 0.377, 'grad_norm': 0.5807981491088867, 'learning_rate': 5.0122910386916656e-05, 'epoch': 1.5, 'num_input_tokens_seen': 920144, 'train_runtime': 1481.9612, 'train_tokens_per_second': 620.896}
 51%|████████████████████▊                    | 325/639 [25:02<21:51,  4.18s/it][INFO|2025-12-04 19:31:12] llamafactory.train.callbacks:143 >> {'loss': 0.3885, 'learning_rate': 4.8894e-05, 'epoch': 1.53, 'throughput': 621.83}
{'loss': 0.3885, 'grad_norm': 0.6108599901199341, 'learning_rate': 4.889389564234066e-05, 'epoch': 1.53, 'num_input_tokens_seen': 934536, 'train_runtime': 1502.8902, 'train_tokens_per_second': 621.826}
 52%|█████████████████████▏                   | 330/639 [25:23<22:08,  4.30s/it][INFO|2025-12-04 19:31:33] llamafactory.train.callbacks:143 >> {'loss': 0.4047, 'learning_rate': 4.7666e-05, 'epoch': 1.55, 'throughput': 622.72}
{'loss': 0.4047, 'grad_norm': 0.4433024525642395, 'learning_rate': 4.766554926056707e-05, 'epoch': 1.55, 'num_input_tokens_seen': 948784, 'train_runtime': 1523.6097, 'train_tokens_per_second': 622.721}
 52%|█████████████████████▍                   | 335/639 [25:44<20:33,  4.06s/it][INFO|2025-12-04 19:31:53] llamafactory.train.callbacks:143 >> {'loss': 0.3847, 'learning_rate': 4.6439e-05, 'epoch': 1.57, 'throughput': 623.36}
{'loss': 0.3847, 'grad_norm': 0.4564903974533081, 'learning_rate': 4.643861346905781e-05, 'epoch': 1.57, 'num_input_tokens_seen': 962552, 'train_runtime': 1544.1407, 'train_tokens_per_second': 623.358}
 53%|█████████████████████▊                   | 340/639 [26:05<20:51,  4.18s/it][INFO|2025-12-04 19:32:14] llamafactory.train.callbacks:143 >> {'loss': 0.4056, 'learning_rate': 4.5214e-05, 'epoch': 1.60, 'throughput': 624.08}
{'loss': 0.4056, 'grad_norm': 0.5551087856292725, 'learning_rate': 4.521382964292663e-05, 'epoch': 1.6, 'num_input_tokens_seen': 976848, 'train_runtime': 1565.2702, 'train_tokens_per_second': 624.076}
 54%|██████████████████████▏                  | 345/639 [26:28<22:02,  4.50s/it][INFO|2025-12-04 19:32:38] llamafactory.train.callbacks:143 >> {'loss': 0.4301, 'learning_rate': 4.3992e-05, 'epoch': 1.62, 'throughput': 624.96}
{'loss': 0.4301, 'grad_norm': 0.40703701972961426, 'learning_rate': 4.399193785696366e-05, 'epoch': 1.62, 'num_input_tokens_seen': 992824, 'train_runtime': 1588.6224, 'train_tokens_per_second': 624.959}
 55%|██████████████████████▍                  | 350/639 [26:48<19:13,  3.99s/it][INFO|2025-12-04 19:32:57] llamafactory.train.callbacks:143 >> {'loss': 0.3942, 'learning_rate': 4.2774e-05, 'epoch': 1.64, 'throughput': 625.40}
{'loss': 0.3942, 'grad_norm': 0.6319884657859802, 'learning_rate': 4.277367643844574e-05, 'epoch': 1.64, 'num_input_tokens_seen': 1005672, 'train_runtime': 1608.055, 'train_tokens_per_second': 625.397}
 56%|██████████████████████▊                  | 355/639 [27:09<20:05,  4.24s/it][INFO|2025-12-04 19:33:18] llamafactory.train.callbacks:143 >> {'loss': 0.4638, 'learning_rate': 4.1560e-05, 'epoch': 1.67, 'throughput': 626.10}
{'loss': 0.4638, 'grad_norm': 0.5046573281288147, 'learning_rate': 4.1559781521002664e-05, 'epoch': 1.67, 'num_input_tokens_seen': 1020032, 'train_runtime': 1629.198, 'train_tokens_per_second': 626.095}
 56%|███████████████████████                  | 360/639 [27:29<18:43,  4.03s/it][INFO|2025-12-04 19:33:38] llamafactory.train.callbacks:143 >> {'loss': 0.3601, 'learning_rate': 4.0351e-05, 'epoch': 1.69, 'throughput': 626.74}
{'loss': 0.3601, 'grad_norm': 0.44419023394584656, 'learning_rate': 4.035098659980891e-05, 'epoch': 1.69, 'num_input_tokens_seen': 1033576, 'train_runtime': 1649.1295, 'train_tokens_per_second': 626.74}
 57%|███████████████████████▍                 | 365/639 [27:51<19:06,  4.18s/it][INFO|2025-12-04 19:34:00] llamafactory.train.callbacks:143 >> {'loss': 0.4354, 'learning_rate': 3.9148e-05, 'epoch': 1.72, 'throughput': 627.66}
{'loss': 0.4354, 'grad_norm': 0.5245075225830078, 'learning_rate': 3.914802208836973e-05, 'epoch': 1.72, 'num_input_tokens_seen': 1048952, 'train_runtime': 1671.2104, 'train_tokens_per_second': 627.66}
 58%|███████████████████████▋                 | 370/639 [28:12<19:29,  4.35s/it][INFO|2025-12-04 19:34:22] llamafactory.train.callbacks:143 >> {'loss': 0.3863, 'learning_rate': 3.7952e-05, 'epoch': 1.74, 'throughput': 628.56}
{'loss': 0.3863, 'grad_norm': 0.5564009547233582, 'learning_rate': 3.7951614877169284e-05, 'epoch': 1.74, 'num_input_tokens_seen': 1063992, 'train_runtime': 1692.7572, 'train_tokens_per_second': 628.556}
 59%|████████████████████████                 | 375/639 [28:35<20:16,  4.61s/it][INFO|2025-12-04 19:34:45] llamafactory.train.callbacks:143 >> {'loss': 0.3883, 'learning_rate': 3.6762e-05, 'epoch': 1.76, 'throughput': 629.44}
{'loss': 0.3883, 'grad_norm': 0.3564496338367462, 'learning_rate': 3.67624878944475e-05, 'epoch': 1.76, 'num_input_tokens_seen': 1080056, 'train_runtime': 1715.9085, 'train_tokens_per_second': 629.437}
 59%|████████████████████████▍                | 380/639 [28:57<19:05,  4.42s/it][INFO|2025-12-04 19:35:07] llamafactory.train.callbacks:143 >> {'loss': 0.4027, 'learning_rate': 3.5581e-05, 'epoch': 1.79, 'throughput': 630.13}
{'loss': 0.4027, 'grad_norm': 0.5695515871047974, 'learning_rate': 3.558135966937123e-05, 'epoch': 1.79, 'num_input_tokens_seen': 1095056, 'train_runtime': 1737.8423, 'train_tokens_per_second': 630.124}
 60%|████████████████████████▋                | 385/639 [29:19<18:08,  4.29s/it][INFO|2025-12-04 19:35:28] llamafactory.train.callbacks:143 >> {'loss': 0.3612, 'learning_rate': 3.4409e-05, 'epoch': 1.81, 'throughput': 630.77}
{'loss': 0.3612, 'grad_norm': 0.45643237233161926, 'learning_rate': 3.440894389786352e-05, 'epoch': 1.81, 'num_input_tokens_seen': 1109552, 'train_runtime': 1759.0607, 'train_tokens_per_second': 630.764}
 61%|█████████████████████████                | 390/639 [29:39<16:53,  4.07s/it][INFO|2025-12-04 19:35:49] llamafactory.train.callbacks:143 >> {'loss': 0.3676, 'learning_rate': 3.3246e-05, 'epoch': 1.83, 'throughput': 631.35}
{'loss': 0.3676, 'grad_norm': 0.6372836828231812, 'learning_rate': 3.3245949011353264e-05, 'epoch': 1.83, 'num_input_tokens_seen': 1123480, 'train_runtime': 1779.4816, 'train_tokens_per_second': 631.352}
 62%|█████████████████████████▎               | 395/639 [29:59<16:25,  4.04s/it][INFO|2025-12-04 19:36:09] llamafactory.train.callbacks:143 >> {'loss': 0.4137, 'learning_rate': 3.2093e-05, 'epoch': 1.86, 'throughput': 632.08}
{'loss': 0.4137, 'grad_norm': 0.5252777934074402, 'learning_rate': 3.209307774870603e-05, 'epoch': 1.86, 'num_input_tokens_seen': 1137664, 'train_runtime': 1799.8748, 'train_tokens_per_second': 632.08}
 63%|█████████████████████████▋               | 400/639 [30:20<16:37,  4.17s/it][INFO|2025-12-04 19:36:30] llamafactory.train.callbacks:143 >> {'loss': 0.3426, 'learning_rate': 3.0951e-05, 'epoch': 1.88, 'throughput': 632.60}
{'loss': 0.3426, 'grad_norm': 0.5461771488189697, 'learning_rate': 3.0951026731594635e-05, 'epoch': 1.88, 'num_input_tokens_seen': 1151792, 'train_runtime': 1820.7239, 'train_tokens_per_second': 632.601}
 63%|█████████████████████████▋               | 400/639 [30:20<16:37,  4.17s/it][INFO|trainer.py:4327] 2025-12-04 19:36:30,264 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:36:30,264 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:36:30,264 >>   Batch size = 1

  0%|                                                   | 0/150 [00:00<?, ?it/s]
  1%|▌                                          | 2/150 [00:00<00:19,  7.62it/s]
  2%|▊                                          | 3/150 [00:00<00:27,  5.30it/s]
  3%|█▏                                         | 4/150 [00:00<00:37,  3.93it/s]
  3%|█▍                                         | 5/150 [00:01<00:38,  3.74it/s]
  4%|█▋                                         | 6/150 [00:01<00:39,  3.61it/s]
  5%|██                                         | 7/150 [00:01<00:40,  3.51it/s]
  5%|██▎                                        | 8/150 [00:02<00:41,  3.43it/s]
  6%|██▌                                        | 9/150 [00:02<00:39,  3.55it/s]
  7%|██▊                                       | 10/150 [00:02<00:40,  3.49it/s]
  7%|███                                       | 11/150 [00:02<00:40,  3.46it/s]
  8%|███▎                                      | 12/150 [00:03<00:40,  3.43it/s]
  9%|███▋                                      | 13/150 [00:03<00:42,  3.22it/s]
  9%|███▉                                      | 14/150 [00:03<00:43,  3.11it/s]
 10%|████▏                                     | 15/150 [00:04<00:42,  3.18it/s]
 11%|████▍                                     | 16/150 [00:04<00:45,  2.98it/s]
 11%|████▊                                     | 17/150 [00:04<00:43,  3.08it/s]
 12%|█████                                     | 18/150 [00:05<00:42,  3.14it/s]
 13%|█████▎                                    | 19/150 [00:05<00:40,  3.20it/s]
 13%|█████▌                                    | 20/150 [00:05<00:42,  3.09it/s]
 14%|█████▉                                    | 21/150 [00:06<00:37,  3.41it/s]
 15%|██████▏                                   | 22/150 [00:06<00:34,  3.71it/s]
 15%|██████▍                                   | 23/150 [00:06<00:37,  3.40it/s]
 16%|██████▋                                   | 24/150 [00:06<00:37,  3.38it/s]
 17%|███████                                   | 25/150 [00:07<00:37,  3.36it/s]
 17%|███████▎                                  | 26/150 [00:07<00:35,  3.47it/s]
 18%|███████▌                                  | 27/150 [00:07<00:35,  3.49it/s]
 19%|███████▊                                  | 28/150 [00:08<00:37,  3.28it/s]
 19%|████████                                  | 29/150 [00:08<00:38,  3.13it/s]
 20%|████████▍                                 | 30/150 [00:08<00:37,  3.23it/s]
 21%|████████▋                                 | 31/150 [00:09<00:35,  3.37it/s]
 21%|████████▉                                 | 32/150 [00:09<00:31,  3.71it/s]
 22%|█████████▏                                | 33/150 [00:09<00:35,  3.32it/s]
 23%|█████████▌                                | 34/150 [00:09<00:33,  3.45it/s]
 23%|█████████▊                                | 35/150 [00:10<00:35,  3.24it/s]
 24%|██████████                                | 36/150 [00:10<00:32,  3.54it/s]
 25%|██████████▎                               | 37/150 [00:10<00:35,  3.17it/s]
 25%|██████████▋                               | 38/150 [00:11<00:34,  3.23it/s]
 26%|██████████▉                               | 39/150 [00:11<00:35,  3.11it/s]
 27%|███████████▏                              | 40/150 [00:11<00:34,  3.18it/s]
 27%|███████████▍                              | 41/150 [00:12<00:33,  3.28it/s]
 28%|███████████▊                              | 42/150 [00:12<00:32,  3.35it/s]
 29%|████████████                              | 43/150 [00:12<00:31,  3.37it/s]
 29%|████████████▎                             | 44/150 [00:13<00:35,  2.96it/s]
 30%|████████████▌                             | 45/150 [00:13<00:33,  3.11it/s]
 31%|████████████▉                             | 46/150 [00:13<00:35,  2.91it/s]
 31%|█████████████▏                            | 47/150 [00:14<00:31,  3.29it/s]
 32%|█████████████▍                            | 48/150 [00:14<00:31,  3.28it/s]
 33%|█████████████▋                            | 49/150 [00:14<00:29,  3.39it/s]
 33%|██████████████                            | 50/150 [00:14<00:29,  3.41it/s]
 34%|██████████████▎                           | 51/150 [00:15<00:28,  3.47it/s]
 35%|██████████████▌                           | 52/150 [00:15<00:32,  3.05it/s]
 35%|██████████████▊                           | 53/150 [00:15<00:29,  3.25it/s]
 36%|███████████████                           | 54/150 [00:16<00:28,  3.36it/s]
 37%|███████████████▍                          | 55/150 [00:16<00:27,  3.46it/s]
 37%|███████████████▋                          | 56/150 [00:16<00:30,  3.13it/s]
 38%|███████████████▉                          | 57/150 [00:17<00:30,  3.05it/s]
 39%|████████████████▏                         | 58/150 [00:17<00:29,  3.16it/s]
 39%|████████████████▌                         | 59/150 [00:17<00:27,  3.29it/s]
 40%|████████████████▊                         | 60/150 [00:17<00:27,  3.31it/s]
 41%|█████████████████                         | 61/150 [00:18<00:26,  3.31it/s]
 41%|█████████████████▎                        | 62/150 [00:18<00:26,  3.31it/s]
 42%|█████████████████▋                        | 63/150 [00:18<00:26,  3.31it/s]
 43%|█████████████████▉                        | 64/150 [00:19<00:23,  3.59it/s]
 43%|██████████████████▏                       | 65/150 [00:19<00:24,  3.53it/s]
 44%|██████████████████▍                       | 66/150 [00:19<00:25,  3.26it/s]
 45%|██████████████████▊                       | 67/150 [00:20<00:28,  2.90it/s]
 45%|███████████████████                       | 68/150 [00:20<00:27,  3.03it/s]
 46%|███████████████████▎                      | 69/150 [00:20<00:24,  3.36it/s]
 47%|███████████████████▌                      | 70/150 [00:20<00:23,  3.47it/s]
 47%|███████████████████▉                      | 71/150 [00:21<00:21,  3.73it/s]
 48%|████████████████████▏                     | 72/150 [00:21<00:19,  4.00it/s]
 49%|████████████████████▍                     | 73/150 [00:21<00:20,  3.80it/s]
 49%|████████████████████▋                     | 74/150 [00:21<00:19,  3.97it/s]
 50%|█████████████████████                     | 75/150 [00:22<00:19,  3.75it/s]
 51%|█████████████████████▎                    | 76/150 [00:22<00:22,  3.29it/s]
 51%|█████████████████████▌                    | 77/150 [00:22<00:20,  3.62it/s]
 52%|█████████████████████▊                    | 78/150 [00:23<00:20,  3.59it/s]
 53%|██████████████████████                    | 79/150 [00:23<00:20,  3.53it/s]
 53%|██████████████████████▍                   | 80/150 [00:23<00:19,  3.59it/s]
 54%|██████████████████████▋                   | 81/150 [00:23<00:18,  3.83it/s]
 55%|██████████████████████▉                   | 82/150 [00:24<00:19,  3.48it/s]
 55%|███████████████████████▏                  | 83/150 [00:24<00:19,  3.50it/s]
 56%|███████████████████████▌                  | 84/150 [00:24<00:20,  3.28it/s]
 57%|███████████████████████▊                  | 85/150 [00:25<00:19,  3.35it/s]
 57%|████████████████████████                  | 86/150 [00:25<00:21,  3.01it/s]
 58%|████████████████████████▎                 | 87/150 [00:25<00:21,  2.93it/s]
 59%|████████████████████████▋                 | 88/150 [00:26<00:20,  3.05it/s]
 59%|████████████████████████▉                 | 89/150 [00:26<00:21,  2.78it/s]
 60%|█████████████████████████▏                | 90/150 [00:26<00:19,  3.01it/s]
 61%|█████████████████████████▍                | 91/150 [00:27<00:20,  2.95it/s]
 61%|█████████████████████████▊                | 92/150 [00:27<00:19,  2.91it/s]
 62%|██████████████████████████                | 93/150 [00:27<00:17,  3.24it/s]
 63%|██████████████████████████▎               | 94/150 [00:28<00:16,  3.34it/s]
 63%|██████████████████████████▌               | 95/150 [00:28<00:16,  3.34it/s]
 64%|██████████████████████████▉               | 96/150 [00:28<00:17,  3.14it/s]
 65%|███████████████████████████▏              | 97/150 [00:29<00:16,  3.20it/s]
 65%|███████████████████████████▍              | 98/150 [00:29<00:16,  3.24it/s]
 66%|███████████████████████████▋              | 99/150 [00:29<00:16,  3.12it/s]
 67%|███████████████████████████▎             | 100/150 [00:30<00:17,  2.88it/s]
 67%|███████████████████████████▌             | 101/150 [00:30<00:16,  3.06it/s]
 68%|███████████████████████████▉             | 102/150 [00:30<00:14,  3.39it/s]
 69%|████████████████████████████▏            | 103/150 [00:31<00:15,  3.09it/s]
 69%|████████████████████████████▍            | 104/150 [00:31<00:14,  3.17it/s]
 70%|████████████████████████████▋            | 105/150 [00:31<00:13,  3.34it/s]
 71%|████████████████████████████▉            | 106/150 [00:31<00:11,  3.78it/s]
 71%|█████████████████████████████▏           | 107/150 [00:32<00:11,  3.60it/s]
 72%|█████████████████████████████▌           | 108/150 [00:32<00:11,  3.59it/s]
 73%|█████████████████████████████▊           | 109/150 [00:32<00:12,  3.21it/s]
 73%|██████████████████████████████           | 110/150 [00:33<00:12,  3.25it/s]
 74%|██████████████████████████████▎          | 111/150 [00:33<00:11,  3.29it/s]
 75%|██████████████████████████████▌          | 112/150 [00:33<00:11,  3.33it/s]
 75%|██████████████████████████████▉          | 113/150 [00:34<00:11,  3.17it/s]
 76%|███████████████████████████████▏         | 114/150 [00:34<00:10,  3.34it/s]
 77%|███████████████████████████████▍         | 115/150 [00:34<00:09,  3.64it/s]
 77%|███████████████████████████████▋         | 116/150 [00:34<00:10,  3.36it/s]
 78%|███████████████████████████████▉         | 117/150 [00:35<00:10,  3.07it/s]
 79%|████████████████████████████████▎        | 118/150 [00:35<00:09,  3.20it/s]
 79%|████████████████████████████████▌        | 119/150 [00:35<00:09,  3.25it/s]
 80%|████████████████████████████████▊        | 120/150 [00:36<00:09,  3.10it/s]
 81%|█████████████████████████████████        | 121/150 [00:36<00:09,  3.21it/s]
 81%|█████████████████████████████████▎       | 122/150 [00:36<00:09,  3.06it/s]
 82%|█████████████████████████████████▌       | 123/150 [00:37<00:08,  3.23it/s]
 83%|█████████████████████████████████▉       | 124/150 [00:37<00:07,  3.37it/s]
 83%|██████████████████████████████████▏      | 125/150 [00:37<00:07,  3.48it/s]
 84%|██████████████████████████████████▍      | 126/150 [00:37<00:06,  3.44it/s]
 85%|██████████████████████████████████▋      | 127/150 [00:38<00:07,  3.20it/s]
 85%|██████████████████████████████████▉      | 128/150 [00:38<00:06,  3.34it/s]
 86%|███████████████████████████████████▎     | 129/150 [00:38<00:05,  3.63it/s]
 87%|███████████████████████████████████▌     | 130/150 [00:39<00:05,  3.55it/s]
 87%|███████████████████████████████████▊     | 131/150 [00:39<00:05,  3.32it/s]
 88%|████████████████████████████████████     | 132/150 [00:39<00:05,  3.17it/s]
 89%|████████████████████████████████████▎    | 133/150 [00:40<00:05,  3.31it/s]
 89%|████████████████████████████████████▋    | 134/150 [00:40<00:04,  3.38it/s]
 90%|████████████████████████████████████▉    | 135/150 [00:40<00:04,  3.43it/s]
 91%|█████████████████████████████████████▏   | 136/150 [00:40<00:04,  3.41it/s]
 91%|█████████████████████████████████████▍   | 137/150 [00:41<00:04,  2.99it/s]
 92%|█████████████████████████████████████▋   | 138/150 [00:41<00:03,  3.20it/s]
 93%|█████████████████████████████████████▉   | 139/150 [00:41<00:03,  3.06it/s]
 93%|██████████████████████████████████████▎  | 140/150 [00:42<00:03,  3.18it/s]
 94%|██████████████████████████████████████▌  | 141/150 [00:42<00:03,  2.96it/s]
 95%|██████████████████████████████████████▊  | 142/150 [00:42<00:02,  3.06it/s]
 95%|███████████████████████████████████████  | 143/150 [00:43<00:02,  3.15it/s]
 96%|███████████████████████████████████████▎ | 144/150 [00:43<00:01,  3.21it/s]
 97%|███████████████████████████████████████▋ | 145/150 [00:43<00:01,  3.28it/s]
 97%|███████████████████████████████████████▉ | 146/150 [00:44<00:01,  3.61it/s]
 98%|████████████████████████████████████████▏| 147/150 [00:44<00:00,  3.67it/s]
 99%|████████████████████████████████████████▍| 148/150 [00:44<00:00,  4.08it/s]
 99%|████████████████████████████████████████▋| 149/150 [00:44<00:00,  4.21it/s]
                                                                                
{'eval_loss': 0.582048237323761, 'eval_runtime': 45.2294, 'eval_samples_per_second': 3.316, 'eval_steps_per_second': 3.316, 'epoch': 1.88, 'num_input_tokens_seen': 1151792}
 63%|█████████████████████████▋               | 400/639 [31:05<16:37,  4.17s/it]
100%|█████████████████████████████████████████| 150/150 [00:44<00:00,  3.98it/s]
                                                                                [INFO|trainer.py:3993] 2025-12-04 19:37:15,490 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-400
[INFO|configuration_utils.py:696] 2025-12-04 19:37:15,538 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:37:15,538 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:37:15,623 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-400/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:37:15,623 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-400/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:37:15,623 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-400/special_tokens_map.json
 63%|█████████████████████████▉               | 405/639 [31:26<28:30,  7.31s/it][INFO|2025-12-04 19:37:35] llamafactory.train.callbacks:143 >> {'loss': 0.3577, 'learning_rate': 2.9820e-05, 'epoch': 1.90, 'throughput': 617.66}
{'loss': 0.3577, 'grad_norm': 0.7871028184890747, 'learning_rate': 2.9820486043565854e-05, 'epoch': 1.9, 'num_input_tokens_seen': 1165048, 'train_runtime': 1886.2362, 'train_tokens_per_second': 617.658}
 64%|██████████████████████████▎              | 410/639 [31:47<17:55,  4.70s/it][INFO|2025-12-04 19:37:56] llamafactory.train.callbacks:143 >> {'loss': 0.3717, 'learning_rate': 2.8702e-05, 'epoch': 1.93, 'throughput': 618.36}
{'loss': 0.3717, 'grad_norm': 0.5074209570884705, 'learning_rate': 2.870213881305802e-05, 'epoch': 1.93, 'num_input_tokens_seen': 1179328, 'train_runtime': 1907.2045, 'train_tokens_per_second': 618.354}
 65%|██████████████████████████▋              | 415/639 [32:08<16:37,  4.45s/it][INFO|2025-12-04 19:38:17] llamafactory.train.callbacks:143 >> {'loss': 0.4490, 'learning_rate': 2.7597e-05, 'epoch': 1.95, 'throughput': 618.92}
{'loss': 0.449, 'grad_norm': 0.47436830401420593, 'learning_rate': 2.7596660800621078e-05, 'epoch': 1.95, 'num_input_tokens_seen': 1193472, 'train_runtime': 1928.3147, 'train_tokens_per_second': 618.92}
 66%|██████████████████████████▉              | 420/639 [32:28<15:05,  4.13s/it][INFO|2025-12-04 19:38:37] llamafactory.train.callbacks:143 >> {'loss': 0.3816, 'learning_rate': 2.6505e-05, 'epoch': 1.97, 'throughput': 619.43}
{'loss': 0.3816, 'grad_norm': 0.5366332530975342, 'learning_rate': 2.650471999058875e-05, 'epoch': 1.97, 'num_input_tokens_seen': 1206920, 'train_runtime': 1948.436, 'train_tokens_per_second': 619.43}
 67%|███████████████████████████▎             | 425/639 [32:50<14:34,  4.09s/it][INFO|2025-12-04 19:38:59] llamafactory.train.callbacks:143 >> {'loss': 0.3983, 'learning_rate': 2.5427e-05, 'epoch': 2.00, 'throughput': 620.33}
{'loss': 0.3983, 'grad_norm': 0.5117286443710327, 'learning_rate': 2.542697618744945e-05, 'epoch': 2.0, 'num_input_tokens_seen': 1222080, 'train_runtime': 1970.0385, 'train_tokens_per_second': 620.333}
 67%|███████████████████████████▌             | 430/639 [33:07<13:01,  3.74s/it][INFO|2025-12-04 19:39:17] llamafactory.train.callbacks:143 >> {'loss': 0.2464, 'learning_rate': 2.4364e-05, 'epoch': 2.02, 'throughput': 620.99}
{'loss': 0.2464, 'grad_norm': 0.8380260467529297, 'learning_rate': 2.4364080617159886e-05, 'epoch': 2.02, 'num_input_tokens_seen': 1234488, 'train_runtime': 1987.9502, 'train_tokens_per_second': 620.985}
 68%|███████████████████████████▉             | 435/639 [33:28<13:56,  4.10s/it][INFO|2025-12-04 19:39:38] llamafactory.train.callbacks:143 >> {'loss': 0.2237, 'learning_rate': 2.3317e-05, 'epoch': 2.04, 'throughput': 621.53}
{'loss': 0.2237, 'grad_norm': 0.4559262990951538, 'learning_rate': 2.3316675533642214e-05, 'epoch': 2.04, 'num_input_tokens_seen': 1248328, 'train_runtime': 2008.4729, 'train_tokens_per_second': 621.531}
 69%|████████████████████████████▏            | 440/639 [33:50<14:28,  4.36s/it][INFO|2025-12-04 19:40:00] llamafactory.train.callbacks:143 >> {'loss': 0.2394, 'learning_rate': 2.2285e-05, 'epoch': 2.07, 'throughput': 622.10}
{'loss': 0.2394, 'grad_norm': 0.5217824578285217, 'learning_rate': 2.22853938307025e-05, 'epoch': 2.07, 'num_input_tokens_seen': 1263224, 'train_runtime': 2030.5946, 'train_tokens_per_second': 622.096}
 70%|████████████████████████████▌            | 445/639 [34:09<12:38,  3.91s/it][INFO|2025-12-04 19:40:18] llamafactory.train.callbacks:143 >> {'loss': 0.2701, 'learning_rate': 2.1271e-05, 'epoch': 2.09, 'throughput': 622.51}
{'loss': 0.2701, 'grad_norm': 0.9445173144340515, 'learning_rate': 2.1270858659605158e-05, 'epoch': 2.09, 'num_input_tokens_seen': 1275792, 'train_runtime': 2049.4348, 'train_tokens_per_second': 622.509}
 70%|████████████████████████████▊            | 450/639 [34:28<12:30,  3.97s/it][INFO|2025-12-04 19:40:38] llamafactory.train.callbacks:143 >> {'loss': 0.2105, 'learning_rate': 2.0274e-05, 'epoch': 2.11, 'throughput': 622.88}
{'loss': 0.2105, 'grad_norm': 0.5436469912528992, 'learning_rate': 2.0273683052534175e-05, 'epoch': 2.11, 'num_input_tokens_seen': 1288592, 'train_runtime': 2068.766, 'train_tokens_per_second': 622.88}
 71%|█████████████████████████████▏           | 455/639 [34:50<12:50,  4.19s/it][INFO|2025-12-04 19:40:59] llamafactory.train.callbacks:143 >> {'loss': 0.2344, 'learning_rate': 1.9294e-05, 'epoch': 2.14, 'throughput': 623.59}
{'loss': 0.2344, 'grad_norm': 0.6321210265159607, 'learning_rate': 1.9294469552168813e-05, 'epoch': 2.14, 'num_input_tokens_seen': 1303376, 'train_runtime': 2090.1165, 'train_tokens_per_second': 623.59}
 72%|█████████████████████████████▌           | 460/639 [35:11<12:15,  4.11s/it][INFO|2025-12-04 19:41:20] llamafactory.train.callbacks:143 >> {'loss': 0.2572, 'learning_rate': 1.8334e-05, 'epoch': 2.16, 'throughput': 624.18}
{'loss': 0.2572, 'grad_norm': 0.6212900876998901, 'learning_rate': 1.8333809847597642e-05, 'epoch': 2.16, 'num_input_tokens_seen': 1317856, 'train_runtime': 2111.3592, 'train_tokens_per_second': 624.174}
 73%|█████████████████████████████▊           | 465/639 [35:32<12:09,  4.19s/it][INFO|2025-12-04 19:41:41] llamafactory.train.callbacks:143 >> {'loss': 0.2222, 'learning_rate': 1.7392e-05, 'epoch': 2.18, 'throughput': 624.64}
{'loss': 0.2222, 'grad_norm': 0.8060994148254395, 'learning_rate': 1.739228441679081e-05, 'epoch': 2.18, 'num_input_tokens_seen': 1331792, 'train_runtime': 2132.1068, 'train_tokens_per_second': 624.637}
 74%|██████████████████████████████▏          | 470/639 [35:53<12:13,  4.34s/it][INFO|2025-12-04 19:42:03] llamafactory.train.callbacks:143 >> {'loss': 0.2205, 'learning_rate': 1.6470e-05, 'epoch': 2.21, 'throughput': 625.32}
{'loss': 0.2205, 'grad_norm': 0.4554629921913147, 'learning_rate': 1.647046217584661e-05, 'epoch': 2.21, 'num_input_tokens_seen': 1346640, 'train_runtime': 2153.5175, 'train_tokens_per_second': 625.321}
 74%|██████████████████████████████▍          | 475/639 [36:13<11:07,  4.07s/it][INFO|2025-12-04 19:42:23] llamafactory.train.callbacks:143 >> {'loss': 0.2254, 'learning_rate': 1.5569e-05, 'epoch': 2.23, 'throughput': 625.93}
{'loss': 0.2254, 'grad_norm': 0.6985599398612976, 'learning_rate': 1.556890013522428e-05, 'epoch': 2.23, 'num_input_tokens_seen': 1360720, 'train_runtime': 2173.9366, 'train_tokens_per_second': 625.924}
 75%|██████████████████████████████▊          | 480/639 [36:36<11:25,  4.31s/it][INFO|2025-12-04 19:42:45] llamafactory.train.callbacks:143 >> {'loss': 0.2295, 'learning_rate': 1.4688e-05, 'epoch': 2.25, 'throughput': 626.58}
{'loss': 0.2295, 'grad_norm': 0.6668505072593689, 'learning_rate': 1.4688143063170923e-05, 'epoch': 2.25, 'num_input_tokens_seen': 1375984, 'train_runtime': 2196.0303, 'train_tokens_per_second': 626.578}
 76%|███████████████████████████████          | 485/639 [36:57<10:36,  4.13s/it][INFO|2025-12-04 19:43:06] llamafactory.train.callbacks:143 >> {'loss': 0.2024, 'learning_rate': 1.3829e-05, 'epoch': 2.28, 'throughput': 627.21}
{'loss': 0.2024, 'grad_norm': 0.5283108949661255, 'learning_rate': 1.3828723156545553e-05, 'epoch': 2.28, 'num_input_tokens_seen': 1390624, 'train_runtime': 2217.1625, 'train_tokens_per_second': 627.209}
 77%|███████████████████████████████▍         | 490/639 [37:20<11:41,  4.71s/it][INFO|2025-12-04 19:43:30] llamafactory.train.callbacks:143 >> {'loss': 0.2335, 'learning_rate': 1.2991e-05, 'epoch': 2.30, 'throughput': 627.73}
{'loss': 0.2335, 'grad_norm': 0.6252283453941345, 'learning_rate': 1.2991159719239582e-05, 'epoch': 2.3, 'num_input_tokens_seen': 1406672, 'train_runtime': 2240.8972, 'train_tokens_per_second': 627.727}
 77%|███████████████████████████████▊         | 495/639 [37:42<10:32,  4.39s/it][INFO|2025-12-04 19:43:52] llamafactory.train.callbacks:143 >> {'loss': 0.2429, 'learning_rate': 1.2176e-05, 'epoch': 2.32, 'throughput': 628.42}
{'loss': 0.2429, 'grad_norm': 0.7144733667373657, 'learning_rate': 1.2175958848387765e-05, 'epoch': 2.32, 'num_input_tokens_seen': 1421880, 'train_runtime': 2262.6184, 'train_tokens_per_second': 628.422}
 78%|████████████████████████████████         | 500/639 [38:04<10:01,  4.33s/it][INFO|2025-12-04 19:44:13] llamafactory.train.callbacks:143 >> {'loss': 0.2311, 'learning_rate': 1.1384e-05, 'epoch': 2.35, 'throughput': 629.07}
{'loss': 0.2311, 'grad_norm': 0.7061449289321899, 'learning_rate': 1.1383613128559306e-05, 'epoch': 2.35, 'num_input_tokens_seen': 1437056, 'train_runtime': 2284.4098, 'train_tokens_per_second': 629.071}
 78%|████████████████████████████████         | 500/639 [38:04<10:01,  4.33s/it][INFO|trainer.py:4327] 2025-12-04 19:44:13,950 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:44:13,950 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:44:13,950 >>   Batch size = 1

  0%|                                                   | 0/150 [00:00<?, ?it/s]
  1%|▌                                          | 2/150 [00:00<00:19,  7.63it/s]
  2%|▊                                          | 3/150 [00:00<00:27,  5.33it/s]
  3%|█▏                                         | 4/150 [00:00<00:36,  3.95it/s]
  3%|█▍                                         | 5/150 [00:01<00:38,  3.74it/s]
  4%|█▋                                         | 6/150 [00:01<00:39,  3.61it/s]
  5%|██                                         | 7/150 [00:01<00:40,  3.52it/s]
  5%|██▎                                        | 8/150 [00:02<00:41,  3.44it/s]
  6%|██▌                                        | 9/150 [00:02<00:39,  3.56it/s]
  7%|██▊                                       | 10/150 [00:02<00:40,  3.50it/s]
  7%|███                                       | 11/150 [00:02<00:40,  3.47it/s]
  8%|███▎                                      | 12/150 [00:03<00:40,  3.43it/s]
  9%|███▋                                      | 13/150 [00:03<00:42,  3.22it/s]
  9%|███▉                                      | 14/150 [00:03<00:43,  3.11it/s]
 10%|████▏                                     | 15/150 [00:04<00:42,  3.18it/s]
 11%|████▍                                     | 16/150 [00:04<00:45,  2.97it/s]
 11%|████▊                                     | 17/150 [00:04<00:43,  3.08it/s]
 12%|█████                                     | 18/150 [00:05<00:41,  3.15it/s]
 13%|█████▎                                    | 19/150 [00:05<00:40,  3.22it/s]
 13%|█████▌                                    | 20/150 [00:05<00:41,  3.10it/s]
 14%|█████▉                                    | 21/150 [00:06<00:37,  3.42it/s]
 15%|██████▏                                   | 22/150 [00:06<00:34,  3.71it/s]
 15%|██████▍                                   | 23/150 [00:06<00:37,  3.40it/s]
 16%|██████▋                                   | 24/150 [00:06<00:37,  3.38it/s]
 17%|███████                                   | 25/150 [00:07<00:37,  3.37it/s]
 17%|███████▎                                  | 26/150 [00:07<00:35,  3.48it/s]
 18%|███████▌                                  | 27/150 [00:07<00:35,  3.49it/s]
 19%|███████▊                                  | 28/150 [00:08<00:37,  3.28it/s]
 19%|████████                                  | 29/150 [00:08<00:38,  3.13it/s]
 20%|████████▍                                 | 30/150 [00:08<00:37,  3.23it/s]
 21%|████████▋                                 | 31/150 [00:09<00:35,  3.38it/s]
 21%|████████▉                                 | 32/150 [00:09<00:31,  3.71it/s]
 22%|█████████▏                                | 33/150 [00:09<00:35,  3.32it/s]
 23%|█████████▌                                | 34/150 [00:09<00:33,  3.45it/s]
 23%|█████████▊                                | 35/150 [00:10<00:35,  3.25it/s]
 24%|██████████                                | 36/150 [00:10<00:32,  3.55it/s]
 25%|██████████▎                               | 37/150 [00:10<00:35,  3.17it/s]
 25%|██████████▋                               | 38/150 [00:11<00:34,  3.22it/s]
 26%|██████████▉                               | 39/150 [00:11<00:35,  3.11it/s]
 27%|███████████▏                              | 40/150 [00:11<00:34,  3.17it/s]
 27%|███████████▍                              | 41/150 [00:12<00:33,  3.28it/s]
 28%|███████████▊                              | 42/150 [00:12<00:32,  3.35it/s]
 29%|████████████                              | 43/150 [00:12<00:31,  3.37it/s]
 29%|████████████▎                             | 44/150 [00:13<00:35,  2.96it/s]
 30%|████████████▌                             | 45/150 [00:13<00:33,  3.11it/s]
 31%|████████████▉                             | 46/150 [00:13<00:35,  2.91it/s]
 31%|█████████████▏                            | 47/150 [00:13<00:31,  3.29it/s]
 32%|█████████████▍                            | 48/150 [00:14<00:31,  3.29it/s]
 33%|█████████████▋                            | 49/150 [00:14<00:29,  3.39it/s]
 33%|██████████████                            | 50/150 [00:14<00:29,  3.42it/s]
 34%|██████████████▎                           | 51/150 [00:15<00:28,  3.47it/s]
 35%|██████████████▌                           | 52/150 [00:15<00:32,  3.04it/s]
 35%|██████████████▊                           | 53/150 [00:15<00:29,  3.25it/s]
 36%|███████████████                           | 54/150 [00:16<00:28,  3.35it/s]
 37%|███████████████▍                          | 55/150 [00:16<00:27,  3.46it/s]
 37%|███████████████▋                          | 56/150 [00:16<00:30,  3.12it/s]
 38%|███████████████▉                          | 57/150 [00:17<00:30,  3.05it/s]
 39%|████████████████▏                         | 58/150 [00:17<00:29,  3.15it/s]
 39%|████████████████▌                         | 59/150 [00:17<00:27,  3.29it/s]
 40%|████████████████▊                         | 60/150 [00:17<00:27,  3.31it/s]
 41%|█████████████████                         | 61/150 [00:18<00:26,  3.31it/s]
 41%|█████████████████▎                        | 62/150 [00:18<00:26,  3.31it/s]
 42%|█████████████████▋                        | 63/150 [00:18<00:26,  3.31it/s]
 43%|█████████████████▉                        | 64/150 [00:19<00:24,  3.58it/s]
 43%|██████████████████▏                       | 65/150 [00:19<00:24,  3.52it/s]
 44%|██████████████████▍                       | 66/150 [00:19<00:25,  3.26it/s]
 45%|██████████████████▊                       | 67/150 [00:20<00:28,  2.90it/s]
 45%|███████████████████                       | 68/150 [00:20<00:27,  3.03it/s]
 46%|███████████████████▎                      | 69/150 [00:20<00:24,  3.35it/s]
 47%|███████████████████▌                      | 70/150 [00:20<00:23,  3.45it/s]
 47%|███████████████████▉                      | 71/150 [00:21<00:21,  3.72it/s]
 48%|████████████████████▏                     | 72/150 [00:21<00:19,  4.02it/s]
 49%|████████████████████▍                     | 73/150 [00:21<00:20,  3.80it/s]
 49%|████████████████████▋                     | 74/150 [00:21<00:19,  3.98it/s]
 50%|█████████████████████                     | 75/150 [00:22<00:19,  3.76it/s]
 51%|█████████████████████▎                    | 76/150 [00:22<00:22,  3.30it/s]
 51%|█████████████████████▌                    | 77/150 [00:22<00:20,  3.62it/s]
 52%|█████████████████████▊                    | 78/150 [00:23<00:20,  3.58it/s]
 53%|██████████████████████                    | 79/150 [00:23<00:20,  3.52it/s]
 53%|██████████████████████▍                   | 80/150 [00:23<00:19,  3.57it/s]
 54%|██████████████████████▋                   | 81/150 [00:23<00:18,  3.82it/s]
 55%|██████████████████████▉                   | 82/150 [00:24<00:19,  3.48it/s]
 55%|███████████████████████▏                  | 83/150 [00:24<00:19,  3.49it/s]
 56%|███████████████████████▌                  | 84/150 [00:24<00:20,  3.27it/s]
 57%|███████████████████████▊                  | 85/150 [00:25<00:19,  3.35it/s]
 57%|████████████████████████                  | 86/150 [00:25<00:21,  3.02it/s]
 58%|████████████████████████▎                 | 87/150 [00:25<00:21,  2.93it/s]
 59%|████████████████████████▋                 | 88/150 [00:26<00:20,  3.04it/s]
 59%|████████████████████████▉                 | 89/150 [00:26<00:21,  2.78it/s]
 60%|█████████████████████████▏                | 90/150 [00:26<00:19,  3.00it/s]
 61%|█████████████████████████▍                | 91/150 [00:27<00:20,  2.93it/s]
 61%|█████████████████████████▊                | 92/150 [00:27<00:19,  2.90it/s]
 62%|██████████████████████████                | 93/150 [00:27<00:17,  3.24it/s]
 63%|██████████████████████████▎               | 94/150 [00:28<00:16,  3.34it/s]
 63%|██████████████████████████▌               | 95/150 [00:28<00:16,  3.33it/s]
 64%|██████████████████████████▉               | 96/150 [00:28<00:17,  3.14it/s]
 65%|███████████████████████████▏              | 97/150 [00:29<00:16,  3.21it/s]
 65%|███████████████████████████▍              | 98/150 [00:29<00:16,  3.24it/s]
 66%|███████████████████████████▋              | 99/150 [00:29<00:16,  3.12it/s]
 67%|███████████████████████████▎             | 100/150 [00:30<00:17,  2.88it/s]
 67%|███████████████████████████▌             | 101/150 [00:30<00:16,  3.06it/s]
 68%|███████████████████████████▉             | 102/150 [00:30<00:14,  3.39it/s]
 69%|████████████████████████████▏            | 103/150 [00:31<00:15,  3.09it/s]
 69%|████████████████████████████▍            | 104/150 [00:31<00:14,  3.16it/s]
 70%|████████████████████████████▋            | 105/150 [00:31<00:13,  3.33it/s]
 71%|████████████████████████████▉            | 106/150 [00:31<00:11,  3.77it/s]
 71%|█████████████████████████████▏           | 107/150 [00:32<00:11,  3.59it/s]
 72%|█████████████████████████████▌           | 108/150 [00:32<00:11,  3.58it/s]
 73%|█████████████████████████████▊           | 109/150 [00:32<00:12,  3.21it/s]
 73%|██████████████████████████████           | 110/150 [00:33<00:12,  3.25it/s]
 74%|██████████████████████████████▎          | 111/150 [00:33<00:11,  3.28it/s]
 75%|██████████████████████████████▌          | 112/150 [00:33<00:11,  3.32it/s]
 75%|██████████████████████████████▉          | 113/150 [00:34<00:11,  3.17it/s]
 76%|███████████████████████████████▏         | 114/150 [00:34<00:10,  3.33it/s]
 77%|███████████████████████████████▍         | 115/150 [00:34<00:09,  3.64it/s]
 77%|███████████████████████████████▋         | 116/150 [00:34<00:10,  3.35it/s]
 78%|███████████████████████████████▉         | 117/150 [00:35<00:10,  3.06it/s]
 79%|████████████████████████████████▎        | 118/150 [00:35<00:10,  3.18it/s]
 79%|████████████████████████████████▌        | 119/150 [00:35<00:09,  3.25it/s]
 80%|████████████████████████████████▊        | 120/150 [00:36<00:09,  3.09it/s]
 81%|█████████████████████████████████        | 121/150 [00:36<00:09,  3.20it/s]
 81%|█████████████████████████████████▎       | 122/150 [00:36<00:09,  3.05it/s]
 82%|█████████████████████████████████▌       | 123/150 [00:37<00:08,  3.23it/s]
 83%|█████████████████████████████████▉       | 124/150 [00:37<00:07,  3.36it/s]
 83%|██████████████████████████████████▏      | 125/150 [00:37<00:07,  3.47it/s]
 84%|██████████████████████████████████▍      | 126/150 [00:37<00:06,  3.43it/s]
 85%|██████████████████████████████████▋      | 127/150 [00:38<00:07,  3.20it/s]
 85%|██████████████████████████████████▉      | 128/150 [00:38<00:06,  3.35it/s]
 86%|███████████████████████████████████▎     | 129/150 [00:38<00:05,  3.63it/s]
 87%|███████████████████████████████████▌     | 130/150 [00:39<00:05,  3.55it/s]
 87%|███████████████████████████████████▊     | 131/150 [00:39<00:05,  3.30it/s]
 88%|████████████████████████████████████     | 132/150 [00:39<00:05,  3.16it/s]
 89%|████████████████████████████████████▎    | 133/150 [00:40<00:05,  3.31it/s]
 89%|████████████████████████████████████▋    | 134/150 [00:40<00:04,  3.37it/s]
 90%|████████████████████████████████████▉    | 135/150 [00:40<00:04,  3.43it/s]
 91%|█████████████████████████████████████▏   | 136/150 [00:40<00:04,  3.41it/s]
 91%|█████████████████████████████████████▍   | 137/150 [00:41<00:04,  2.99it/s]
 92%|█████████████████████████████████████▋   | 138/150 [00:41<00:03,  3.20it/s]
 93%|█████████████████████████████████████▉   | 139/150 [00:41<00:03,  3.05it/s]
 93%|██████████████████████████████████████▎  | 140/150 [00:42<00:03,  3.17it/s]
 94%|██████████████████████████████████████▌  | 141/150 [00:42<00:03,  2.96it/s]
 95%|██████████████████████████████████████▊  | 142/150 [00:42<00:02,  3.06it/s]
 95%|███████████████████████████████████████  | 143/150 [00:43<00:02,  3.15it/s]
 96%|███████████████████████████████████████▎ | 144/150 [00:43<00:01,  3.21it/s]
 97%|███████████████████████████████████████▋ | 145/150 [00:43<00:01,  3.28it/s]
 97%|███████████████████████████████████████▉ | 146/150 [00:44<00:01,  3.60it/s]
 98%|████████████████████████████████████████▏| 147/150 [00:44<00:00,  3.67it/s]
 99%|████████████████████████████████████████▍| 148/150 [00:44<00:00,  4.08it/s]
 99%|████████████████████████████████████████▋| 149/150 [00:44<00:00,  4.20it/s]
                                                                                
{'eval_loss': 0.6827391386032104, 'eval_runtime': 45.2589, 'eval_samples_per_second': 3.314, 'eval_steps_per_second': 3.314, 'epoch': 2.35, 'num_input_tokens_seen': 1437056}
 78%|████████████████████████████████         | 500/639 [38:49<10:01,  4.33s/it]
100%|█████████████████████████████████████████| 150/150 [00:44<00:00,  3.98it/s]
                                                                                [INFO|trainer.py:3993] 2025-12-04 19:44:59,205 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-500
[INFO|configuration_utils.py:696] 2025-12-04 19:44:59,254 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:44:59,254 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:44:59,339 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-500/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:44:59,340 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-500/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:44:59,340 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-500/special_tokens_map.json
 79%|████████████████████████████████▍        | 505/639 [39:11<16:56,  7.59s/it][INFO|2025-12-04 19:45:20] llamafactory.train.callbacks:143 >> {'loss': 0.2051, 'learning_rate': 1.0615e-05, 'epoch': 2.37, 'throughput': 617.34}
{'loss': 0.2051, 'grad_norm': 0.7082416415214539, 'learning_rate': 1.0614601334114099e-05, 'epoch': 2.37, 'num_input_tokens_seen': 1451560, 'train_runtime': 2351.3321, 'train_tokens_per_second': 617.335}
 80%|████████████████████████████████▋        | 510/639 [39:32<10:29,  4.88s/it][INFO|2025-12-04 19:45:42] llamafactory.train.callbacks:143 >> {'loss': 0.2161, 'learning_rate': 9.8694e-06, 'epoch': 2.40, 'throughput': 617.87}
{'loss': 0.2161, 'grad_norm': 0.5055545568466187, 'learning_rate': 9.869388139903496e-06, 'epoch': 2.4, 'num_input_tokens_seen': 1466016, 'train_runtime': 2372.7021, 'train_tokens_per_second': 617.868}
 81%|█████████████████████████████████        | 515/639 [39:51<08:21,  4.05s/it][INFO|2025-12-04 19:46:01] llamafactory.train.callbacks:143 >> {'loss': 0.1847, 'learning_rate': 9.1484e-06, 'epoch': 2.42, 'throughput': 618.32}
{'loss': 0.1847, 'grad_norm': 0.6534410715103149, 'learning_rate': 9.148423840490954e-06, 'epoch': 2.42, 'num_input_tokens_seen': 1478944, 'train_runtime': 2391.883, 'train_tokens_per_second': 618.318}
 81%|█████████████████████████████████▎       | 520/639 [40:14<08:35,  4.33s/it][INFO|2025-12-04 19:46:23] llamafactory.train.callbacks:143 >> {'loss': 0.2416, 'learning_rate': 8.4521e-06, 'epoch': 2.44, 'throughput': 618.99}
{'loss': 0.2416, 'grad_norm': 0.543830156326294, 'learning_rate': 8.452144078061818e-06, 'epoch': 2.44, 'num_input_tokens_seen': 1494488, 'train_runtime': 2414.415, 'train_tokens_per_second': 618.986}
 82%|█████████████████████████████████▋       | 525/639 [40:35<08:14,  4.34s/it][INFO|2025-12-04 19:46:45] llamafactory.train.callbacks:143 >> {'loss': 0.2410, 'learning_rate': 7.7810e-06, 'epoch': 2.47, 'throughput': 619.55}
{'loss': 0.241, 'grad_norm': 0.5543383359909058, 'learning_rate': 7.780969579186814e-06, 'epoch': 2.47, 'num_input_tokens_seen': 1509128, 'train_runtime': 2435.8485, 'train_tokens_per_second': 619.549}
 83%|██████████████████████████████████       | 530/639 [40:58<08:27,  4.66s/it][INFO|2025-12-04 19:47:07] llamafactory.train.callbacks:143 >> {'loss': 0.2784, 'learning_rate': 7.1353e-06, 'epoch': 2.49, 'throughput': 620.17}
{'loss': 0.2784, 'grad_norm': 0.37712037563323975, 'learning_rate': 7.135305900598321e-06, 'epoch': 2.49, 'num_input_tokens_seen': 1524464, 'train_runtime': 2458.1382, 'train_tokens_per_second': 620.17}
 84%|██████████████████████████████████▎      | 535/639 [41:21<07:54,  4.57s/it][INFO|2025-12-04 19:47:30] llamafactory.train.callbacks:143 >> {'loss': 0.2794, 'learning_rate': 6.5155e-06, 'epoch': 2.51, 'throughput': 620.91}
{'loss': 0.2794, 'grad_norm': 0.7365447878837585, 'learning_rate': 6.515543184132999e-06, 'epoch': 2.51, 'num_input_tokens_seen': 1540688, 'train_runtime': 2481.3489, 'train_tokens_per_second': 620.907}
 85%|██████████████████████████████████▋      | 540/639 [41:41<06:39,  4.04s/it][INFO|2025-12-04 19:47:50] llamafactory.train.callbacks:143 >> {'loss': 0.2341, 'learning_rate': 5.9221e-06, 'epoch': 2.54, 'throughput': 621.39}
{'loss': 0.2341, 'grad_norm': 0.5706935524940491, 'learning_rate': 5.922055920988817e-06, 'epoch': 2.54, 'num_input_tokens_seen': 1554264, 'train_runtime': 2501.2651, 'train_tokens_per_second': 621.391}
 85%|██████████████████████████████████▉      | 545/639 [42:02<06:35,  4.21s/it][INFO|2025-12-04 19:48:11] llamafactory.train.callbacks:143 >> {'loss': 0.2178, 'learning_rate': 5.3552e-06, 'epoch': 2.56, 'throughput': 621.88}
{'loss': 0.2178, 'grad_norm': 0.7282590270042419, 'learning_rate': 5.355202725439046e-06, 'epoch': 2.56, 'num_input_tokens_seen': 1568584, 'train_runtime': 2522.3372, 'train_tokens_per_second': 621.877}
 86%|███████████████████████████████████▎     | 550/639 [42:22<05:57,  4.02s/it][INFO|2025-12-04 19:48:32] llamafactory.train.callbacks:143 >> {'loss': 0.2050, 'learning_rate': 4.8153e-06, 'epoch': 2.58, 'throughput': 622.38}
{'loss': 0.205, 'grad_norm': 0.8727412819862366, 'learning_rate': 4.8153261181398125e-06, 'epoch': 2.58, 'num_input_tokens_seen': 1582712, 'train_runtime': 2542.9866, 'train_tokens_per_second': 622.383}
 87%|███████████████████████████████████▌     | 555/639 [42:42<05:27,  3.90s/it][INFO|2025-12-04 19:48:52] llamafactory.train.callbacks:143 >> {'loss': 0.2161, 'learning_rate': 4.3028e-06, 'epoch': 2.61, 'throughput': 622.80}
{'loss': 0.2161, 'grad_norm': 0.5880314111709595, 'learning_rate': 4.302752319162212e-06, 'epoch': 2.61, 'num_input_tokens_seen': 1596192, 'train_runtime': 2562.9466, 'train_tokens_per_second': 622.796}
 88%|███████████████████████████████████▉     | 560/639 [43:04<05:49,  4.42s/it][INFO|2025-12-04 19:49:14] llamafactory.train.callbacks:143 >> {'loss': 0.2147, 'learning_rate': 3.8178e-06, 'epoch': 2.63, 'throughput': 623.33}
{'loss': 0.2147, 'grad_norm': 0.4431954324245453, 'learning_rate': 3.81779105087407e-06, 'epoch': 2.63, 'num_input_tokens_seen': 1611088, 'train_runtime': 2584.6615, 'train_tokens_per_second': 623.326}
 88%|████████████████████████████████████▎    | 565/639 [43:26<05:10,  4.20s/it][INFO|2025-12-04 19:49:35] llamafactory.train.callbacks:143 >> {'loss': 0.2420, 'learning_rate': 3.3607e-06, 'epoch': 2.65, 'throughput': 623.83}
{'loss': 0.242, 'grad_norm': 0.5944216847419739, 'learning_rate': 3.3607353507904283e-06, 'epoch': 2.65, 'num_input_tokens_seen': 1625696, 'train_runtime': 2606.0146, 'train_tokens_per_second': 623.825}
 89%|████████████████████████████████████▌    | 570/639 [43:48<05:19,  4.64s/it][INFO|2025-12-04 19:49:58] llamafactory.train.callbacks:143 >> {'loss': 0.2359, 'learning_rate': 2.9319e-06, 'epoch': 2.68, 'throughput': 624.29}
{'loss': 0.2359, 'grad_norm': 0.5675414204597473, 'learning_rate': 2.931861394505764e-06, 'epoch': 2.68, 'num_input_tokens_seen': 1641048, 'train_runtime': 2628.6622, 'train_tokens_per_second': 624.29}
 90%|████████████████████████████████████▉    | 575/639 [44:10<04:48,  4.51s/it][INFO|2025-12-04 19:50:20] llamafactory.train.callbacks:143 >> {'loss': 0.2082, 'learning_rate': 2.5314e-06, 'epoch': 2.70, 'throughput': 624.80}
{'loss': 0.2082, 'grad_norm': 0.6842307448387146, 'learning_rate': 2.531428328815155e-06, 'epoch': 2.7, 'num_input_tokens_seen': 1656160, 'train_runtime': 2650.7123, 'train_tokens_per_second': 624.798}
 91%|█████████████████████████████████████▏   | 580/639 [44:30<03:56,  4.01s/it][INFO|2025-12-04 19:50:39] llamafactory.train.callbacks:143 >> {'loss': 0.2095, 'learning_rate': 2.1597e-06, 'epoch': 2.72, 'throughput': 625.15}
{'loss': 0.2095, 'grad_norm': 0.8588398098945618, 'learning_rate': 2.1596781151249524e-06, 'epoch': 2.72, 'num_input_tokens_seen': 1669320, 'train_runtime': 2670.2655, 'train_tokens_per_second': 625.151}
 92%|█████████████████████████████████████▌   | 585/639 [44:51<03:37,  4.04s/it][INFO|2025-12-04 19:51:00] llamafactory.train.callbacks:143 >> {'loss': 0.2409, 'learning_rate': 1.8168e-06, 'epoch': 2.75, 'throughput': 625.59}
{'loss': 0.2409, 'grad_norm': 0.777917206287384, 'learning_rate': 1.8168353832477947e-06, 'epoch': 2.75, 'num_input_tokens_seen': 1683616, 'train_runtime': 2691.2351, 'train_tokens_per_second': 625.592}
 92%|█████████████████████████████████████▊   | 590/639 [45:13<03:31,  4.32s/it][INFO|2025-12-04 19:51:22] llamafactory.train.callbacks:143 >> {'loss': 0.2346, 'learning_rate': 1.5031e-06, 'epoch': 2.77, 'throughput': 626.06}
{'loss': 0.2346, 'grad_norm': 0.6048610210418701, 'learning_rate': 1.5031072956701697e-06, 'epoch': 2.77, 'num_input_tokens_seen': 1698728, 'train_runtime': 2713.3782, 'train_tokens_per_second': 626.056}
 93%|██████████████████████████████████████▏  | 595/639 [45:32<02:51,  3.90s/it][INFO|2025-12-04 19:51:41] llamafactory.train.callbacks:143 >> {'loss': 0.2282, 'learning_rate': 1.2187e-06, 'epoch': 2.80, 'throughput': 626.36}
{'loss': 0.2282, 'grad_norm': 0.5462923049926758, 'learning_rate': 1.2186834223746612e-06, 'epoch': 2.8, 'num_input_tokens_seen': 1711320, 'train_runtime': 2732.1752, 'train_tokens_per_second': 626.358}
 94%|██████████████████████████████████████▍  | 600/639 [45:54<02:50,  4.37s/it][INFO|2025-12-04 19:52:03] llamafactory.train.callbacks:143 >> {'loss': 0.2684, 'learning_rate': 9.6374e-07, 'epoch': 2.82, 'throughput': 626.84}
{'loss': 0.2684, 'grad_norm': 0.5676615238189697, 'learning_rate': 9.637356262923725e-07, 'epoch': 2.82, 'num_input_tokens_seen': 1726344, 'train_runtime': 2754.0608, 'train_tokens_per_second': 626.836}
 94%|██████████████████████████████████████▍  | 600/639 [45:54<02:50,  4.37s/it][INFO|trainer.py:4327] 2025-12-04 19:52:03,601 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:52:03,601 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:52:03,601 >>   Batch size = 1

  0%|                                                   | 0/150 [00:00<?, ?it/s]
  1%|▌                                          | 2/150 [00:00<00:19,  7.60it/s]
  2%|▊                                          | 3/150 [00:00<00:27,  5.31it/s]
  3%|█▏                                         | 4/150 [00:00<00:36,  3.96it/s]
  3%|█▍                                         | 5/150 [00:01<00:38,  3.73it/s]
  4%|█▋                                         | 6/150 [00:01<00:39,  3.62it/s]
  5%|██                                         | 7/150 [00:01<00:40,  3.52it/s]
  5%|██▎                                        | 8/150 [00:02<00:41,  3.44it/s]
  6%|██▌                                        | 9/150 [00:02<00:39,  3.56it/s]
  7%|██▊                                       | 10/150 [00:02<00:40,  3.50it/s]
  7%|███                                       | 11/150 [00:02<00:40,  3.46it/s]
  8%|███▎                                      | 12/150 [00:03<00:40,  3.43it/s]
  9%|███▋                                      | 13/150 [00:03<00:42,  3.21it/s]
  9%|███▉                                      | 14/150 [00:03<00:43,  3.10it/s]
 10%|████▏                                     | 15/150 [00:04<00:42,  3.17it/s]
 11%|████▍                                     | 16/150 [00:04<00:45,  2.97it/s]
 11%|████▊                                     | 17/150 [00:04<00:43,  3.08it/s]
 12%|█████                                     | 18/150 [00:05<00:41,  3.15it/s]
 13%|█████▎                                    | 19/150 [00:05<00:40,  3.21it/s]
 13%|█████▌                                    | 20/150 [00:05<00:41,  3.10it/s]
 14%|█████▉                                    | 21/150 [00:06<00:37,  3.42it/s]
 15%|██████▏                                   | 22/150 [00:06<00:34,  3.71it/s]
 15%|██████▍                                   | 23/150 [00:06<00:37,  3.39it/s]
 16%|██████▋                                   | 24/150 [00:06<00:37,  3.38it/s]
 17%|███████                                   | 25/150 [00:07<00:37,  3.37it/s]
 17%|███████▎                                  | 26/150 [00:07<00:35,  3.48it/s]
 18%|███████▌                                  | 27/150 [00:07<00:35,  3.50it/s]
 19%|███████▊                                  | 28/150 [00:08<00:37,  3.28it/s]
 19%|████████                                  | 29/150 [00:08<00:38,  3.13it/s]
 20%|████████▍                                 | 30/150 [00:08<00:37,  3.23it/s]
 21%|████████▋                                 | 31/150 [00:09<00:35,  3.38it/s]
 21%|████████▉                                 | 32/150 [00:09<00:31,  3.72it/s]
 22%|█████████▏                                | 33/150 [00:09<00:35,  3.32it/s]
 23%|█████████▌                                | 34/150 [00:09<00:33,  3.45it/s]
 23%|█████████▊                                | 35/150 [00:10<00:35,  3.24it/s]
 24%|██████████                                | 36/150 [00:10<00:32,  3.54it/s]
 25%|██████████▎                               | 37/150 [00:10<00:35,  3.17it/s]
 25%|██████████▋                               | 38/150 [00:11<00:34,  3.21it/s]
 26%|██████████▉                               | 39/150 [00:11<00:35,  3.10it/s]
 27%|███████████▏                              | 40/150 [00:11<00:34,  3.18it/s]
 27%|███████████▍                              | 41/150 [00:12<00:33,  3.29it/s]
 28%|███████████▊                              | 42/150 [00:12<00:32,  3.36it/s]
 29%|████████████                              | 43/150 [00:12<00:31,  3.37it/s]
 29%|████████████▎                             | 44/150 [00:13<00:35,  2.96it/s]
 30%|████████████▌                             | 45/150 [00:13<00:33,  3.11it/s]
 31%|████████████▉                             | 46/150 [00:13<00:35,  2.91it/s]
 31%|█████████████▏                            | 47/150 [00:13<00:31,  3.30it/s]
 32%|█████████████▍                            | 48/150 [00:14<00:31,  3.28it/s]
 33%|█████████████▋                            | 49/150 [00:14<00:29,  3.39it/s]
 33%|██████████████                            | 50/150 [00:14<00:29,  3.42it/s]
 34%|██████████████▎                           | 51/150 [00:15<00:28,  3.46it/s]
 35%|██████████████▌                           | 52/150 [00:15<00:32,  3.04it/s]
 35%|██████████████▊                           | 53/150 [00:15<00:29,  3.24it/s]
 36%|███████████████                           | 54/150 [00:16<00:28,  3.33it/s]
 37%|███████████████▍                          | 55/150 [00:16<00:27,  3.43it/s]
 37%|███████████████▋                          | 56/150 [00:16<00:30,  3.12it/s]
 38%|███████████████▉                          | 57/150 [00:17<00:30,  3.03it/s]
 39%|████████████████▏                         | 58/150 [00:17<00:29,  3.15it/s]
 39%|████████████████▌                         | 59/150 [00:17<00:27,  3.28it/s]
 40%|████████████████▊                         | 60/150 [00:17<00:27,  3.30it/s]
 41%|█████████████████                         | 61/150 [00:18<00:27,  3.30it/s]
 41%|█████████████████▎                        | 62/150 [00:18<00:26,  3.30it/s]
 42%|█████████████████▋                        | 63/150 [00:18<00:26,  3.30it/s]
 43%|█████████████████▉                        | 64/150 [00:19<00:24,  3.57it/s]
 43%|██████████████████▏                       | 65/150 [00:19<00:24,  3.52it/s]
 44%|██████████████████▍                       | 66/150 [00:19<00:25,  3.26it/s]
 45%|██████████████████▊                       | 67/150 [00:20<00:28,  2.90it/s]
 45%|███████████████████                       | 68/150 [00:20<00:27,  3.02it/s]
 46%|███████████████████▎                      | 69/150 [00:20<00:24,  3.35it/s]
 47%|███████████████████▌                      | 70/150 [00:20<00:23,  3.45it/s]
 47%|███████████████████▉                      | 71/150 [00:21<00:21,  3.72it/s]
 48%|████████████████████▏                     | 72/150 [00:21<00:19,  4.01it/s]
 49%|████████████████████▍                     | 73/150 [00:21<00:20,  3.79it/s]
 49%|████████████████████▋                     | 74/150 [00:21<00:19,  3.98it/s]
 50%|█████████████████████                     | 75/150 [00:22<00:19,  3.76it/s]
 51%|█████████████████████▎                    | 76/150 [00:22<00:22,  3.29it/s]
 51%|█████████████████████▌                    | 77/150 [00:22<00:20,  3.60it/s]
 52%|█████████████████████▊                    | 78/150 [00:23<00:20,  3.58it/s]
 53%|██████████████████████                    | 79/150 [00:23<00:20,  3.52it/s]
 53%|██████████████████████▍                   | 80/150 [00:23<00:19,  3.57it/s]
 54%|██████████████████████▋                   | 81/150 [00:23<00:18,  3.83it/s]
 55%|██████████████████████▉                   | 82/150 [00:24<00:19,  3.49it/s]
 55%|███████████████████████▏                  | 83/150 [00:24<00:19,  3.50it/s]
 56%|███████████████████████▌                  | 84/150 [00:24<00:20,  3.27it/s]
 57%|███████████████████████▊                  | 85/150 [00:25<00:19,  3.36it/s]
 57%|████████████████████████                  | 86/150 [00:25<00:21,  3.02it/s]
 58%|████████████████████████▎                 | 87/150 [00:25<00:21,  2.93it/s]
 59%|████████████████████████▋                 | 88/150 [00:26<00:20,  3.05it/s]
 59%|████████████████████████▉                 | 89/150 [00:26<00:21,  2.78it/s]
 60%|█████████████████████████▏                | 90/150 [00:26<00:19,  3.00it/s]
 61%|█████████████████████████▍                | 91/150 [00:27<00:20,  2.93it/s]
 61%|█████████████████████████▊                | 92/150 [00:27<00:19,  2.90it/s]
 62%|██████████████████████████                | 93/150 [00:27<00:17,  3.24it/s]
 63%|██████████████████████████▎               | 94/150 [00:28<00:16,  3.33it/s]
 63%|██████████████████████████▌               | 95/150 [00:28<00:16,  3.33it/s]
 64%|██████████████████████████▉               | 96/150 [00:28<00:17,  3.14it/s]
 65%|███████████████████████████▏              | 97/150 [00:29<00:16,  3.21it/s]
 65%|███████████████████████████▍              | 98/150 [00:29<00:16,  3.24it/s]
 66%|███████████████████████████▋              | 99/150 [00:29<00:16,  3.12it/s]
 67%|███████████████████████████▎             | 100/150 [00:30<00:17,  2.88it/s]
 67%|███████████████████████████▌             | 101/150 [00:30<00:16,  3.06it/s]
 68%|███████████████████████████▉             | 102/150 [00:30<00:14,  3.39it/s]
 69%|████████████████████████████▏            | 103/150 [00:31<00:15,  3.09it/s]
 69%|████████████████████████████▍            | 104/150 [00:31<00:14,  3.17it/s]
 70%|████████████████████████████▋            | 105/150 [00:31<00:13,  3.33it/s]
 71%|████████████████████████████▉            | 106/150 [00:31<00:11,  3.78it/s]
 71%|█████████████████████████████▏           | 107/150 [00:32<00:11,  3.59it/s]
 72%|█████████████████████████████▌           | 108/150 [00:32<00:11,  3.58it/s]
 73%|█████████████████████████████▊           | 109/150 [00:32<00:12,  3.20it/s]
 73%|██████████████████████████████           | 110/150 [00:33<00:12,  3.23it/s]
 74%|██████████████████████████████▎          | 111/150 [00:33<00:11,  3.27it/s]
 75%|██████████████████████████████▌          | 112/150 [00:33<00:11,  3.31it/s]
 75%|██████████████████████████████▉          | 113/150 [00:34<00:11,  3.16it/s]
 76%|███████████████████████████████▏         | 114/150 [00:34<00:10,  3.32it/s]
 77%|███████████████████████████████▍         | 115/150 [00:34<00:09,  3.63it/s]
 77%|███████████████████████████████▋         | 116/150 [00:34<00:10,  3.35it/s]
 78%|███████████████████████████████▉         | 117/150 [00:35<00:10,  3.05it/s]
 79%|████████████████████████████████▎        | 118/150 [00:35<00:10,  3.18it/s]
 79%|████████████████████████████████▌        | 119/150 [00:35<00:09,  3.25it/s]
 80%|████████████████████████████████▊        | 120/150 [00:36<00:09,  3.09it/s]
 81%|█████████████████████████████████        | 121/150 [00:36<00:09,  3.20it/s]
 81%|█████████████████████████████████▎       | 122/150 [00:36<00:09,  3.06it/s]
 82%|█████████████████████████████████▌       | 123/150 [00:37<00:08,  3.23it/s]
 83%|█████████████████████████████████▉       | 124/150 [00:37<00:07,  3.37it/s]
 83%|██████████████████████████████████▏      | 125/150 [00:37<00:07,  3.47it/s]
 84%|██████████████████████████████████▍      | 126/150 [00:37<00:06,  3.43it/s]
 85%|██████████████████████████████████▋      | 127/150 [00:38<00:07,  3.19it/s]
 85%|██████████████████████████████████▉      | 128/150 [00:38<00:06,  3.33it/s]
 86%|███████████████████████████████████▎     | 129/150 [00:38<00:05,  3.63it/s]
 87%|███████████████████████████████████▌     | 130/150 [00:39<00:05,  3.55it/s]
 87%|███████████████████████████████████▊     | 131/150 [00:39<00:05,  3.30it/s]
 88%|████████████████████████████████████     | 132/150 [00:39<00:05,  3.16it/s]
 89%|████████████████████████████████████▎    | 133/150 [00:40<00:05,  3.31it/s]
 89%|████████████████████████████████████▋    | 134/150 [00:40<00:04,  3.36it/s]
 90%|████████████████████████████████████▉    | 135/150 [00:40<00:04,  3.41it/s]
 91%|█████████████████████████████████████▏   | 136/150 [00:40<00:04,  3.41it/s]
 91%|█████████████████████████████████████▍   | 137/150 [00:41<00:04,  2.97it/s]
 92%|█████████████████████████████████████▋   | 138/150 [00:41<00:03,  3.19it/s]
 93%|█████████████████████████████████████▉   | 139/150 [00:42<00:03,  3.04it/s]
 93%|██████████████████████████████████████▎  | 140/150 [00:42<00:03,  3.16it/s]
 94%|██████████████████████████████████████▌  | 141/150 [00:42<00:03,  2.95it/s]
 95%|██████████████████████████████████████▊  | 142/150 [00:42<00:02,  3.05it/s]
 95%|███████████████████████████████████████  | 143/150 [00:43<00:02,  3.14it/s]
 96%|███████████████████████████████████████▎ | 144/150 [00:43<00:01,  3.21it/s]
 97%|███████████████████████████████████████▋ | 145/150 [00:43<00:01,  3.27it/s]
 97%|███████████████████████████████████████▉ | 146/150 [00:44<00:01,  3.60it/s]
 98%|████████████████████████████████████████▏| 147/150 [00:44<00:00,  3.67it/s]
 99%|████████████████████████████████████████▍| 148/150 [00:44<00:00,  4.08it/s]
 99%|████████████████████████████████████████▋| 149/150 [00:44<00:00,  4.20it/s]
                                                                                
{'eval_loss': 0.6894978284835815, 'eval_runtime': 45.301, 'eval_samples_per_second': 3.311, 'eval_steps_per_second': 3.311, 'epoch': 2.82, 'num_input_tokens_seen': 1726344}
 94%|██████████████████████████████████████▍  | 600/639 [46:39<02:50,  4.37s/it]
100%|█████████████████████████████████████████| 150/150 [00:45<00:00,  3.98it/s]
                                                                                [INFO|trainer.py:3993] 2025-12-04 19:52:48,899 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-600
[INFO|configuration_utils.py:696] 2025-12-04 19:52:48,948 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:52:48,949 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:52:49,046 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-600/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:52:49,047 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-600/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:52:49,047 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-600/special_tokens_map.json
 95%|██████████████████████████████████████▊  | 605/639 [47:00<04:06,  7.24s/it][INFO|2025-12-04 19:53:09] llamafactory.train.callbacks:143 >> {'loss': 0.2441, 'learning_rate': 7.3842e-07, 'epoch': 2.84, 'throughput': 617.14}
{'loss': 0.2441, 'grad_norm': 0.8173639178276062, 'learning_rate': 7.384179594548957e-07, 'epoch': 2.84, 'num_input_tokens_seen': 1740472, 'train_runtime': 2820.2348, 'train_tokens_per_second': 617.137}
 95%|███████████████████████████████████████▏ | 610/639 [47:20<02:12,  4.56s/it][INFO|2025-12-04 19:53:30] llamafactory.train.callbacks:143 >> {'loss': 0.2168, 'learning_rate': 5.4287e-07, 'epoch': 2.87, 'throughput': 617.50}
{'loss': 0.2168, 'grad_norm': 0.5683459639549255, 'learning_rate': 5.428665699084789e-07, 'epoch': 2.87, 'num_input_tokens_seen': 1754136, 'train_runtime': 2840.7246, 'train_tokens_per_second': 617.496}
 96%|███████████████████████████████████████▍ | 615/639 [47:42<01:46,  4.44s/it][INFO|2025-12-04 19:53:51] llamafactory.train.callbacks:143 >> {'loss': 0.2421, 'learning_rate': 3.7720e-07, 'epoch': 2.89, 'throughput': 618.01}
{'loss': 0.2421, 'grad_norm': 0.5586677193641663, 'learning_rate': 3.7719961944664985e-07, 'epoch': 2.89, 'num_input_tokens_seen': 1768960, 'train_runtime': 2862.3719, 'train_tokens_per_second': 618.005}
 97%|███████████████████████████████████████▊ | 620/639 [48:02<01:16,  4.05s/it][INFO|2025-12-04 19:54:12] llamafactory.train.callbacks:143 >> {'loss': 0.2027, 'learning_rate': 2.4152e-07, 'epoch': 2.91, 'throughput': 618.39}
{'loss': 0.2027, 'grad_norm': 0.7121002674102783, 'learning_rate': 2.415172122110343e-07, 'epoch': 2.91, 'num_input_tokens_seen': 1782728, 'train_runtime': 2882.8828, 'train_tokens_per_second': 618.384}
 98%|████████████████████████████████████████ | 625/639 [48:24<00:59,  4.24s/it][INFO|2025-12-04 19:54:34] llamafactory.train.callbacks:143 >> {'loss': 0.2514, 'learning_rate': 1.3590e-07, 'epoch': 2.94, 'throughput': 618.89}
{'loss': 0.2514, 'grad_norm': 0.6133226752281189, 'learning_rate': 1.3590133420350315e-07, 'epoch': 2.94, 'num_input_tokens_seen': 1797592, 'train_runtime': 2904.5693, 'train_tokens_per_second': 618.884}
 99%|████████████████████████████████████████▍| 630/639 [48:45<00:38,  4.25s/it][INFO|2025-12-04 19:54:55] llamafactory.train.callbacks:143 >> {'loss': 0.2103, 'learning_rate': 6.0416e-08, 'epoch': 2.96, 'throughput': 619.38}
{'loss': 0.2103, 'grad_norm': 0.6335474848747253, 'learning_rate': 6.041580374618328e-08, 'epoch': 2.96, 'num_input_tokens_seen': 1812104, 'train_runtime': 2925.6652, 'train_tokens_per_second': 619.382}
 99%|████████████████████████████████████████▋| 635/639 [49:05<00:15,  3.93s/it][INFO|2025-12-04 19:55:14] llamafactory.train.callbacks:143 >> {'loss': 0.2202, 'learning_rate': 1.5106e-08, 'epoch': 2.98, 'throughput': 619.76}
{'loss': 0.2202, 'grad_norm': 0.7536317110061646, 'learning_rate': 1.5106232919276375e-08, 'epoch': 2.98, 'num_input_tokens_seen': 1825320, 'train_runtime': 2945.211, 'train_tokens_per_second': 619.759}
100%|█████████████████████████████████████████| 639/639 [49:19<00:00,  3.48s/it][INFO|trainer.py:3993] 2025-12-04 19:55:29,244 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-639
[INFO|configuration_utils.py:696] 2025-12-04 19:55:29,292 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:55:29,293 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:55:29,378 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-639/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:55:29,378 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-639/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:55:29,378 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-639/special_tokens_map.json
[INFO|trainer.py:2676] 2025-12-04 19:55:29,734 >>

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 2960.1995, 'train_samples_per_second': 0.861, 'train_steps_per_second': 0.216, 'train_loss': 0.40086729491074136, 'epoch': 3.0, 'num_input_tokens_seen': 1835184}
100%|█████████████████████████████████████████| 639/639 [49:20<00:00,  4.63s/it]
[INFO|trainer.py:3993] 2025-12-04 19:55:29,736 >> Saving model checkpoint to saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58
[INFO|configuration_utils.py:696] 2025-12-04 19:55:29,785 >> loading configuration file /mnt/workspace/Qwen3-8B/config.json
[INFO|configuration_utils.py:770] 2025-12-04 19:55:29,786 >> Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.52.4",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

[INFO|tokenization_utils_base.py:2356] 2025-12-04 19:55:29,870 >> chat template saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/chat_template.jinja
[INFO|tokenization_utils_base.py:2525] 2025-12-04 19:55:29,871 >> tokenizer config file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-12-04 19:55:29,871 >> Special tokens file saved in saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/special_tokens_map.json
***** train metrics *****
  epoch                    =        3.0
  num_input_tokens_seen    =    1835184
  total_flos               = 77836961GF
  train_loss               =     0.4009
  train_runtime            = 0:49:20.19
  train_samples_per_second =      0.861
  train_steps_per_second   =      0.216
Figure saved at: saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/training_loss.png
Figure saved at: saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/training_eval_loss.png
[WARNING|2025-12-04 19:55:30] llamafactory.extras.ploting:148 >> No metric eval_accuracy to plot.
[INFO|trainer.py:4327] 2025-12-04 19:55:30,222 >>
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-12-04 19:55:30,222 >>   Num examples = 150
[INFO|trainer.py:4332] 2025-12-04 19:55:30,222 >>   Batch size = 1
100%|█████████████████████████████████████████| 150/150 [00:44<00:00,  3.34it/s]
***** eval metrics *****
  epoch                   =        3.0
  eval_loss               =      0.688
  eval_runtime            = 0:00:45.23
  eval_samples_per_second =      3.316
  eval_steps_per_second   =      3.316
  num_input_tokens_seen   =    1835184
[INFO|modelcard.py:450] 2025-12-04 19:56:15,453 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
```



#### 3.3.1 训练效率与稳定性深度解析 (Efficiency & Stability)

这部分主要评估您的硬件资源是否被充分利用，以及训练过程是否健康。

- **硬件适配度：满分**
  - **显存状态**：日志全称无 OOM (Out Of Memory) 报错，说明 `4-bit 量化` + `梯度检查点` 的策略非常成功。在 A10 (24G) 显卡上，显存占用被牢牢控制在安全线以内。
  - **计算精度**：日志显示 `compute dtype: torch.bfloat16`。A10 显卡原生支持 `bf16`，这比 `fp16` 更稳定，能有效防止训练中出现 Loss=NaN（数值溢出）的灾难性错误。
- **训练速度：极快**
  - **吞吐量**：稳定在 **620 - 630 tokens/s**。对于 8B 参数量的模型，在单卡上达到这个速度属于**第一梯队**的效率。
  - **样本处理**：约 `0.86 samples/s`。这意味着处理一条几千字的复杂医疗推理数据，只需要 1.2 秒左右。
- **结论**：您的环境配置（PyTorch 2.6 + CUDA 12.4 + A10）非常健壮，完全可以支撑后续更大规模的训练。

#### 3.3.2 损失函数与过拟合深度诊断 (Loss Analysis) —— **最关键的发现**
> 输出目录（在界面上显示的 train_2025-12-04-19-04-58），路径通常是： saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/
![image-20251204201640965](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042016031.png)
![image-20251204201557168](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042015285.png)

这是整个日志中最有价值的数据，它告诉我们模型到底学得怎么样。请重点关注 **Eval Loss (模拟考试成绩)** 的变化轨迹。

- **第一阶段：快速学习期 (Step 0 - 200)**
  - **表现**：Train Loss 迅速下降，Eval Loss 也从 `0.5516` 降至 **`0.5446`** (Step 200)。
  - **解读**：此时模型正在像海绵一样吸收知识，它学会了如何模仿医生的语气，也掌握了基本的推理逻辑。
  - **状态**：**最佳 (Optimal)**。
- **第二阶段：死记硬背期 (Step 200 - 400)**
  - **表现**：Train Loss 继续下降（平时作业全对），但 Eval Loss 开始反弹至 `0.57` - `0.60`。
  - **解读**：模型开始为了刷低训练集的 Loss，去死记硬背某些特定的病历描述，而不是理解其中的医理。它开始“偏科”了。
- **第三阶段：严重过拟合期 (Step 400 - 639)**
  - **表现**：Train Loss 降到了极低的 `0.21`，但 Eval Loss 飙升至 **`0.688`**，甚至比刚开始训练时还要差。
  - **解读**：模型彻底“学傻了”。如果你问它训练集里一模一样的问题，它能答得很完美；但如果你问一个新的病人情况，它可能会胡言乱语，因为它的通用能力被破坏了。

#### 3.3.3 学习内容与数据质量验证 (Data Quality Verification)

日志中有一段数据的 Input/Output 预览，这证实了您的数据清洗工作是成功的。

- **标签识别**：日志显示 `labels: <think>从中医的角度来看...</think>`。
- **深度解读**：
  - 模型不仅学到了“答案”，还学到了被 `<think>` 标签包裹的**“隐式思维链”**。
  - 这意味着在推理时，模型会先在内部进行一步步的逻辑推导（比如先分析病因，再定性，最后给方案），这正是 OpenAI o1 等推理模型的核心特性。

#### 3.3.4 最佳模型锁定 (Model Selection)

这是新手最容易犯错的地方：**千万不要以为跑完最后一步的模型就是最好的。**

- **当前陷阱**：默认输出的 `checkpoint-639` 是一个过拟合的模型，泛化能力很差。
- **正确选择**：请回溯到 **`checkpoint-200`**。
- **操作建议**：在 WebUI 中加载模型时，请手动选择 `checkpoint-200` 文件夹。如果可能，建议您把这个文件夹单独复制出来备份，重命名为 `Qwen3-Medical-Best`。

------

#### 3.3.5 核心指标总结表 (Summary Table)

为了方便您存档和对比，我为您总结了这份详细的训练报告表：

| **核心维度**   | **细分指标** | **结果数值/状态**      | **初学者解读与建议**                                         |
| -------------- | ------------ | ---------------------- | ------------------------------------------------------------ |
| **硬件与环境** | 显卡资源     | A10 (24G) 单卡         | **完美**。显存占用合理，无溢出，利用率高。                   |
|                | 计算精度     | bfloat16 (bf16)        | **优秀**。比 fp16 更稳定，适合 A10 架构。                    |
|                | 量化策略     | 4-bit (QLoRA)          | **关键**。这是实现 620+ tokens/s 高速训练的核心原因。        |
| **数据情况**   | 数据总量     | 850 条                 | **偏少**。数据越少，越容易过拟合（本次 Step 200 就过拟合了）。 |
|                | 数据格式     | CoT (思维链)           | **正确**。成功识别 `<think>` 标签，模型学会了推理步骤。      |
| **训练表现**   | 总耗时       | 49 分钟                | 效率很高，适合快速迭代实验。                                 |
|                | 最佳轮数     | Epoch ~0.94 (Step 200) | **重要知识点**：并不是轮数越多越好，对于小数据，1 轮往往就够了。 |
| **模型效果**   | 最佳 Loss    | **0.5446** (验证集)    | 出现在 **Step 200**。这是模型“智力”的巅峰时刻。              |
|                | 最终 Loss    | 0.6880 (验证集)        | 出现在结束时。说明模型后期在“负优化”。                       |
| **最终决策**   | **最佳存档** | **checkpoint-200**     | **请使用此版本！** 它是本次训练的精华。                      |
|                | **避坑存档** | checkpoint-639         | 已过拟合，建议仅作对比研究，不建议用于生产。                 |
| **后续优化**   | 策略调整     | 减少 Epoch / 增加数据  | 建议下次将 `Epoch` 设为 1，或者将数据扩充到 3000 条以上。    |

**一句话总结**：这是一次技术上非常成功的运行，环境与参数配置完美，但由于数据量较少（850条），模型在第 1 轮（Step 200）就达到了最佳状态，后续的训练属于过度学习。请使用 **Checkpoint-200** 作为最终成果。

## 4. 模型评估


> 如果训练时候，有配置评估数据集，其实已经评估过，可以跳过

> 注意：训练数据集和评估集不要一样（因为训练时候已经评估过，这里只是为了演示）

微调完成后，点击页面顶部的「检查点路径」，然后点击检查点路径路径，即可弹出刚刚训练完成的LoRA权重，点击选择下拉列表中的选项，在模型启动时即可加载微调结果。

选择「Evaluate&Predict」栏，在数据集下拉列表中选择「eval」（验证集）评估模型。可以手动更改输出目录为，模型评估结果将会保存在该目录中。最后点击开始按钮启动模型评估。
![image-20251204215633395](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042156629.png)



模型评估大约需要5分钟左右，评估完成后会在界面上显示验证集的分数。其中ROUGE分数衡量了模型输出答案（predict）和验证集中标准答案（label）的相似度，ROUGE分数越高代表模型学习得更好。
![image-20251205183330224](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512051833451.png)


| **特性**         | **BLEU**                                          | **ROUGE**                                         |
| ---------------- | ------------------------------------------------- | ------------------------------------------------- |
| **全称**         | Bilingual Evaluation Understudy                   | Recall-Oriented Understudy for Gisting Evaluation |
| **侧重点**       | **精确率 (Precision)** 我说的每一句话是否都正确？ | **召回率 (Recall)** 该说的重点我是否都说到了？    |
| **惩罚机制**     | 惩罚“胡编乱造”（多余的错误词）                    | 惩罚“遗漏信息”（没提到的关键词）                  |
| **最擅长领域**   | 机器翻译、代码生成                                | 文本摘要、问答系统（QA）、阅读理解                |
| **你的微调场景** | 衡量医疗术语是否严谨，没有乱用词。                | 衡量医疗诊断是否遗漏了关键的治疗措施。            |
### 4.1. 质量评估指标（核心关注）

这些分数主要反映模型生成的回答与“标准答案”有多像。分数越高，代表微调效果越好。

| 指标名称 | 数值 | 核心含义 | 详细解释与评价 |
| :--- | :--- | :--- | :--- |
| **predict\_bleu-4** | **32.18** | **“连词”精准度** | **评价：优秀**<br>它检查模型是否能连续说对4个词。在生成任务中，超过30分通常意味着模型不仅逻辑通顺，而且**专业术语、成语、固定搭配**（如“子宫收缩乏力”）使用得非常准确，没有出现严重的胡言乱语。 |
| **predict\_rouge-1** | **58.75** | **关键词覆盖率** | **评价：非常优秀**<br>它衡量标准答案里的字/词，有多少被模型说出来了。接近60分说明**核心信息点丢失极少**。比如标准答案里有“止血、输血、抗生素”，模型基本上全覆盖了。 |
| **predict\_rouge-2** | **34.10** | **短语匹配度** | **评价：良好**<br>它衡量连续两个词的匹配情况。比单纯的关键词匹配更难，它说明模型不仅记住了词，还学会了正确的**修饰关系**（例如是“预防-感染”而不是“治疗-感染”）。 |
| **predict\_rouge-l** | **43.00** | **句子结构相似度** | **评价：优秀**<br>它衡量句子级别的逻辑结构。43分说明模型的**叙述逻辑**（如：先诊断 -\> 后治疗 -\> 再预防）与标准答案高度一致，这正是你看到的“医嘱格式”对齐的体现。 |

-----

### 4.2. 性能与效率指标（参考关注）

这些指标与模型的“智商”无关，只与你的硬件（GPU）和参数设置有关。

| 指标名称 | 数值 | 核心含义 | 详细解释与评价 |
| :--- | :--- | :--- | :--- |
| **predict\_samples\_per\_second** | **0.114** | **推理速度** | **评价：较慢**<br>意味着每秒只能处理0.114条数据，换算下来处理一条数据大约需要 **8.7秒**。 |
| **predict\_runtime** | **877.78** | **总耗时** | **评价：正常**<br>评估完这100条数据（你设置的验证集大小）总共花了877秒，约14.6分钟。 |
| **predict\_model\_preparation\_time** | **0.0061** | **模型准备时间** | **评价：极快**<br>加载模型到显存准备开始推理的时间，几乎可以忽略不计。 |

-----

### 4.3. 综合专家点评

**结论：这是一个非常成功的微调（SFT）。**

**为什么这么说？**
在开放域对话（Open Chat）微调中，BLEU-4 能达到 20-25 分就很不错了。你的模型在医疗数据集上跑出了 **32+** 的分数，且 ROUGE-1 接近 **60**，这说明：

1.  **强对齐（Strong Alignment）：** 模型非常听话，它几乎是在“复刻”你提供的数据集风格。
2.  **低幻觉（Low Hallucination）：** 因为 ROUGE 分数很高，说明模型没有大量编造标准答案以外的废话。
3.  **关于速度：** 推理速度较慢（8秒/条）可能是因为你设置的 `最大生成长度` (2048) 较长，或者是因为使用了 A10 显卡进行 FP16/BF16 推理时的正常负载表现。这不影响模型质量，只影响生产效率。

**一句话总结：** 你的模型现在就像一个**背书很熟练的医学生**，能够按照教科书（数据集）的标准规范来回答问题。

## 5. 模型对话

选择「Chat」栏，确保适配器路径是`checkpoint-200`（评估效果最好的检查点），点击「加载模型」即可在Web UI中和微调模型进行对话。

在页面底部的对话框输入想要和模型对话的内容，点击「提交」即可发送消息。发送后模型会逐字生成回答，从回答中可以发现模型学习到了数据集中的内容，能够恰当地模仿诸葛亮的语气对话。
这四个关键配置项。是进入“模型对话（Chat）”界面的入口，配置正确与否直接决定了您能否与刚刚训练好的“最佳模型”进行交互。

### 5.1 模型名称 (Model Name)

- **当前设置**：`Qwen3-8B-Instruct`
- **参数解释**：这是给 LLaMA-Factory 的一个“暗号”。
  - 它告诉系统：“我要加载的模型是 Qwen3 系列的 Instruct 版本”。
  - 系统收到这个暗号后，会自动去查找对应的**模型结构模板**（比如知道 Qwen3 用的是 `swiglu` 激活函数，用的是 `rope` 位置编码等）。
  - **注意**：这个名称必须与您训练时选择的名称保持一致，否则可能会因为模板不匹配导致加载失败。

### 5.2 模型路径 (Model Path)

- **当前设置**：`/mnt/workspace/Qwen3-8B`
- **参数解释**：这是基座模型（Base Model）权重的物理存储位置。
- **形象比喻**：
  - **模型路径** = 原版教科书（Qwen3 基座）。
  - **检查点路径** = 您做的医疗笔记（LoRA 权重）。
  - **加载过程** = 系统先打开“教科书”，然后把您的“笔记”贴上去，组合成一个懂医疗的专家模型。
- **常见错误**：千万不要把这里填成 LoRA 的路径，否则系统会因为找不到基座模型的配置文件（config.json）而报错。

### 5.3 检查点路径 (Checkpoint Path)

- 当前设置：

  `/mnt/workspace/LLaMA-Factory/saves/Qwen3-8B-Instruct/lora/train_2025-12-04-19-04-58/checkpoint-200`

- **参数解释**：这是您刚刚训练出来的“医疗外挂大脑”的存放位置。

- **为什么是这个路径**：

  - 回顾之前的分析，您的模型在 **Step 200** 时效果最好（验证集 Loss 最低），之后就过拟合了。
  - 您已经成功执行了 Python 脚本或 Shell 命令，将 `checkpoint-200` 从深层目录中提取出来（或者直接选中了该子文件夹），这非常关键。
  - **如果不选**：WebUI 默认只会加载基座模型（Qwen3），那您之前的 49 分钟就白练了，模型还是那个不懂医疗的通用模型。

### 5.4 推理引擎 (Inference Engine)

- **当前设置**：`huggingface`
- **参数解释**：这是决定“谁来负责计算”的核心选项。
  - **huggingface**：
    - **是什么**：使用 PyTorch 原生的 `transformers` 库进行推理。
    - **特点**：**兼容性之王**。它几乎支持所有模型架构和 LoRA 权重，虽然速度不如 vLLM 极致，但在加载 LoRA 适配器（尤其是您刚刚手动提取的 `checkpoint-200`）时最不容易报错。
    - **为什么选它**：之前尝试 `vllm` 时遇到了 `LoRALRUCache` 的兼容性错误（AttributeError），切换到 `huggingface` 是最稳妥的解决之道。
  - **vLLM**：
    - **是什么**：一个追求极速推理的引擎，通常用于生产环境部署。
    - **缺点**：对 LoRA 的动态加载支持有时不稳定，且对显卡驱动和 CUDA 版本要求更严苛。

![image-20251204211702449](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042117609.png)



### 5.5 微调模型-对话测试

基于我们的 `medical-o1-reasoning-SFT` 数据集样例，这些数据具有很强的**“病例描述 -> 逐步推理 -> 最终诊断/方案”**的特征。

为了验证模型是否真的学会了**“思维链（CoT）”**推理能力，而不仅仅是死记硬背，我设计了 **3 类** 测试提示词（Prompts）。

建议你在 Chat 界面中输入这些问题，并**重点观察模型输出中是否包含 `<think>` 标签或明显的推理过程**。

------

#### 第一类：中医辨证推理测试（验证逻辑复现）

*这类数据在你的训练集中占比较大（如蝼蛄疖、肺阴虚等）。*

**测试输入 1：**

> 患者表现为久咳不止，痰少而粘，不易咳出，口鼻干燥，咽喉干痛，午后潮热，颧红，盗汗，舌红少苔，脉细数。请问该患者的中医辨证是什么？请逐步推理并给出答案。

**预期模型表现：**

1. **推理过程 (`<think>`)**：模型应分析“痰少而粘、口鼻干燥”指向**阴虚**（津液不足）；“午后潮热、盗汗、颧红、脉细数”指向**内热**（阴虚生内热）；病位在肺（咳嗽）。

2. **最终结论**：**肺阴亏耗**（或肺阴虚证）。

   ![image-20251204210831894](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042108984.png)

   ![image-20251204210917029](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042109178.png)

------

#### 第二类：急诊/临床处置决策测试（验证优先级判断）

*对应数据集中“急性左心衰”、“高血压急症”等案例。*

**测试输入 2：**

> 一位65岁男性患者，既往有冠心病史。突发胸骨后压榨性疼痛2小时，伴大汗淋漓、恶心呕吐。心电图显示II、III、aVF导联ST段弓背向上抬高。此时首选的再灌注治疗策略是什么？请逐步推理。

**预期模型表现：**

1. **推理过程 (`<think>`)**：首先识别症状（胸痛、大汗）和心电图特征（II, III, aVF抬高）诊断为**急性下壁心肌梗死**。接着判断时间窗（发病2小时），属于再灌注治疗的最佳窗口期。比较PCI（支架）和溶栓。如果有条件，PCI是首选；无条件则溶栓。

2. **最终结论**：首选**急诊PCI**（经皮冠状动脉介入治疗）。

   ![image-20251204211850523](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042118603.png)

   ![image-20251204211934793](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042119983.png)

------

#### 第三类：复杂外科/妇产科鉴别（验证因果分析）

*对应数据集中“产后出血”、“阑尾包块”等案例。*

**测试输入 3：**

> 一位初产妇，胎儿娩出后10分钟，出现阴道流血，色暗红，且胎盘未娩出。检查发现子宫轮廓不清，按压宫底有大量血块涌出。请推断出血原因及处理原则。

**预期模型表现：**

1. **推理过程 (`<think>`)**：分析出血时机（胎儿娩出后，胎盘未娩出前）。分析出血特征（暗红，非鲜红，排除软产道裂伤）。分析体征（子宫轮廓不清 -> 宫缩乏力？或者胎盘剥离不全？）。由于胎盘未娩出，首先考虑**胎盘因素**（滞留或剥离不全）导致的出血，继发宫缩乏力。

2. **最终结论**：**胎盘因素**（胎盘滞留/剥离不全）。处理原则是**协助胎盘娩出**（如手取胎盘）并加强宫缩。

   ![image-20251204212040659](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042120821.png)

   ![image-20251204212206132](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042122202.png)

------

#### 🟢 如何判断模型是否“聪明”？

在你加载了 `checkpoint-200` 后，观察回答：

1. **满分回答**：
   - 输出包含 `<think>...</think>` 内容（如果 WebUI 渲染了 HTML 标签可能看不见，但能看到一段明显的分析文本）。
   - 推理逻辑是：**症状提取 -> 病理机制分析 -> 排除其他可能 -> 锁定诊断 -> 给出方案**。
   - 这说明模型学到了数据集的**思维逻辑 (Reasoning)**。
2. **及格回答**：
   - 直接给出了正确答案，但没有详细的推理过程。
   - 这说明模型学到了**知识 (Knowledge)**，但丢失了**思维链 (CoT)**，可能是 Epoch 训练过多导致的轻微过拟合，或者是 Prompt 没有触发推理。
3. **失败回答**：
   - 答非所问，或者像复读机一样重复问题。
   - 这是严重的过拟合或训练失败（通常出现在 checkpoint-639）。

### 5.6 卸载模型：与微调前的原始模型聊天
点击「卸载模型」，`检查点路径`为空，再次点击「加载模型」，即可与微调前的原始模型聊天。
![image-20251205183418359](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512051834516.png)
重新向模型发送相同的内容。

![image-20251204214828861](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202512042148948.png)

发现差异不大，其实是**好事**，说明微调没有破坏模型的知识准确性。具体原因如下：

1. 医学答案的唯一性（保底线）

   治疗方案（如缩宫素、按摩子宫）是标准医学知识，核心内容必须一致。如果微调后瞎编出一种新疗法，反而是严重的“幻觉”。

2. **LoRA 改的是“规矩”而非“知识”**

   - **图1（微调前）**：像百科全书，泛泛而谈，罗列所有知识。
   - **图2（微调后）**：像**临床医生**。它结合了**具体病例**（“根据描述...”），且排版变成了更清晰的**医嘱格式**（加粗关键词、优先级排序）。

**结论**：微调生效了。模型从“背书模式”切换到了“看病模式”，这正是你想要的垂直领域对齐（Alignment）。


## 6. 总结

本次教程介绍了如何使用PAI和LLaMA Factory框架，基于轻量化LoRA方法微调Qwen3模型，使其能够进行中文问答和角色扮演，同时通过验证集ROUGE分数和人工测试验证了微调的效果。在后续实践中，可以使用实际业务数据集，对模型进行微调，得到能够解决实际业务场景问题的本地领域大模型。